## Importazione librerie

In [1]:
import string
import sys
import io 
import os
import nltk
import pandas as pd
import numpy as np
import os
from collections import Counter
from gensim.models import Word2Vec
from cade.cade import CADE
import treetaggerwrapper

C:\ProgramData\Anaconda3\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
C:\ProgramData\Anaconda3\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
C:\ProgramData\Anaconda3\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
C:\ProgramData\Anaconda3\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


### Definizione path

In [2]:
path_storico_w2v = "C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Models/periodi_storici/frasi_NON_lemmatizzate/w2v"

path_letterario_w2v = "C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Models/fasi_letterarie/frasi_NON_lemmatizzate/w2v"

path_autori_w2v = "C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Models/autori/frasi_NON_lemmatizzate/w2v"

path_storico_cade = "C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Models/periodi_storici/frasi_NON_lemmatizzate/CADE/CADE_5"

path_letterario_cade = "C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Models/fasi_letterarie/frasi_NON_lemmatizzate/CADE/CADE_5"

path_autori_cade = "C:/Users/cafe6/Università degli Studi di Milano-Bicocca/g.carbone8@campus.unimib.it - data/Models/autori/frasi_NON_lemmatizzate/CADE/CADE_5"

### Definizione funzioni

In [3]:
# Funzione per calcolare la frequenza dei topos nei modelli

def topos_frequencies(words: list, n_mod: int, text: str):
    
    embeddings = []
    for k in range(n_mod):
        model = Word2Vec.load(text + '_' + str(k) + '.model') #adattare nome
        embeddings.append(model) #ottengo una lista contenente gli n modelli per il testo desiderato
    
    topos_freq = dict()
    
    for k in range(n_mod):
        for word in words:
            if word not in topos_freq.keys():
                topos_freq[word] = embeddings[k].wv.get_vecattr(word, "count")
            else:
                topos_freq[word] += embeddings[k].wv.get_vecattr(word, "count")
    
    return topos_freq

In [4]:
# Funzione che estrae le n parole più simili ad una parola target, calcolando la similarità media per un dato numero di modelli

def similar_words_per_topos(word, n_mod, n_words, text):
        #parola target, numero di modelli, numero di parole simili da estrarre, corpus da analizzare
    text_2 = text
    text = text.lower()
    
    #ottengo gli n modelli del testo considerato
    embeddings = []
    for k in range(n_mod):
        model = Word2Vec.load(text + '_' + str(k) + '.model') #adattare nome
        embeddings.append(model) #ottengo una lista contenente gli n modelli per il testo desiderato
        
    #modello per modello, ottengo la lista delle m parole (n_words) più simili al target
    words = []
    for k in range(n_mod):
        for tupla in embeddings[k].wv.most_similar(word)[:n_words]: #ottengo una lista di tuple (parola, similarità)
            words.append(tupla[0]) #appendo solamente la parola, senza considerare il valore di similarità rispetto al target
        
    #creo un dizionario che presenta le parole uniche contenute nella lista "words", e le rispettive frequenze assolute
    c = Counter(words)
    word_frequencies = dict(c)
    
    #creo un dizionario che contiene la lista di parole uniche in "words" e calcola la similarità media di ciascuna di esse
    init = [0] * len(np.unique(words))
    word_similarities = dict(zip(np.unique(words), init))

    for k in range(n_mod):
        for tupla in embeddings[k].wv.most_similar(word)[:n_words]:
            word_similarities[tupla[0]] += tupla[1]
    for word in np.unique(words):
        word_similarities[word] = word_similarities[word] / word_frequencies[word]

    #trasformo il dizionario con le frequenze assolute e quello con le similarità medie in dataframe, e li unisco
    df_1 = pd.DataFrame(list(word_frequencies.items()), columns=['Words','Abs.frequency'])
    df_2 = pd.DataFrame(list(word_similarities.items()), columns=['Words','Med.similarity'])
  
    #calcolo anche la similarità normalizzata, nel caso dovesse servie
    normalized_similarity = (df_2['Med.similarity']-df_2['Med.similarity'].min())/(df_2['Med.similarity'].max() - df_2['Med.similarity'].min()) 
    df = pd.merge(df_1, df_2, on = 'Words')
    df['Med.similarity(norm.)'] = normalized_similarity

    #aggiungo la colonna col nome del testo considerato
    df['Text'] = [text_2] * len(df)
    
    return df

In [5]:
# Funzioni per estrarre, per una parola target, le parole più simili appartenenti a una specifica categoria grammaticale (nomi,
# aggettivi, ...)

def find_similar(word: str, n_similar: int, n_mod:int, pos_tag_prefix: str, models: list) -> list:
    #parola da analizzare, numero di parole simili volute, numero modelli, prefisso POS desiderato, lista dei modelli
    most_similar = dict()
    
    #ottengo la lista delle n parole simili alla parola target filtrando per il POS tag desiderato
    for k in range(n_mod):
        similar = models[k].wv.most_similar(word, topn=(5*n_similar)) #moltiplico per avere un campione più ampio
        similar_filtered = list(filter(lambda similar: check_pos_tag(similar[0], pos_tag_prefix), similar))

    #combino le parole ottenute dai vari modelli
    for (string, similarity) in similar_filtered:
        if string not in most_similar.keys():
            most_similar[string] = similarity
        else:
            most_similar[string] = max(most_similar[string], similarity)

    #ordino le parole in base alla similarità e seleziono solo il numero desiderato
    similar_filtered_sorted = sorted(most_similar.items(), key=lambda item: item[1], reverse=True)[:n_similar]
    similar_final = [w for (w, sim) in similar_filtered_sorted]
    
    return similar_final

In [6]:
# Funzione utile in quella precedente, che definisce il "part-of-speech" tag
tagger = treetaggerwrapper.TreeTagger(TAGLANG='it')

def check_pos_tag(word: str, tag_prefix: str) -> bool:
    tag = tagger.tag_text(word)
    part_of_speech = treetaggerwrapper.make_tags(tag)
    return part_of_speech[0].pos.startswith(tag_prefix)

In [7]:
# Declinazione della funzione generale per trovare i sinonimi

def find_synonyms(word: str, n_synonyms: int, n_mod: int, models: list) -> list:
    return find_similar(word, n_synonyms, n_mod, 'NOM', models) #controllare che il tag sia giusto

# Declinazione della funzione generale per trovare gli aggettivi descrittivi

def find_describing_adjectives(words: list, n_adjectives: int, n_mod: int, models: list) -> None:
    #in questo caso passo come argomento la lista contenente i sinonimi
    for word in words:
        print(find_similar(word, n_adjectives, n_mod, 'ADJ', models))

In [8]:
# Funzione per comparare una parola fra embedding diversi

def compare_word_btw_embeddings(word_to_study, n_mod, text1, text2):
    #parola da studiare, numero di embedding creati per corpus, testo preso come riferimento (da cui estraggo il vettore della
    #parola target), testo da cui estraggo le parole simili
    
    #prendo il numero totale di embedding dei due testi considerati
    embeddings1 = []
    for it in range(n_mod):
        model = Word2Vec.load('CADE_' + text1 + '_cade_' + str(it) + '.model') #sistemare il nome se necessario
        embeddings1.append(model)
        
    embeddings2 = []
    for it in range(n_mod):
        model = Word2Vec.load('CADE_' + text2 + '_cade_' + str(it) + '.model')
        embeddings2.append(model)

    #ottengo la lista delle parole più simili alla target per ciascun modello
    words = []
    for k in range(n_mod):
        word_vector = embeddings1[k].wv[word_to_study]
        for tupla in embeddings2[k].wv.most_similar(positive=[word_vector])[:10]: #modificare a piacere il numero
            words.append(tupla[0])
    
    #creo dizionario con parole uniche e relativa frequenza assoluta
    c = Counter(words)
    word_frequencies = dict(c)
    
    #creo dizionario con parole uniche calcolando la similarità media
    init = [0] * len(np.unique(words))
    word_similarities = dict(zip(np.unique(words), init))
    
    for k in range(n_mod):
        word_vector = embeddings1[k].wv[word_to_study]
        for tupla in embeddings2[k].wv.most_similar(positive=[word_vector])[:10]:
            word_similarities[tupla[0]] += tupla[1]
    
    for word in np.unique(words):
        word_similarities[word] = word_similarities[word] / word_frequencies[word]
  
    #genero il dataframe finale
    df = pd.DataFrame(list(word_similarities.items()), columns=['Words','Med.similarity'])
    df = df.sort_values(by=['Med.similarity'], ascending=False)
    df = df.head(n=10)
    
    return df

# ANALISI WORD2VEC

## ANALISI PER PERIODO STORICO

In [9]:
os.chdir(path_storico_w2v)

In [11]:
lista_periodi = ['1250_1454_Tardo_Medioevo', '1454_1559_Rinascimento', '1559_1707_Seicento', '1707_1796_Settecento',
                 '1796_1814_Periodo_napoleonico', '1814_1860_Risorgimento', '1861_1914_L\'Italia_liberale',
                 '1915_1922_Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra', '1922_1945_Ventennio_Fascista_Seconda_Guerra_Mondiale',
                 '1945_presente_La_Repubblica_Italiana']

### TOPOS: PASTORE

In [13]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("pastore", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       gattivo              5        0.656023               0.727594
1    battezzato              5        0.662622               1.000000
2    ubbidiente              5        0.656196               0.319199
3      riverito              1        0.640724               0.374263
4    apostolica              5        0.651634               0.662450
5   fedelissimo              1        0.626735               0.430538
6   persecutore              5        0.646884               0.110320
7     cattolico              3        0.635160               0.836409
8     religioso              1        0.623095               0.765903
9      ispirito              1        0.622285               0.000000
10      prelato              3        0.650286               0.367955
11   obbediente              2        0.644871               0.254217
12      eretico              3        0.639651             

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    mandriano              5        0.585911               0.181058
1      pastori              5        0.525099               0.000000
2      porcaro              5        0.531320               0.233586
3        ameto              5        0.502250               0.186796
4    contadino              4        0.512448               0.280884
5        boaro              1        0.502836               0.179313
6     pecoraio              2        0.501695               0.433147
7   fraticello              2        0.502071               0.124304
8       pecore              3        0.506837               1.000000
9      nuorese              1        0.489044               0.051788
10      gregge              3        0.528003               0.192410
11       ovile              4        0.503409               0.142696
12   sacerdote              1        0.509574               0.404727
13     armento              2     

### Altre parole correlate

In [15]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("pastori", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        infedeli              5        0.709807               0.386466
1         gattivi              5        0.724896               0.514789
2         moderni              5        0.726717               0.279424
3    reformazione              4        0.697350               0.500807
4        chierici              2        0.701961               0.964306
5      scellerati              4        0.699694               0.465982
6        giudicii              2        0.681222               0.108314
7         tentati              4        0.696287               0.668574
8      correggono              2        0.689952               0.070705
9         ingrati              1        0.679303               0.828118
10  ecclesiastici              3        0.701247               1.000000
11       scandali              1        0.701168               0.000000
12      giacciono              1 

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      pastore              2        0.544397               0.056691
1    mandriani              5        0.558006               0.372819
2      nuoresi              3        0.544976               0.211297
3   pastorelle              5        0.550169               0.581682
4     barbagia              4        0.528506               0.000000
5     bifolchi              5        0.542359               0.292661
6       caprai              2        0.531549               0.057455
7   pascolando              2        0.529743               0.374180
8      presepi              1        0.527480               0.481795
9        capre              3        0.522751               0.149457
10     villici              2        0.526107               1.000000
11   contadini              3        0.538623               0.651629
12  boscaiuoli              1        0.520602               0.244375
13     eremiti              2     

### TOPOS: AMORE

In [16]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("amore", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            amor              5        0.648131               0.197705
1          ardore              5        0.561142               0.352031
2          amando              5        0.536192               0.269734
3            core              1        0.530745               0.094521
4             ami              5        0.514999               0.174737
5            amar              2        0.521975               0.229353
6         amatore              4        0.505564               1.000000
7         amadore              2        0.509531               0.496457
8            crea              2        0.504379               0.127385
9       tenerezza              1        0.496274               0.320500
10     dileczione              3        0.503439               0.167880
11          amare              2        0.491706               0.162438
12       caritade              2 

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       affetto              5        0.684076               1.000000
1          amor              5        0.656454               0.030666
2       giuravi              5        0.599899               0.084396
3      passione              5        0.598628               0.017697
4        amante              1        0.539004               0.042843
5        svelai              3        0.541885               0.817754
6      amandolo              2        0.537158               0.000000
7         nutro              1        0.534219               0.098169
8       donarsi              2        0.542735               0.133770
9   corrisposti              1        0.532510               0.067461
10        odiai              2        0.547059               0.030732
11     disamare              1        0.547389               0.444618
12     amandomi              1        0.545302               0.070230
13      riamava     

### Altre parole correlate

In [17]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("amor", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       amore              5        0.648131               0.817357
1      disire              5        0.633631               1.000000
2    piagente              3        0.632934               0.531858
3       pïetà              5        0.636475               0.378069
4        core              1        0.631823               0.401951
5        amar              5        0.641769               0.583756
6      deleto              3        0.626466               0.201312
7     dolzore              1        0.625882               0.361293
8       ochie              1        0.624299               0.323672
9       desio              2        0.627298               0.606809
10     piacer              3        0.636655               0.137111
11        mïo              3        0.641312               0.508819
12    floruça              2        0.624571               0.567483
13      nente    

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       amore              5        0.600786               0.121661
1      desiro              5        0.540823               0.117088
2       desìo              1        0.557095               0.225591
3   perdonami              3        0.537026               0.223954
4    estingua              1        0.539020               1.000000
5      vivrai              3        0.524854               0.162028
6       merto              3        0.522518               0.000000
7       gioir              5        0.533766               0.228473
8      perdon              3        0.519582               0.311881
9    amistade              4        0.533161               0.498611
10      festi              1        0.548706               0.020249
11         fé              2        0.534204               0.291192
12       aïta              1        0.533554               0.402350
13    rapisti              1        0.531413    

In [18]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("innamorato", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       giannetta              5        0.729666               0.382074
1      gentilesca              4        0.719790               0.685877
2       inamorato              5        0.718051               0.248892
3      giovanetta              4        0.717265               0.365617
4     onestissima              4        0.711547               0.000000
5        parvegli              2        0.705756               0.262744
6         giovine              1        0.715913               0.424453
7      innamorata              5        0.735093               0.627850
8        pregiata              4        0.715969               0.524561
9             amò              1        0.709006               0.868013
10  innamoramento              1        0.719825               0.566447
11           amai              2        0.722176               0.533582
12      tenerezza              1 

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      perdutamente              5        0.689447               0.340004
1           riamata              3        0.663061               0.407273
2           riamato              5        0.663194               0.066308
3       adolescente              5        0.649586               0.242013
4        giovanotto              3        0.656074               0.447433
5        garzonetto              3        0.643667               0.264915
6       novellatore              3        0.651814               0.551423
7          reverito              2        0.649102               0.328801
8       innamorarsi              4        0.645051               0.376899
9   svisceratamente              2        0.652171               1.000000
10        invaghito              4        0.662355               0.619371
11        avvenente              4        0.653648               0.331990
12          isalina              1    

### TOPOS: CHIESA

In [19]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("chiesa", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     scomunicato              5        0.582856               0.200698
1       sismatici              5        0.555956               0.092671
2     persecutore              5        0.556303               0.171529
3        ecclesia              5        0.530245               0.195864
4        reparata              2        0.534618               0.188661
5      obbedienza              4        0.534697               0.213533
6      contumacia              5        0.528581               0.101291
7       comunione              2        0.527435               0.181576
8        cappella              4        0.529386               0.444528
9        laterano              2        0.528107               0.280084
10      predicava              1        0.531227               0.081047
11  congregazione              3        0.529063               0.074997
12          santa              1 

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     cappella              5        0.620410               0.262796
1    cattolica              5        0.603846               1.000000
2     convento              5        0.577873               0.023462
3    monastero              5        0.563922               0.804161
4   parrocchia              5        0.553560               0.118057
5      damiano              2        0.546353               0.497084
6       tempio              2        0.549162               0.124411
7      pregare              2        0.535830               0.117207
8        santi              2        0.537802               0.332132
9    sagrestia              4        0.548384               0.209623
10    basilica              3        0.558057               0.000000
11       duomo              2        0.545743               0.148427
12       santa              2        0.549877               0.166085
13   santuario              3     

### Altre parole correlate

In [20]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("papa", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    bonifazio              5        0.760918               0.077358
1    cardinali              5        0.740218               1.000000
2    cardinale              5        0.693686               0.500035
3       papato              4        0.635418               0.846064
4    bonifacio              4        0.636847               0.071325
5    scomunica              2        0.628893               0.111708
6   concestoro              5        0.641466               0.016869
7     concilio              3        0.628712               0.086588
8      vignone              2        0.626444               0.008577
9   innocenzio              3        0.626797               0.002628
10     piuvico              3        0.633731               0.066732
11   scomunicò              4        0.632190               0.054189
12   celestino              2        0.636035               0.018212
13 

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        sinodo              4        0.723032               0.055541
1        nicolò              2        0.711639               0.249971
2     pontefice              5        0.725098               0.221847
3     innocenzo              5        0.718315               0.462958
4      clemente              2        0.724299               0.740209
5      gregorio              5        0.733949               0.702190
6     carignano              2        0.716548               0.181629
7       borbone              2        0.709647               0.498267
8   arcivescovo              1        0.703347               0.223548
9      templari              1        0.701548               1.000000
10   successore              4        0.723538               0.190840
11    richelieu              3        0.713113               0.290610
12     pisacane              3        0.717041               0.517487
13     gandusio     

In [21]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("prete", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     pasquino              5        0.746746               0.057959
1     pistello              5        0.708409               0.000000
2        ganfo              5        0.691083               0.087954
3       juccio              5        0.692117               0.083247
4      massaia              4        0.676779               0.129379
5      tabarro              4        0.674191               0.340491
6       checca              2        0.669768               0.130817
7      rustico              2        0.669669               0.023391
8    giglietta              4        0.673386               0.352746
9      mortaio              4        0.689377               0.171011
10  figliuccio              1        0.673265               0.320284
11   belcolore              4        0.667237               0.063844
12     mugnaio              1        0.667734               1.000000
13 

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     sacerdote              5        0.640994               0.257526
1      spretato              5        0.623425               0.000000
2       parroco              5        0.629342               0.233340
3    cappellano              4        0.574429               0.029851
4        curato              5        0.572260               0.162616
5       pretino              2        0.552605               0.287248
6   missionario              1        0.564243               0.143909
7    sagrestano              3        0.560457               0.005126
8        virdis              2        0.552176               0.135499
9     officiava              4        0.563650               0.137297
10        janni              3        0.577093               0.870038
11   officiante              2        0.563489               0.014095
12        filìa              3        0.554017               1.000000
13        frate     

In [22]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("frate", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       masseo              5        0.642550               0.240990
1    francesco              5        0.618019               0.214839
2      ruffino              5        0.585313               0.340865
3       zelone              5        0.554911               0.281415
4   cattivello              2        0.543861               0.811331
5         elia              2        0.549117               0.274256
6       gomita              4        0.531155               0.143272
7     domenico              4        0.540461               0.000000
8    venturino              3        0.525670               1.000000
9       pietro              5        0.550805               0.294399
10      egidio              4        0.556847               0.125443
11       frati              3        0.548186               0.559791
12    ispirito              1        0.512527               0.052125
13 

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    francescano              5        0.631289               0.152222
1    predicatore              4        0.592207               0.094834
2     domenicano              5        0.593830               0.609211
3       sfratato              5        0.603328               0.000000
4     cappuccino              5        0.602319               0.190411
5     zoccolante              5        0.599908               0.494694
6       minorita              5        0.574701               1.000000
7         ochino              1        0.566039               0.008320
8         priore              4        0.579750               0.236643
9    carmelitano              1        0.557159               0.119798
10    bernardino              1        0.564189               0.117121
11         prete              3        0.565920               0.472791
12  cisterciense              3        0.571274               0.118189
13    

In [24]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("dio", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          idio              5        0.605079               0.049757
1         iddio              5        0.607632               0.083773
2      creatore              5        0.550810               0.671963
3        divina              5        0.525256               0.177673
4          creò              5        0.465191               0.044679
5       peccata              5        0.469321               0.524435
6   onnipotente              5        0.454897               0.038417
7      potensia              1        0.454839               1.000000
8        severa              1        0.444698               0.985263
9    cooperante              3        0.448926               0.036009
10    trinitade              2        0.443581               0.054341
11      maiestá              1        0.443828               0.000000
12    immediate              2        0.440652             

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          iddio              5        0.537375               0.080697
1         cristo              5        0.462300               0.070903
2         divina              5        0.481830               0.095064
3    provvidenza              5        0.473390               0.334281
4         scampi              5        0.452981               0.507459
5        assista              4        0.433703               0.157892
6      peccatore              4        0.435375               0.099144
7   misericordia              4        0.436941               1.000000
8       divinità              2        0.442409               0.109412
9       precetto              1        0.424603               0.095525
10      creatore              3        0.435323               0.000000
11          thor              4        0.429975               0.432613
12      clemenza              1        0.432599               0.061516
13    

In [25]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("cristo", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            gesù              5        0.644070               0.216190
1    resurrezione              5        0.561741               0.117596
2            iesù              5        0.567642               0.212475
3       salvatore              5        0.562062               0.095181
4       discepoli              5        0.532890               0.191343
5        apostoli              4        0.530983               0.000000
6      consecrato              3        0.513524               0.229403
7       benedetto              5        0.530447               1.000000
8        apostuli              5        0.516758               0.077164
9         lazzaro              1        0.515320               0.470275
10   predicazione              2        0.516638               0.107627
11      crocifixo              1        0.499792               0.116767
12      risuscitò              1 

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         gesù              5        0.765126               0.003217
1    redentore              5        0.643622               0.029937
2      vangelo              5        0.633232               0.035101
3        santi              3        0.602519               0.006076
4     precetto              2        0.606097               1.000000
5   sacramento              5        0.618512               0.000000
6   santissimo              5        0.603468               0.027661
7     apostolo              5        0.596937               0.056972
8        iddio              2        0.591747               0.082768
9     apostoli              3        0.592304               0.299204
10   salvatore              2        0.608994               0.154375
11     martire              1        0.596543               0.099478
12      bambin              4        0.597832               0.062131
13        mosè              2     

### TOPOS: FELICITA'

In [26]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("felicità", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      solitudine              4        0.750678               0.571470
1       pervenire              1        0.738090               0.542254
2    acrescimento              4        0.745432               0.720055
3      consistere              2        0.732646               0.330736
4         indurre              1        0.728261               0.809114
5        durabile              5        0.744242               0.262417
6      felicitade              4        0.745313               0.173517
7      conseguire              3        0.735767               0.516206
8    tranquillità              4        0.739518               0.387592
9          ignota              1        0.720656               0.539657
10  accrescimento              3        0.746767               0.000000
11      persevera              1        0.766346               0.166456
12  virtuosamente              3 

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             gioia              5        0.574888               0.224645
1        infelicità              5        0.567223               0.360460
2       beatitudine              3        0.541140               0.312572
3         benessere              3        0.538613               0.036760
4        prosperità              3        0.544435               0.379886
5             gioie              4        0.544495               0.044313
6        possedervi              5        0.547360               1.000000
7           quaggiú              4        0.547729               0.424040
8            gioire              1        0.530572               0.160186
9           ridarti              3        0.533456               0.070964
10        esistenza              1        0.542165               0.854745
11       possederti              2        0.530571               0.096155
12          anelato              2    

### Altre parole correlate

In [28]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("gioia", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   allegranza              5        0.724434               0.964364
1         gioi              5        0.711155               0.329777
2      dolzore              4        0.694452               0.434756
3      pesanza              5        0.715543               0.479421
4      martìre              4        0.699051               0.130972
5         noia              3        0.696259               0.339071
6       prendo              4        0.698704               0.377600
7       manera              1        0.693940               0.687414
8      beltate              5        0.694006               1.000000
9       disire              1        0.684474               0.328397
10     gioiosa              4        0.726143               0.434976
11         nòi              2        0.700378               0.000000
12     biltate              2        0.699040               0.376763
13 

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           gioja              5        0.730274               0.535073
1     contentezza              5        0.693709               0.000000
2      allegrezza              5        0.648898               0.074181
3         giubilo              5        0.627488               0.791091
4       esultanza              5        0.585494               0.047490
5        felicità              4        0.583025               0.172831
6         letizia              5        0.572513               0.158726
7    consolazione              5        0.568228               1.000000
8        ebbrezza              5        0.563556               0.412751
9   soddisfazione              5        0.560680               0.098668
10       angoscia              1        0.555243               0.031061
###1915_1922_Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)


### TOPOS: PATRIA/ITALIA

In [29]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("patria", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       essilio              5        0.606277               0.295030
1      ateniesi              3        0.584054               0.395075
2         natio              4        0.584883               0.158948
3       origine              4        0.598769               0.019196
4      fabrizio              3        0.591597               0.468663
5   familiarità              3        0.592301               0.473298
6       publica              5        0.591438               0.496003
7          odii              3        0.580110               1.000000
8   benivolenza              1        0.575380               0.600387
9     democrito              3        0.586928               0.619547
10       civile              3        0.586758               0.000000
11      ingrata              3        0.591518               0.598233
12    republica              2        0.580897             

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    indipendenza              5        0.459567               0.004255
1      proscritto              5        0.425679               0.173871
2          patrio              3        0.416135               0.122774
3      repubblica              5        0.429081               0.262383
4       timoleone              3        0.418500               0.315355
5   virtuosamente              2        0.427368               0.142064
6     santificata              2        0.420422               0.378949
7        fiorenza              2        0.427357               0.498576
8     rivendicare              1        0.412638               1.000000
9         umanità              1        0.411077               0.115921
10    sagrificava              4        0.427819               0.162588
11    avventurosi              2        0.416720               0.044110
12        felloni              1        0.415071               0

In [30]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("italia", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      gallia              5        0.722306               0.244506
1    alamagna              5        0.687537               0.169569
2       gotti              5        0.664153               0.527202
3      africa              5        0.666748               1.000000
4     toscana              5        0.664461               0.209212
5   lombardia              3        0.664898               0.219336
6      spagna              4        0.656958               0.297988
7    africano              4        0.661238               0.000000
8      persia              1        0.648768               0.203974
9     numidia              5        0.670681               0.090310
10    vandali              4        0.665585               0.111365
11  sottomise              2        0.655409               0.213398
12     puglia              2        0.663768               0.228686
###1454_1559_Rina

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       bologna              5        0.794477               0.871877
1   rivoluzione              4        0.786106               0.879781
2   inghilterra              5        0.794290               0.240732
3        milano              3        0.793564               0.697343
4         daria              1        0.794744               1.000000
5        europa              5        0.788585               0.762081
6      germania              5        0.790771               0.594810
7        romana              4        0.787760               0.866323
8        vienna              4        0.788233               0.659631
9      guarnati              5        0.785124               0.844819
10       russia              2        0.797796               0.372315
11      europea              1        0.798802               0.333239
12       londra              2        0.787312               0.623905
13     edizione     

### Altre parole correlate

In [31]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("stato", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         istato              5        0.480544               0.093292
1         vivuto              5        0.447241               0.292182
2        gattivo              5        0.429780               0.036349
3      rifiutato              4        0.426267               0.182089
4          stati              5        0.449981               0.143782
5      familiare              3        0.430671               0.186329
6       reverito              2        0.426114               0.171805
7       esaltato              2        0.430411               1.000000
8       pacefico              2        0.422286               0.050149
9          operò              1        0.422323               0.049542
10      reputato              2        0.421731               0.051633
11     adoperato              1        0.424968               0.044580
12      riverito              1        0.42490

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        manicomio              5        0.493591               0.373069
1        mantenuto              4        0.474967               0.273349
2         discusso              5        0.468415               0.414873
3         camerale              2        0.459958               0.163759
4   interinalmente              4        0.466845               0.430334
5     dipartimento              3        0.467814               0.435679
6        sostituto              2        0.469247               0.359097
7       archivista              5        0.464365               0.374040
8     supponendolo              1        0.453371               0.212102
9       fallimento              2        0.464615               0.394637
10     funzionario              1        0.480334               0.279552
11      mandatario              1        0.473297               0.670397
12        ciampoli              1        0.470894  

In [32]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("impero", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       glesia              5        0.798735               0.024463
1      risuona              1        0.779336               0.660706
2       electo              4        0.771767               0.311245
3        digno              4        0.764923               0.450660
4    magistero              3        0.761161               0.794124
5        summo              2        0.768525               1.000000
6      iocondo              1        0.764856               0.430216
7    imperador              4        0.770763               0.309893
8       perito              4        0.775350               0.234613
9      primero              5        0.767462               0.053940
10        fiol              3        0.788628               0.370398
11     nutrito              1        0.767827               0.466366
12    virtüoso              1        0.752860               0.000000
13 

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     teocratico              5        0.552044               0.411935
1       bisanzio              5        0.530034               0.372408
2     macedonica              5        0.529330               0.537525
3      monarcato              5        0.547476               0.251718
4    dominazione              1        0.524771               0.445793
5   protettorato              4        0.528118               0.426933
6     elettorato              3        0.519469               0.309068
7       ottomano              5        0.522938               0.315529
8      autocrate              4        0.522176               0.522727
9     soggiogate              1        0.504452               0.904008
10   napoleonico              1        0.526972               0.473174
11    teocratica              1        0.518420               0.388422
12     danubiani              2        0.525668               0.213825
13    

In [33]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("nazione", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           nazion              5        0.790536               0.124238
1         ateniese              5        0.790485               0.999094
2        legnaggio              4        0.769922               0.336111
3        spagnuolo              3        0.761135               0.231750
4     crudelissimo              3        0.752777               0.314764
5          quinzio              5        0.748424               0.637557
6           stirpe              5        0.761631               0.096506
7        lignaggio              5        0.739149               1.000000
8           famoso              3        0.751562               0.000000
9        affricano              1        0.740726               0.369697
10          publio              4        0.740767               0.219603
11     famosissimo              3        0.746841               0.124969
12       parentela  

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      nazionalità              5        0.602440               0.321605
1      plutocrazia              5        0.575933               0.585824
2          nazioni              1        0.570988               0.216267
3      oligarchica              3        0.574760               0.295722
4     plutocratica              4        0.572381               0.000000
5       elettorato              5        0.568655               0.294139
6        smembrata              5        0.572323               0.278468
7     rivendicarsi              3        0.564098               0.396004
8     costituitasi              4        0.568730               1.000000
9        clericume              4        0.569969               0.342885
10    collettività              2        0.582616               0.204400
11       schleswig              1        0.566185               0.421708
12       decaddero              1        0.554576  

In [34]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("bandiera", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        comuna              5        0.823661               0.208446
1       pennone              5        0.845143               0.319372
2       guidava              4        0.834967               0.000000
3    linguadoco              5        0.830496               0.189031
4      ischiera              4        0.814277               0.528344
5           vic              2        0.812424               0.143111
6       gualesi              3        0.811034               0.073023
7      salmeria              3        0.812112               0.251094
8         lilla              2        0.810223               0.185216
9     guidatore              2        0.817643               0.396212
10  guarnimento              2        0.808034               0.776586
11     feditori              2        0.806116               0.322286
12     assaliro              2        0.809092             

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      vessillo              5        0.687883               0.026586
1     stendardo              5        0.652017               0.010581
2     tricolore              5        0.666985               0.000000
3      sventola              5        0.623350               0.017503
4    inalberata              5        0.619926               0.406508
5   sventolante              5        0.596344               0.125707
6    sventolava              5        0.609899               0.686775
7    sventolerà              5        0.606734               0.436410
8      bandiere              1        0.576424               0.200559
9   inalberando              1        0.573380               0.399792
10   sventolare              4        0.619157               0.318934
11     inalbera              1        0.574591               0.291300
12   inalberare              2        0.575384               0.817490
13   inalberava     

### TOPOS: GUERRA

In [35]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("guerra", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         briga              5        0.575874               0.000000
1     discordia              5        0.546348               0.247026
2       triegua              5        0.531149               0.128819
3      vettoria              3        0.526992               1.000000
4        tregua              3        0.504783               0.321079
5      annibale              4        0.505494               0.210799
6         guera              1        0.510054               0.114415
7     sconfitta              1        0.508322               0.684111
8   dissensione              5        0.513684               0.334642
9     fiaminghi              5        0.512231               0.319098
10     catalani              2        0.512416               0.295803
11      impresa              1        0.501507               0.204365
12    cittadina              3        0.502108             

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   conflagrazione              5        0.516260               0.253879
1          europea              5        0.521375               0.194377
2         mondiale              5        0.514490               0.933296
3         germania              3        0.482804               0.201038
4          turchia              4        0.464106               1.000000
5           libica              3        0.467070               0.496972
6        sconfitta              4        0.471986               0.176592
7            lotta              4        0.471593               0.000000
8        battaglia              2        0.464164               0.291777
9       neutralità              2        0.459875               0.350774
10     rivoluzione              4        0.478268               0.910204
11          guerre              2        0.458237               0.214241
12       scoppiata              2        0.463105  

### TOPOS: FIGURA FEMMINILE

In [36]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("donna", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      madonna              5        0.571829               0.236879
1      giovana              5        0.555148               0.252875
2   giovanetta              5        0.541137               0.082035
3      filippa              5        0.536703               0.302465
4   margherita              1        0.525118               0.139632
5        gemma              4        0.522823               0.430981
6     isabetta              4        0.519441               0.186068
7   margharita              2        0.517378               0.206128
8    ermellina              3        0.518718               0.729781
9        vanna              4        0.517745               0.502816
10      vedova              2        0.521408               0.000000
11     antonia              2        0.525708               0.151357
12      agnola              2        0.524721               0.054446
13 

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      fanciulla              5        0.598926               0.307098
1        ragazza              5        0.619087               0.310299
2      almerinda              5        0.570951               0.167313
3    innocentina              5        0.570774               0.167919
4       creatura              5        0.598428               0.222456
5    sempliciona              1        0.576017               0.219792
6      assuntina              2        0.571173               0.702615
7         costei              2        0.564886               0.165240
8       lionetta              3        0.575458               0.147911
9   contadinella              2        0.561282               0.709794
10       donnina              1        0.561096               0.000000
11       signora              1        0.558588               0.304546
12    superbiosa              4        0.570088               0.185266
13    

In [37]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("madre", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   fanciulla              5        0.568568               0.069792
1   sirocchia              5        0.569946               0.140668
2    matrigna              5        0.575799               0.553653
3   genitrice              5        0.538775               0.862193
4     vergine              5        0.541830               0.024481
5      cugina              5        0.552378               0.294414
6     rebecca              5        0.551134               0.000000
7      isposa              1        0.525750               0.131792
8     sposata              2        0.530073               0.031631
9   fanciullo              4        0.524611               0.046194
10    sorella              2        0.528723               1.000000
11   gostanza              1        0.530242               0.529943
12     creusa              2        0.530708               0.888459
13   giovanna    

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        sorella              5        0.663975               0.000000
1    figlioletta              5        0.658537               0.217677
2          mamma              5        0.655533               0.256309
3      orfanella              5        0.637669               0.031777
4    figliuolina              5        0.634979               0.906781
5   amorosissima              5        0.618341               0.102777
6   dilettissima              5        0.620594               0.223048
7      figliuola              1        0.611638               0.095302
8   figliuoletto              1        0.611202               0.502905
9   figliuoletta              3        0.618654               0.855285
10         padre              4        0.635385               0.176344
11        moglie              3        0.615930               0.199532
12      adottiva              1        0.605643               0.549035
13    

In [38]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("femmina", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        femina              5        0.697940               0.489786
1       maschio              5        0.688764               0.324854
2     adulterio              4        0.656009               0.301597
3     partorito              3        0.643295               0.393584
4     fanciulla              5        0.658125               0.316931
5      giaciuto              5        0.645421               0.515529
6     concubito              5        0.648103               1.000000
7     avolterio              5        0.642455               0.360951
8   carnalmente              4        0.640543               0.228986
9       incesto              1        0.615757               0.000000
10       mascio              1        0.641652               0.888341
11      gravida              2        0.634576               0.315083
12     matrigna              3        0.648398             

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        maschio              5        0.657843               0.052815
1          balba              5        0.582134               0.041663
2        femmine              5        0.544039               0.616970
3         femina              5        0.546902               0.053385
4         maschi              5        0.500368               0.126613
5    sempliciona              3        0.487126               0.064186
6          donna              5        0.485210               0.000000
7       tacchina              4        0.470738               0.438727
8   quadrantaria              3        0.487872               0.424241
9          druda              1        0.472870               0.019618
10       allatta              1        0.470623               0.203299
11      meticcia              1        0.461607               1.000000
12        scimia              1        0.471954               0.007201
13    

In [39]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("sposa", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          sposo              5        0.686916               0.159501
1          suora              4        0.635594               0.224497
2         isposa              5        0.649789               0.215608
3     elisabetta              1        0.627673               0.195066
4        sorella              5        0.641031               0.036934
5       allevata              4        0.625921               0.246782
6        sposata              5        0.634552               0.069421
7         lavina              2        0.616057               0.398917
8          sposò              5        0.626675               0.527963
9       donzella              3        0.611168               0.000000
10    dolcissima              1        0.623606               0.099089
11   acompagnata              3        0.620808               0.222134
12     damigella              2        0.62522

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        sposo              5        0.770096               0.074270
1    fidanzata              5        0.581131               0.082617
2        sûryâ              5        0.569690               0.001479
3       moglie              4        0.553206               0.055769
4    impalmata              5        0.553894               0.111188
5         caia              2        0.551681               0.016589
6     bazvalan              4        0.549694               0.206311
7       figlia              2        0.533988               0.008300
8        nozze              3        0.546095               0.000000
9   divorziata              4        0.545289               0.091909
10    doralice              2        0.558484               0.101651
11      erardo              1        0.535961               0.089021
12  cresimanda              2        0.532364               0.059154
13   figliuola              2     

### TOPOS: NOTTE/SERA

In [40]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("notte", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    mezzanotte              5        0.544583               0.104946
1          sera              5        0.553070               0.314517
2       venerdì              4        0.513747               0.308626
3        vespro              4        0.517582               0.333014
4       piovere              1        0.514012               0.836911
5     vegghiare              5        0.525477               0.249431
6     mercoledì              5        0.518361               0.068056
7         stemo              3        0.506220               1.000000
8        siando              2        0.501032               0.000000
9   sopravenuta              3        0.503468               0.046826
10      mattina              4        0.517398               0.099708
11        scurò              2        0.504573               0.469754
12     vegnente              3        0.520072             

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      nottata              5        0.652857               0.130468
1         sera              5        0.640237               0.090021
2    mattinata              5        0.549857               0.240292
3     stanotte              5        0.547526               0.011080
4       dormii              5        0.533273               0.251052
5      insonne              5        0.548377               0.117398
6   mezzanotte              2        0.531476               1.000000
7      vegliai              3        0.520381               0.044526
8        notti              1        0.521454               0.000000
9     giornata              3        0.527711               0.908237
10   vegliando              4        0.538983               0.001844
11     mattina              2        0.516854               0.234100
12      vegliò              3        0.522864               0.036720
13      serata              1     

In [41]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("sera", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      mattina              5        0.719935               0.023628
1   mezzanotte              5        0.692590               0.053419
2       cenato              5        0.700707               0.737231
3       vespro              5        0.681525               0.000000
4      desnare              5        0.670908               0.195168
5      dormire              4        0.667749               0.329989
6     desinare              5        0.661042               0.080235
7     domenica              4        0.652632               0.286825
8         cenò              2        0.646761               1.000000
9       cenare              5        0.650670               0.626311
10    caminare              1        0.648490               0.091762
11    vantagio              4        0.653476               0.475091
###1454_1559_Rinascimento###
       Words  Abs.frequency  Med.similarity

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       mattina              5        0.590781               0.118516
1    mezzanotte              5        0.512535               0.186665
2          sere              5        0.509972               0.243061
3      domenica              5        0.497854               0.131119
4          cena              1        0.490930               0.184292
5      indomani              4        0.490639               0.018391
6     mercoledì              5        0.489823               1.000000
7   mezzogiorno              4        0.504685               0.177650
8       giovedì              3        0.484111               0.362650
9     viareggio              2        0.474838               0.078667
10      venerdì              3        0.500473               0.298707
11   pomeriggio              1        0.495853               0.226764
12       sabato              3        0.496362               0.000000
13      arrivai     

### Altre parole correlate:

In [44]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("tenebre", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    tenebrosa              4        0.698473               0.315168
1     oscurità              5        0.691749               0.516054
2   immondizia              2        0.681732               0.448603
3      difecto              3        0.689555               0.720935
4     notturne              4        0.678396               0.834323
5      mentale              1        0.678114               0.362395
6      lagrima              5        0.677819               0.604868
7      trinitá              3        0.670645               0.688996
8     ciechitá              5        0.683451               0.616298
9     giongono              2        0.664151               0.621789
10   sprendore              3        0.680523               0.627026
11  ignoranzia              4        0.677203               0.875091
12   tenebrosi              3        0.675603               0.000000
13 

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      caligine              5        0.674023               0.557860
1      oscurità              5        0.691363               0.400678
2       tenebra              5        0.676836               0.375626
3       dissipa              4        0.646814               0.358010
4       letargo              5        0.652698               0.739621
5     barbaglio              2        0.649782               0.210415
6        sperde              1        0.646531               0.331055
7        nebbie              2        0.649990               0.172205
8     immensità              5        0.660817               0.000000
9   dissolversi              2        0.636236               0.541327
10     proietta              1        0.654187               0.089797
11       abissi              2        0.661918               0.419403
12   addensarsi              1        0.651451               0.406928
13      macerie     

In [45]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("ombra", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         specchia              5        0.709391               0.302437
1         matutina              5        0.698480               0.479386
2        mattutina              4        0.699788               0.155358
3         nebulosa              4        0.691679               0.220931
4           cuopre              1        0.695597               0.466304
5            denso              2        0.687012               0.347179
6          orïente              3        0.691031               0.000000
7         elemento              1        0.688749               0.637561
8        risprende              4        0.690957               0.588206
9           remota              4        0.694194               0.331515
10         topazio              3        0.692636               0.307072
11           vampa              2        0.687105               0.354027
12           lucea  

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         apparia              3        0.504151               0.475312
1          celasi              1        0.510506               0.726591
2           larva              2        0.496998               0.577451
3      abbagliata              1        0.509346               0.088105
4         nascoso              5        0.517758               0.759888
5         nascosa              3        0.495701               0.033225
6        estingua              2        0.507598               0.407929
7        tenebria              5        0.499798               0.676409
8        abbaglia              1        0.500593               0.514949
9          vorria              2        0.503821               0.341506
10         tiensi              1        0.518514               0.372135
11      nuvoletta              4        0.498878               0.334893
12      trapelare              4        0.513158               0

### TOPOS: FORTUNA

In [46]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("fortuna", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     prospera              5        0.521624               0.241086
1     infelice              5        0.496534               0.208748
2   prosperità              5        0.491040               0.226335
3     aversità              3        0.480268               0.413415
4      ventura              4        0.493208               0.164711
5    invidiosa              2        0.480929               0.161909
6     felicità              5        0.480042               0.307426
7         casi              1        0.476106               0.069082
8       perire              1        0.474680               0.000000
9      avversa              3        0.478506               0.236932
10     fallace              4        0.483883               0.539578
11    bonaccia              1        0.489659               0.253216
12     destino              3        0.475953               0.003344
13 

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       favoriti              4        0.557933               0.000000
1       propizia              4        0.553382               0.233359
2      rinomanza              3        0.552285               0.215213
3      risparmia              3        0.548033               0.091894
4       sofferta              1        0.557847               0.380256
5    riputazione              2        0.549631               0.630719
6    reputazione              4        0.547241               0.147444
7       sfortuna              4        0.545078               0.342272
8        onorata              2        0.557811               1.000000
9    commediante              1        0.553684               0.752274
10   invidiabile              2        0.554429               0.064917
11   risparmiata              4        0.546689               0.204902
12    compensare              1        0.536789               0.652050
13    

### TOPOS: NOIA

In [47]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("noia", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   malinconia              5        0.721028               0.185595
1        gioia              5        0.682165               0.241504
2      martìre              5        0.685801               0.246041
3     gramezza              3        0.665306               0.275381
4       doglia              5        0.663628               0.299808
5    travaglio              3        0.675420               0.489528
6   gravissima              1        0.658082               0.268077
7     asprezza              3        0.663283               0.173195
8    dubitanza              4        0.665862               0.000000
9       noiosa              5        0.664497               0.184451
10    fallanza              1        0.667721               0.326679
11     gravoso              3        0.658939               1.000000
12    partenza              3        0.661221               0.537291
13 

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         fastidio              5        0.677373               0.027226
1             noja              5        0.613096               0.057902
2            tedio              5        0.571096               0.000000
3         molestia              5        0.560727               1.000000
4       stanchezza              5        0.552191               0.024388
5        seccatura              5        0.532460               0.309396
6            uggia              4        0.520901               0.619447
7       vessazione              4        0.513112               0.005081
8         annojare              1        0.508468               0.025959
9   accapacciatura              2        0.513067               0.142045
10       tristezza              3        0.526351               0.258862
11      malinconia              2        0.512588               0.370789
12        oziosità              2        0.512853  

### TOPOS: TEMPORALE/PIOGGIA

In [48]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("temporale", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      volontaria              5        0.806186               0.297139
1   exercitandosi              4        0.775790               0.259972
2   remunerazione              5        0.773486               0.256482
3       spiritual              5        0.778116               0.000000
4     negligenzia              3        0.781572               0.417168
5      imperfecta              2        0.772171               0.326592
6   satisfaczione              3        0.768656               0.327105
7     professione              2        0.773304               0.347782
8    acrescimento              1        0.767592               0.218517
9     servitudine              4        0.766377               0.528027
10      fragilitá              1        0.771066               0.369503
11     accrescere              1        0.769530               0.372990
12        suddito              1 

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          papato              5        0.505008               0.013432
1       temporali              5        0.474397               0.384224
2          potere              5        0.472443               0.049657
3            papi              5        0.457227               0.074361
4     temporalità              3        0.463350               0.012772
5       conquasso              4        0.463740               0.082786
6         laicale              2        0.443539               0.143414
7       pontefice              3        0.446617               0.079340
8      spirituale              4        0.458922               1.000000
9     pontificato              1        0.437991               0.287030
10    triumvirale              2        0.450642               0.128719
11      nepotismo              1        0.447602               0.000000
12        dominio              2        0.442974               0

In [49]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("pioggia", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
       Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      piova              5        0.786537               0.000000
1   grandine              5        0.750859               0.152868
2      falde              5        0.751438               0.541519
3      brina              5        0.754243               0.314886
4   ghiaccio              5        0.754971               0.501697
5     nebbia              4        0.731602               0.322732
6       gelo              5        0.738832               0.085525
7      vapor              4        0.725647               0.551853
8    aduggia              1        0.716099               0.493479
9   ghiaccia              2        0.722123               0.220092
10      nubi              4        0.730750               0.114309
11    baleno              1        0.726867               0.207989
12      nube              2        0.724151               1.000000
13     calor              2    

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       dirotta              5        0.732982               0.103295
1      scirocco              5        0.683912               1.000000
2   torrenziale              5        0.681222               0.028742
3      raffiche              5        0.671805               0.186409
4    scrosciava              5        0.670148               0.074843
5       scrosci              3        0.663385               0.035778
6      grandine              5        0.670694               0.000000
7       umidità              2        0.656558               0.200925
8    scrosciare              4        0.661439               0.359063
9        folata              3        0.658623               0.090936
10       brezza              3        0.664331               0.065529
11  pioggerella              1        0.659162               0.179277
12         neve              1        0.662152               0.323931
13      raffica     

### TOPOS: GIOVINEZZA

In [50]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("giovinezza", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       sprezza              3        0.793041               0.608905
1        habito              5        0.797734               0.022822
2   benignitate              5        0.795888               0.199437
3       fermeza              1        0.801091               0.487564
4     disprezza              4        0.783232               0.830548
5        bieltà              1        0.793580               0.755368
6        polita              4        0.792302               0.460120
7         sería              4        0.794118               0.000000
8           etá              1        0.788274               0.418361
9      immortal              3        0.786927               0.113601
10      adonque              1        0.789083               0.384079
11     vanitate              1        0.775939               0.787527
12      nutrita              1        0.775060             

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     giovanezza              5        0.725380               0.297894
1       virilità              5        0.679152               0.028965
2     freschezza              5        0.653715               0.159644
3         senile              5        0.657934               0.175294
4   fanciullezza              5        0.652355               1.000000
5       puerizia              1        0.649294               0.201754
6      giovanile              4        0.656014               0.072102
7      testoride              2        0.639736               0.005394
8     rigogliosa              2        0.646005               0.138403
9     inebbriata              1        0.638950               0.145836
10   adolescenza              2        0.664368               0.124428
11    vecchiezza              2        0.656297               0.086577
12     verginale              2        0.649188               0.223853
13    

In [51]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("giovane", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     giannetta              5        0.643647               0.221072
1       giovine              5        0.620595               0.000000
2     martuccio              5        0.610074               0.141425
3      efigenia              4        0.592528               0.319816
4        madama              4        0.597527               0.446795
5     giannotto              2        0.592330               0.207147
6     collatino              5        0.605657               0.255599
7        clelia              3        0.596937               0.290112
8     giuffredi              3        0.594137               0.077639
9       masetto              1        0.595489               1.000000
10    damigella              1        0.589200               0.252720
11   gentilesca              1        0.580307               0.664311
12   mitridanes              3        0.592731             

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        giovine              5        0.861114               0.000000
1     giovanotto              5        0.616129               0.031053
2     giovinetto              5        0.600686               0.049049
3     giovinotto              5        0.607171               0.122084
4     giovinetta              4        0.575774               0.050936
5     giovanetto              5        0.595072               0.191572
6    giovincello              5        0.582374               0.080184
7   giovanissimo              1        0.573511               1.000000
8    sessantenne              3        0.562847               0.058404
9    diciottenne              3        0.567485               0.140610
10       giòvine              1        0.566507               0.162012
11       giòvane              1        0.564408               0.020898
12     fanciulla              3        0.572939               0.027824
13    

### TOPOS: BACIO

In [52]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("bacio", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       menerò              5        0.816976               0.592257
1     stazzone              1        0.832108               0.523846
2       pagata              1        0.828264               0.168297
3          lel              3        0.817694               0.198239
4     tornerai              1        0.821595               0.689461
5   abracciato              2        0.819269               0.333067
6       bascio              4        0.822330               0.710385
7       atento              2        0.806862               0.304586
8        basto              5        0.811108               0.385389
9        vorrè              1        0.810463               0.000000
10      chiesi              1        0.822989               0.347729
11       misso              2        0.818445               0.542226
12         ssì              2        0.823468               0.519417
13 

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           baci              5        0.623898               0.208199
1        carezza              5        0.561149               0.029125
2        bacione              5        0.542660               1.000000
3      stringevi              2        0.514936               0.180143
4        baciare              5        0.553001               0.091703
5        baciami              5        0.520806               0.375361
6          baciò              4        0.515085               0.114966
7        donasti              1        0.512749               0.284252
8        baciata              5        0.523446               0.041303
9        ribaciò              1        0.510397               0.447147
10        baciai              1        0.530844               0.020724
11     abbraccio              4        0.534028               0.067327
12      amplesso              3        0.513703               0.000000
13    

### TOPOS: LUNA

In [53]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("luna", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           cancro              5        0.762249               0.791539
1           ariete              5        0.749598               0.032286
2          eclissi              5        0.736979               1.000000
3         stellato              5        0.745253               0.142016
4          pianeti              2        0.718831               0.583593
5            scurò              5        0.721912               0.137548
6            radii              3        0.713287               0.102946
7        scorpione              4        0.714887               0.205369
8        occidente              5        0.714027               0.284543
9           lunare              1        0.707811               0.193174
10          diurno              3        0.710182               0.000000
11  interposizione              4        0.709911               0.219548
12           tauro  

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       stella              5        0.601184               0.520801
1     chiarore              4        0.602670               0.849248
2       raggio              4        0.595321               0.862861
3     chiarità              4        0.602301               0.398094
4     biancore              4        0.593411               0.000000
5       stelle              4        0.597828               0.537970
6     tramonto              1        0.592892               0.067895
7     tramonta              2        0.592280               0.455762
8   crepuscolo              1        0.590090               0.395674
9       lunare              4        0.593876               0.591341
10  tramontava              3        0.600177               0.403716
11     spuntar              4        0.606382               0.204788
12    stellato              3        0.593060               1.000000
13   scintilla              1     

### TOPOS: SILENZIO

In [54]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("silenzio", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       broncone              5        0.643022               0.331477
1     cantarelle              5        0.647128               0.872006
2   dirottamente              4        0.630751               0.988549
3        sermone              3        0.642384               0.759010
4       recitate              5        0.632038               0.388558
5     consonante              3        0.641529               0.829640
6        tacette              2        0.624817               0.523715
7       cantasse              3        0.639040               1.000000
8          stilo              2        0.622922               0.245127
9      salutevol              1        0.612299               0.203623
10         fioca              3        0.647531               0.482543
11  amorosamente              4        0.623978               0.529519
12        impone              1        0.62093

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          quiete              1        0.461354               0.559996
1        mormorìo              4        0.455070               0.000000
2      silenzioso              4        0.452227               0.227489
3   vaneggiamento              1        0.449730               0.289275
4      svenimento              1        0.448986               0.384968
5   raccoglimento              2        0.440421               0.219392
6        profondo              2        0.448087               0.384497
7          gemito              3        0.439020               0.001208
8           pausa              5        0.443939               0.528721
9    stupefazione              1        0.434929               0.170942
10        lamento              3        0.444254               0.826828
11          tacer              1        0.441123               0.520049
12       esitante              2        0.439038               0

### TOPOS: AMICIZIA

In [55]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("amicizia", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    familiarità              5        0.746105               0.403987
1    benivolenza              5        0.751482               0.186964
2      constanza              5        0.727530               0.480817
3    acquistarsi              2        0.713015               0.480252
4      imperoché              3        0.706990               0.297452
5   espettazione              3        0.702886               1.000000
6      assiduità              4        0.717937               0.403180
7        stabile              1        0.706196               0.628887
8      familiare              1        0.704704               0.711151
9     utilissimi              1        0.703988               0.247045
10    liberalità              4        0.719397               0.201262
11          ozii              2        0.710921               0.197502
12    conoscerai              1        0.71296

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       devozione              5        0.650745               0.405615
1    riconoscenza              4        0.668098               0.259609
2     gratitudine              5        0.651907               0.000000
3   sollecitudine              4        0.630428               0.217799
4           stima              5        0.637503               0.214789
5      affettuosa              3        0.622281               0.236904
6      cordialità              2        0.639036               0.530359
7      complicità              2        0.619508               0.719584
8          lealtà              3        0.627677               0.026487
9         fiducia              3        0.621628               0.249470
10    familiarità              2        0.621654               0.249048
11       intimità              1        0.612651               0.422158
12   famigliarità              1        0.607855               0

In [56]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("amico", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     servidore              5        0.632020               0.478198
1       parente              5        0.588503               0.246411
2       amadore              5        0.589221               0.040879
3          leal              5        0.575930               0.096269
4     sollecito              1        0.571503               0.000000
5         leale              5        0.574154               0.131957
6   fedelissimo              5        0.586506               0.181358
7      indovino              1        0.564596               0.082795
8       aveduto              1        0.553352               0.445089
9        fidato              2        0.553049               0.037189
10    amorevole              3        0.570210               0.131098
11     servidor              1        0.561622               0.177970
12  difenditore              1        0.549999             

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          compagno              5        0.607001               0.800265
1        amicissimo              4        0.599168               0.259523
2           conneau              4        0.577467               0.856343
3             smoky              1        0.576436               0.396598
4        amatissimo              2        0.566625               0.248412
5        conoscente              1        0.572510               0.459784
6       commilitone              5        0.575813               0.467374
7   desideratissimo              3        0.576104               0.428028
8            zaruba              4        0.571332               1.000000
9           raucher              1        0.565103               0.458368
10         ifigenio              3        0.577336               0.367454
11             caro              1        0.566019               0.433368
12          gastoni              3    

### TOPOS: SOLITUDINE

In [57]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("solitudine", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    espettazione              4        0.852501               0.162273
1         desidia              5        0.857460               0.410220
2      inimicizie              2        0.851313               0.585258
3     amplitudine              4        0.848378               0.434306
4      infelicità              3        0.844249               0.477951
5         inerzia              4        0.845090               1.000000
6   pusillanimità              1        0.855750               0.498196
7         voluttà              3        0.845486               0.773540
8     disideriamo              1        0.846472               0.479944
9    acrescimento              3        0.844545               0.435103
10       maturità              4        0.847299               0.396715
11       reputano              1        0.838039               0.719273
12        sentano              1 

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     incresciosa              5        0.633322               0.027264
1       solitaria              5        0.603812               0.129185
2       taciturna              4        0.584534               0.089541
3       monotonia              5        0.596964               0.000000
4     meditazione              2        0.570294               0.242794
5   frequentatori              4        0.577810               1.000000
6           vagar              2        0.569018               0.094065
7       tribolata              2        0.571231               0.063369
8       squallore              4        0.579023               0.140263
9       desolante              1        0.566575               0.141226
10     melanconia              3        0.570364               0.258143
11    desolazione              1        0.560011               0.504067
12      ripensare              1        0.582017               0

In [58]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("solitario", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   riposandosi              3        0.761458               0.651997
1        rimoto              5        0.763529               0.069542
2      apparito              4        0.766728               0.501125
3      torrente              3        0.755459               0.095642
4       riempié              3        0.759987               0.563738
5      bagnarsi              2        0.761282               0.322079
6    ellesponto              2        0.756350               0.188706
7      spazioso              4        0.755537               0.019463
8        aronta              1        0.745702               0.364520
9    circondato              2        0.750003               0.000000
10     rilevato              4        0.779291               0.732235
11   monticello              3        0.769625               0.156046
12    ponendosi              1        0.763817             

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             ermo              5        0.609796               0.273984
1          solingo              5        0.587972               0.244740
2        errabondo              5        0.551952               0.137038
3            èremo              4        0.560322               1.000000
4       montericco              5        0.575484               0.286035
5        solitaria              5        0.546960               0.314357
6         sperduto              4        0.557143               0.201569
7         cenobita              5        0.548606               0.576492
8          isolato              4        0.545108               0.186668
9          ombroso              1        0.540647               0.146506
10  inselvatichito              2        0.554246               0.730622
11      nostalgico              1        0.543901               0.224421
12          vïator              1        0.537303  

### TOPOS: INFINITO

In [59]:
for periodo in lista_periodi:
    print("###" + periodo + "###")
    df_periodo = similar_words_per_topos("infinito", 5, 10, periodo)
    print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])

###1250_1454_Tardo_Medioevo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    inextimabile              5        0.701612               0.285900
1      imperfecto              5        0.703311               0.319985
2   exercitandosi              5        0.686628               0.250329
3         sicurtá              5        0.677653               0.333075
4   satisfaczione              4        0.673538               0.669928
5        aciecati              5        0.667218               1.000000
6        infinita              2        0.670964               0.966387
7   perfectissimo              1        0.674577               0.360015
8      inperfecto              3        0.661440               0.171568
9           bontá              4        0.668941               0.140434
10       durabile              1        0.669603               0.000000
11         vegono              2        0.660941               0.431502
12   disperazione              1 

               Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         sterminato              5        0.534858               0.000000
1            immenso              5        0.578585               0.265507
2          immensità              5        0.526885               0.460361
3             eterno              3        0.508217               1.000000
4         indefinito              5        0.534158               0.140998
5           increato              5        0.507554               0.153756
6      immensurabile              4        0.496289               0.258584
7        interminato              4        0.501196               0.536268
8           infinita              4        0.496572               0.143957
9   incommensurabile              5        0.497511               0.069735
10        sconfinato              2        0.491554               0.192217
11        adunamento              1        0.482781               0.091574
12      infinitesimo     

## ANALISI PER FASE LETTERARIA

In [79]:
os.chdir(path_letterario_w2v)
lista_periodi_letterari = ['1_duecento_poetica_e_prosa Medievo', '2_dolce_stil_novo', '3_petrarchismo', 
                           '4_boccaccio_e_umanesimo', '5_manierismo_e_barocco', '6_classicismo_arcadia', 
                           '7_illuminismo_neoclassicismo', '8_romanticismo', '9_verismo', '10_decadentismo', 
                           '11_avanguardie_primo_novecento', '12_neorealismo']

### TOPOS: PASTORE

In [80]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pastore", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        ovile              5        0.791488               0.361402
1    sacerdote              5        0.749896               0.113770
2         rege              4        0.747211               0.039580
3    conspecto              1        0.752088               0.101462
4        popol              4        0.742398               0.308371
5   consecrato              3        0.744413               0.421138
6     baptista              2        0.730330               0.118479
7    domenicho              1        0.731488               0.124645
8    appolline              1        0.731168               0.193004
9     lombardo              3 

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    mandriano              5        0.672622               0.057910
1    pescatore              5        0.668303               0.129492
2       zuanne              5        0.648066               0.105965
3      porcaro              5        0.644238               0.575143
4    contadino              5        0.647588               0.184005
5       gregge              5        0.629346               0.099208
6    guardiano              5        0.639037               0.265547
7        dejas              1        0.624541               0.430024
8   cacciatore              2        0.619942               1.000000
9      basilio              3        0.621329               0.068235
10       tanca              1        0.621504               0.000000
11       servo              2        0.620072               0.926701
12     eremita              2        0.619544               0.518287
13   pascolava              1     

### Altre parole correlate

In [81]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pastori", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            letti              5        0.802315               0.176423
1        sentirono              1        0.792743               0.078879
2           greggi              5        0.800458               0.635657
3         visitati              4        0.795245               0.457378
4        tenebrosi              5        0.786394               0.640081
5   congiugnimenti              5        0.790950               0.781273
6          cantano              4        0.785180               0.941191
7           scanni              4        0.789826               1.000000
8          bramosi              1        0.773228              

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      agnelli              5        0.696341               1.000000
1      paesani              5        0.695957               0.154800
2         tori              4        0.684785               0.470592
3      banditi              4        0.681363               0.749592
4     carbonai              2        0.685958               0.633024
5    guardiani              5        0.688036               0.241261
6        ovili              4        0.691799               0.446595
7     capretti              5        0.689256               0.706447
8      armenti              3        0.672428               0.685760
9    giovenche              2        0.674874               0.000000
10   lavoranti              3        0.687450               0.839459
11  lavoratori              1        0.668048               0.986411
12       porci              4        0.682533               0.511955
13      gregge              2     

### TOPOS: AMORE

In [82]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amore", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
       Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       solo              5        0.936816               0.561713
1      matto              5        0.934280               0.425585
2      servo              5        0.928907               0.765656
3      amico              1        0.919498               0.315638
4     stesso              3        0.921753               0.915873
5    potenza              4        0.922014               0.053990
6      perde              2        0.923888               0.571185
7       puro              3        0.924359               0.509035
8     signor              1        0.911537               0.586811
9   malvagio              5        0.916183               0.384032
10     amare              4        0.923602               0.000000
11     crede              2        0.929751               0.086142
12      teco              3        0.921531               0.737674
13       reo         

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       cieca              5        0.783658               1.000000
1     affetto              5        0.796283               0.740704
2      verace              5        0.781937               0.355140
3      stolta              5        0.783088               0.526303
4   stromento              2        0.776052               0.388945
5       error              2        0.773219               0.535269
6      spiace              1        0.755362               0.549333
7    spartano              1        0.754503               0.511788
8     inspira              1        0.754020               0.763947
9     stupida              2        0.764372               0.752704
10   dolcezza              3        0.764885               0.131983
11     geloso              1        0.784242               0.248792
12    fallace              1        0.773656               0.771226
13   fierezza              1        0.772512    

### Altre parole correlate

In [83]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amor", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
       Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       gioi              5        0.990760               0.627308
1    gioiosa              1        0.991189               0.286301
2      lasso              5        0.991410               0.712778
3     pregio              4        0.988350               0.320504
4      merzé              5        0.990742               0.452025
5      dolze              4        0.988360               0.317296
6       face              1        0.989196               0.122096
7        deo              5        0.990853               0.698150
8        ahi              1        0.988143               0.765674
9    merzede              4        0.988369               0.800441
10  pietanza              1        0.992679               0.882321
11   madonna              1        0.991931               0.048714
12     aggio              4        0.990310               0.321922
13      noia         

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      duolmi              4        0.846351               0.568486
1       farti              3        0.842723               0.005522
2    chieggio              1        0.837799               0.764990
3       eppur              2        0.839230               0.412808
4       speri              3        0.833477               0.125769
5     genitor              1        0.833797               0.516539
6    vorresti              3        0.836212               0.741563
7     ingrato              1        0.831688               0.880883
8     rimorso              3        0.842253               0.265289
9       puote              1        0.830982               0.563437
10      spera              3        0.844368               0.940637
11       util              1        0.845078               0.587510
12   orgoglio              2        0.842923               0.285526
13      neron              1        0.841004    

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        perdoni              5        0.851669               0.240009
1         carità              5        0.806364               0.054003
2       benedica              5        0.818057               0.556523
3           gesú              5        0.801754               0.402249
4          saulo              5        0.804967               0.167698
5      perdonami              5        0.799320               0.162982
6   misericordia              2        0.786546               0.341425
7          iddio              2        0.789723               0.182686
8    onnipotente              3        0.787563               0.140769
9         domine              4        0.788229               0.154193
10          abbi              2        0.794068               0.000000
11          zeus              1        0.781547               0.309311
12     aiutatemi              1        0.779970               1.000000
13    

In [84]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("innamorato", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        adone              5        0.823888               0.142467
1    inamorato              5        0.797548               1.000000
2      siccheo              5        0.794401               0.169621
3     parlante              3        0.792093               0.136125
4      loquela              2        0.771187               0.361781
5     pulcella              5        0.792889               0.367311
6    giannotto              2        0.768433               0.107050
7      garzone              3        0.774369               0.403364
8     servente              5        0.785583               0.000000
9     affabile              3 

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        taddei              5        0.781241               0.114946
1    innamorata              5        0.775759               0.315383
2       sposata              5        0.773258               0.083939
3       suddito              4        0.740786               0.000000
4       sposato              3        0.743383               0.039505
5        amasse              3        0.744344               0.129845
6      sposasse              5        0.748477               0.159297
7       esserlo              1        0.749721               0.415155
8          lali              4        0.742269               0.104061
9        odiava              3        0.738716               0.898281
10    cuginetta              1        0.735931               0.276887
11      confidò              1        0.734344               0.210961
12  giornalista              1        0.732954               0.321026
13       vedovo     

### TOPOS: CHIESA

In [85]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("chiesa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     chiamati              5        0.977893               0.736165
1         oggi              5        0.974952               0.779082
2    riverenza              1        0.970796               0.598992
3       sedere              4        0.968968               0.594292
4         olio              5        0.968778               0.267509
5      andasse              4        0.968891               1.000000
6       popolo              1        0.966186               0.545420
7    figliuole              2        0.966180               0.472083
8        pieni              1        0.965955               0.518548
9        fossa              3        0.967211               0.774588
10       entro              1        0.967807               0.000000
11       sedia              4        0.967090               0.236292
12       morta              3        0.965436               0.

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      cattolica              5        0.493499               0.270309
1      comunione              5        0.441581               0.093034
2       basilica              5        0.456841               0.156626
3   inquisizione              1        0.432623               0.491526
4       sinagoga              5        0.448789               0.276877
5           sede              1        0.421507               1.000000
6      gerarchia              1        0.421405               0.373683
7     apostolica              5        0.440892               0.279861
8         papato              2        0.439373               0.117986
9         ariani              2        0.428112               0.269015
10    obbedienza              2        0.435839               0.207537
11      celibato              2        0.448345               0.297480
12    scismatica              1        0.436543               0.000000
13  so

### Altre parole correlate

In [86]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("papa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     ambasciata              5        0.981670               0.531688
1           acri              5        0.974826               0.366299
2         istati              4        0.973945               0.000000
3           polo              5        0.979035               0.205343
4         domini              5        0.974766               1.000000
5      partirono              4        0.974737               0.461469
6          milia              1        0.975084               0.219639
7      figliuole              5        0.972169               0.119262
8   ambasciadori              4        0.970056               0.374930
9        vinegia              1        0.972459               0.527632
10          alau              2        0.972409               0.349909
11    alessandro              1        0.967056               0.471421
12        aveano              2     

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         tommaso              5        0.711175               0.139236
1          liccio              5        0.707830               0.268895
2       scomunica              5        0.690357               0.474221
3   confraternita              5        0.697647               0.291744
4   congregazione              5        0.693877               0.677997
5          biagio              4        0.680461               0.588249
6         vicario              2        0.678886               0.402310
7           bellu              2        0.675014               0.000000
8         patrono              2        0.684611               0.439640
9      portogallo              1        0.677524               0.091651
10        borbone              3        0.689086               0.048271
11       salomone              1        0.688718               0.920393
12     travestito              2        0.674184               0

In [87]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("prete", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        coiaio              5        0.812644               0.280584
1       pierino              5        0.779409               0.274552
2          mulo              5        0.774777               0.691440
3      soraggio              2        0.773137               0.054078
4       magnano              5        0.780712               1.000000
5      bastiano              5        0.794625               0.046362
6        notaro              5        0.783512               0.109999
7       fattolo              1        0.756955               0.453182
8       sillano              2        0.772892               0.067966
9   balestriero     

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        verrò              5        0.986650               0.672895
1        dissi              3        0.984830               0.589883
2        dirmi              5        0.986831               0.571439
3     compagno              5        0.984922               0.829476
4     costanza              1        0.984775               0.667986
5      aspetta              1        0.984265               0.756658
6     giuliano              1        0.984207               0.737838
7   margherita              1        0.984189               1.000000
8        pensò              1        0.983379               0.744885
9        borsa              2        0.983469               0.663883
10      eccomi              2        0.984194               0.211811
11         zio              1        0.981892               0.588241
12     rivolto              1        0.981299               0.680525
13   fidanzato              1     

In [88]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("frate", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         dono              3        0.979624               1.000000
1       leanza              3        0.979228               0.354832
2         ciel              5        0.977254               0.623269
3          van              3        0.977436               0.837990
4         poso              1        0.978633               0.697258
5   dispiacere              1        0.978219               0.914757
6          sor              2        0.976933               0.726245
7     corrotto              1        0.977947               0.876035
8        amaro              2        0.974736               0.715536
9       tuttor              1        0.977741               0.455401
10      corona              2        0.979267               0.996303
11       allor              2        0.980787               0.726219
12         fra              3        0.980752               0.

La chiave non esiste nel modello6_classicismo_arcadia
###7_illuminismo_neoclassicismo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      domenicano              5        0.849381               0.182666
1          messer              5        0.824634               0.272528
2        ambrogio              5        0.822667               0.411062
3         damiano              4        0.813345               0.278653
4      confessore              2        0.815648               0.000000
5    plenipotente              3        0.821789               0.075964
6          cierto              1        0.817414               0.258726
7         agnello              2        0.816383               0.235004
8      baldassare              2        0.815757               0.238912
9             gio              1        0.810451               0.295255
10          prete              2        0.830289               0.256341
11       girolamo              1        0.82610

In [89]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("dio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   onnipotente              5        0.914140               0.157135
1        grazia              5        0.918924               0.216705
2      speranza              5        0.893080               0.106980
3      servigio              4        0.866437               0.000000
4     solamente              3        0.874233               0.220457
5        divina              4        0.868134               0.231730
6      sapienza              4        0.868743               0.345855
7        contra              1        0.867141               0.482992
8     peccatore              3        0.863948               1.000000
9          luce              2        0.855579               0.130482
10        anime              1        0.863203               0.041806
11      cruccia              1        0.859887               0.927630
12         data              2        0.852815   

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            iddio              5        0.658482               0.046088
1     misericordia              5        0.572994               0.024261
2        redentore              5        0.482733               0.075204
3      provvidenza              4        0.479516               0.050029
4            cielo              2        0.460150               0.082740
5         creatore              3        0.454751               0.042096
6           cristo              4        0.461766               0.077158
7        peccatore              5        0.459985               0.073256
8           invoca              3        0.456865               1.000000
9            adora              2        0.453906               0.059889
10      santissima              2        0.451673               0.601381
11     onnipotente              1        0.452436               0.000000
12          pieghi              1        0.444527  

In [90]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("cristo", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         jesù              5        0.948043               0.207000
1        santa              5        0.944603               0.559242
2      spirito              5        0.937834               0.550681
3        madre              4        0.931322               0.000000
4   favellando              4        0.925389               0.182771
5        beati              5        0.935587               0.478241
6         gesù              5        0.933579               0.196125
7     vangelio              4        0.927373               0.069162
8     agostino              2        0.926060               1.000000
9     superbia              3        0.924498               0.396819
10    apostoli              4        0.935825               0.875902
11    gregorio              2        0.925759               0.631730
12     isidoro              1        0.922240               0.

La chiave non esiste nel modello6_classicismo_arcadia
###7_illuminismo_neoclassicismo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            gesù              5        0.776553               0.231336
1      protettore              5        0.739146               0.237505
2       cristiana              3        0.702061               0.193221
3         eretico              2        0.698797               0.000000
4           adamo              4        0.709225               0.271648
5       arciprete              5        0.709765               0.149007
6       cattolica              1        0.702013               0.230252
7        commenda              3        0.709130               0.038600
8        dichiarò              3        0.711078               0.110627
9         matilde              1        0.696638               0.114145
10     cattedrale              3        0.712756               0.117056
11      ricevesse              1        0.70119

### TOPOS: FELICITA'

In [91]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("felicità", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     infelicità              5        0.747572               0.041221
1          summa              5        0.725935               0.529777
2    beatitudine              5        0.711871               0.179752
3        miseria              5        0.706251               0.018888
4   tranquillità              5        0.705806               0.215883
5     conseguita              2        0.685297               0.171967
6    immortalità              4        0.688040               1.000000
7        assidua              3        0.674779               0.043930
8         sanità              1        0.675699               0.455753
9      int

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        innocenza              5        0.692141               0.614560
1        sovrumana              4        0.666010               0.554706
2      aspirazione              5        0.670387               0.164058
3          fidente              5        0.680461               0.259105
4        dedizione              2        0.655945               0.415850
5         faticosa              1        0.650343               0.100621
6       infelicità              4        0.663619               0.144432
7         pienezza              5        0.668924               0.760915
8         sovrasta              1        0.645428               0.235672
9        rinascere              1        0.643287               0.416178
10       tristezza              1        0.668412               1.000000
11       esclusiva              1        0.663603               0.313261
12         sicurtà              2        0.659692  

### Altre parole correlate

In [92]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("gioia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
       Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       core              5        0.982734               0.104179
1    madonna              5        0.982007               0.932431
2       gioi              5        0.980396               0.725895
3       noia              3        0.981783               0.182348
4     valore              2        0.981756               0.626293
5        meo              3        0.980554               1.000000
6         ca              4        0.980545               0.449902
7      merzé              2        0.980084               0.707256
8   tormento              3        0.978240               0.397321
9      lasso              5        0.977921               0.770769
10      loco              4        0.980903               0.909043
11      amor              2        0.982194               0.727076
12       cor              3        0.979749               0.668206
13      ched         

       Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   palpitar              5        0.943415               0.054699
1     divide              5        0.935501               0.651269
2     veggio              1        0.929750               0.129107
3      provò              5        0.937164               0.636127
4    vedessi              2        0.928616               0.636456
5    mirarti              2        0.927791               0.043223
6       moro              1        0.925375               0.293442
7    tacendo              3        0.928608               0.188893
8      empio              1        0.922588               0.369751
9       vene              1        0.922343               0.890538
10      gelo              2        0.941032               0.457596
11    costei              3        0.935494               0.167502
12     gioco              3        0.931608               0.009474
13   pietade              1        0.925683               0.00

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      muscoli              5        0.956192               0.056108
1         vene              5        0.948347               0.501623
2   freschezza              5        0.945715               0.356425
3           sè              2        0.939847               0.095574
4     infinita              3        0.939786               0.319131
5      sottile              1        0.936380               0.537417
6           dà              1        0.935707               0.556880
7       grazia              5        0.946155               0.275676
8    tenerezza              3        0.936015               0.791176
9      ardente              1        0.934813               0.464333
10        viva              1        0.940410               1.000000
11    profonda              1        0.937859               0.000000
12     fragile              2        0.940771               0.190583
13       piena              1     

### TOPOS: PATRIA/ITALIA

In [93]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("patria", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          greco              3        0.997170               0.565845
1         scrive              3        0.996290               0.841455
2       chiamati              2        0.997243               0.901705
3           savi              2        0.996791               1.000000
4         popolo              2        0.996505               0.711505
5         catone              1        0.997018               0.421173
6       gloriosa              3        0.995317               0.902080
7         marzia              3        0.995708               0.234701
8       chiamato              4        0.997611               0.386880
9      cittadini              1        0.996532               0.882078
10        ovidio              4        0.996803               0.905696
11       gentili

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        libertá              5        0.452237               0.661505
1         patrio              5        0.462100               0.131526
2      straniero              1        0.456794               0.301943
3       prodighi              5        0.446925               0.085762
4       salvezza              5        0.451733               0.429662
5      timoleone              4        0.443018               0.047969
6       redimere              3        0.435878               0.000000
7            prò              2        0.438423               0.239653
8      anassilao              1        0.437414               0.653022
9      ignominia              2        0.433676               0.050376
10         esuli              3        0.445889               1.000000
11  indipendenza              4        0.440488               0.466118
12        gloria              1        0.435039               0.167020
13    

In [94]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("italia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       bassi              3        0.998848               0.407839
1        rosa              1        0.998536               1.000000
2      intesi              1        0.998361               0.056528
3        visi              1        0.998288               0.000000
4      spazio              2        0.998499               0.731814
5       tolta              1        0.998218               0.803632
6       fenno              1        0.998212               0.294221
7     tornare              3        0.998372               0.390199
8     vergine              2        0.998247               0.626139
9        sovr              1        0.998067               0.459767
10     rimasa              2        0.998575               0.447587
11      freno              2        0.998511        

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      pannonia              5        0.656442               0.423388
1     belisario              5        0.645295               0.000000
2      germania              5        0.632915               0.841746
3     lombardia              5        0.640282               0.360986
4       narsete              5        0.625158               0.348818
5        spagne              3        0.620269               0.321990
6   cistiberina              2        0.611431               0.665986
7       lotario              1        0.605165               0.770581
8    vestrogoti              5        0.618212               0.272025
9        gallie              5        0.608684               0.555868
10      vandali              4        0.623208               1.000000
11      fiandra              2        0.610574               0.279383
12     alemagna              1        0.615826               0.486463
13      augusto     

### Altre parole correlate

In [95]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("stato", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        vista              5        0.947751               0.592849
1        certo              5        0.952976               0.239214
2     saccente              4        0.948025               0.657602
3      sentire              1        0.949773               0.128093
4        mette              4        0.953678               0.169442
5        grave              5        0.949910               0.526136
6        fallo              2        0.950274               0.027949
7      cortese              1        0.942094               0.508409
8        alcun              4        0.951645               0.691791
9         aver              3        0.944378               0.151905
10        puoi              1        0.960012               0.333315
11       voler              4        0.944781               1.000000
12        teco              1        0.952593               0.

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    abbastanza              5        0.455886               1.000000
1       dovette              2        0.445138               0.306820
2      vassallo              4        0.435459               0.530270
3   rivoluzione              2        0.434330               0.280123
4        aversi              1        0.436400               0.525096
5      avendolo              3        0.440919               0.381708
6     impiegato              3        0.437596               0.070201
7    rinunziato              2        0.433656               0.308115
8       credeva              1        0.434081               0.554714
9         abile              1        0.434040               0.291142
10       affari              1        0.441082               0.658952
11    nazionale              1        0.435539               0.000000
12     elettivo              2        0.433254               0.281894
13      assetto     

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          rimasto              4        0.453930               0.360308
1      autorizzato              5        0.448239               0.494020
2      applicabile              4        0.438971               0.573308
3           temuto              2        0.438797               0.621610
4        dimezzato              1        0.436990               0.212285
5         esistito              3        0.429908               0.167430
6   specificamente              2        0.433939               0.856047
7           curato              3        0.447710               0.342343
8        complotto              2        0.427931               0.721265
9        pensabile              2        0.428648               0.241372
10          andato              1        0.437062               0.032227
11      minacciato              1        0.423751               0.215802
12       criminale              1        0.422929  

In [96]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("impero", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      lignaggio              5        0.883212               0.529563
1         devoto              4        0.890922               0.776262
2      obbrobrio              4        0.881116               1.000000
3   riconoscenza              3        0.881971               0.545873
4          fruir              5        0.885748               0.602115
5        mendico              3        0.883227               0.334204
6          vanti              3        0.882246               0.000000
7        indutto              4        0.884596               0.563309
8        perfida              2        0.882144               0.547747
9        s

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        italico              5        0.821165               0.091897
1      fondatore              5        0.805339               0.510866
2        tributo              3        0.780168               0.112775
3          david              1        0.782453               0.374260
4     principati              1        0.777081               0.156572
5   incrollabile              3        0.779146               0.146755
6      magnanimo              1        0.776745               0.000000
7         catone              2        0.777084               0.250682
8       edificio              3        0.776617               0.220533
9      venerando              5        0.778808               0.137712
10       officio              2        0.787544               0.194522
11   dominazione              1        0.780896               0.693653
12   vendicatore              2        0.781712               0.225996
13    

In [97]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("nazione", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           tedesca              5        0.720539               0.176322
1        esercitato              2        0.701362               0.171149
2            superò              3        0.696043               0.168451
3              goti              4        0.688281               0.330448
4         coriolano              3        0.690424               0.319674
5      declinazione              2        0.685209               0.104160
6          prìncipi              4        0.686775               0.201875
7          nobilitá              4        0.683875               0.200767
8          aspirava              3        0.684078    

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   soppressione              5        0.775419               0.663665
1   cooperazione              4        0.778355               0.000000
2       penisola              4        0.775944               1.000000
3      sovranità              1        0.774589               0.129586
4        destini              3        0.760473               0.365678
5        assegna              1        0.768873               0.186381
6    solidarietà              3        0.763412               0.471669
7       sessuale              2        0.769148               0.051361
8      gerarchia              1        0.759275               0.323179
9    legittimare              1        0.758082               0.050006
10   istituzioni              1        0.759379               0.478154
11       utilità              1        0.759170               0.326881
12       posteri              4        0.764711               0.989937
13    

In [98]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("bandiera", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        rotta              4        0.982437               0.596980
1    spronando              5        0.982309               0.816133
2        ariva              5        0.980857               0.795788
3       drieto              5        0.977372               0.184378
4       abatte              5        0.978944               0.155380
5       gionge              2        0.978420               0.083099
6       corona              4        0.979264               0.285258
7      briglia              1        0.973997               0.630282
8     campagna              3        0.975943               0.000000
9       rivera              1        0.973509               0.433647
10     schiere              2        0

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      tricolore              5        0.780806               0.089011
1        patrono              3        0.691431               0.187297
2          duomo              4        0.689313               0.156597
3             ss              1        0.680326               0.252740
4       centrale              1        0.680162               0.156850
5       coccarda              3        0.691635               0.292816
6        termina              1        0.673364               0.233280
7         trofeo              5        0.698472               0.202096
8     plebiscito              1        0.671014               0.257027
9   mezzocannone              1        0.670849               0.099114
10      vaticano              4        0.704969               0.078552
11      pantheon              4        0.684965               0.196841
12        stemma              1        0.684150               0.251029
13    

### TOPOS: GUERRA

In [99]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("guerra", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      venuta              5        0.945740               0.465484
1    stagione              5        0.952852               0.576805
2     apresso              5        0.947778               0.605647
3      sforzo              5        0.944592               0.885584
4      giorno              3        0.930489               0.354329
5        mise              1        0.919115               0.524063
6       aiuto              3        0.929148               0.556217
7          fe              3        0.933172               0.638375
8       croia              2        0.924219               0.495713
9     vertute              1        0.908506               0.468618
10    giammai              1        0.936815               0.512714
11    partita              1        0.930936               0.615387
12     romore              2        0.933463               0.592696
13     

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        raggi              5        0.887572               0.598671
1       grecia              5        0.906356               0.292150
2        ruine              1        0.892249               0.335224
3       stragi              4        0.882339               0.338170
4        sfere              2        0.876868               0.331072
5       aperte              1        0.888558               0.328176
6        genti              3        0.874848               0.287412
7        sonni              1        0.887225               0.251331
8       centro              3        0.877005               0.136971
9        danni              2        0.876690               0.289523
10  vagheggiar              1        0.874349               0.555549
11      ascoso              2        0.874964               0.158135
12     minacce              1        0.867678               1.000000
13       risse              2     

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        futurista              5        0.924540               0.068777
1        futuristi              5        0.894426               0.049611
2           numero              5        0.879234               0.142634
3      rivoluzione              5        0.881850               1.000000
4          nazioni              5        0.880667               0.577350
5          società              5        0.881194               0.284307
6         italiani              3        0.873547               0.323735
7          massimo              5        0.872298               0.266787
8          libertà              5        0.876356               0.384242
9   conflagrazione              2        0.856824               0.364127
10           amore              2        0.858190               0.000000
11         varietà              1        0.862897               0.400841
12            oggi              1        0.853290  

### TOPOS: FIGURA FEMMINILE

In [100]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("donna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
      Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        eo              4        0.962995               0.480041
1       poi              4        0.964170               0.342900
2     gioia              4        0.963259               0.806243
3       meo              5        0.963653               0.810202
4      gioi              3        0.961215               0.876024
5   madonna              3        0.962467               0.688361
6      viso              1        0.959243               0.903890
7       ver              3        0.958390               0.137540
8       par              3        0.954688               0.820411
9       cor              5        0.962333               0.945480
10     loco              1        0.955992               0.516308
11    senza              3        0.959305               0.000000
12       ca              4        0.957940               1.000000
13       en              3        0

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      prassede              5        0.570562               0.170916
1        costei              5        0.577325               0.201795
2       femmina              4        0.543571               1.000000
3     fanciulla              5        0.539246               0.662243
4          dama              5        0.552259               0.486900
5        pudica              5        0.526367               0.545170
6         livia              1        0.534009               0.000000
7      leonilda              2        0.527435               0.327750
8       ragazza              5        0.534135               0.416322
9     contadina              3        0.518088               0.279623
10       orfana              4        0.527003               0.321921
11    figliuola              1        0.503112               0.908873
12       vedova              2        0.521606               0.313355
13  mariantonia     

In [101]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("madre", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           jesù              5        0.963897               0.983334
1           dopo              5        0.958349               0.716400
2          adamo              5        0.963420               0.464843
3            eva              5        0.955406               0.350826
4       apostoli              5        0.955776               0.024374
5        dannati              5        0.945306               0.806258
6         riceve              3        0.944565               0.386538
7            job              4        0.942660               0.703492
8          sposa              1        0.939291               0.375654
9       eternale              4        0.946329               0.025656
10     peccatori              1        0.938237               1.000000
11  gastigamenti              1        0.935994               0.258447
12    domeneddio              1     

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       figlia              5        0.781080               0.308380
1      sorella              5        0.703112               0.215224
2      nutrice              5        0.692970               0.086984
3         orba              5        0.688864               1.000000
4      badessa              5        0.689937               0.352197
5     consorte              4        0.677661               0.000000
6        sposa              5        0.688921               0.331390
7       moglie              4        0.695712               0.300238
8      vergine              2        0.666821               0.037103
9        prole              5        0.678424               0.221013
10      povera              1        0.654188               0.177136
11    rosmonda              1        0.651987               0.020400
12     nudrice              1        0.649299               0.408353
13      regina              1     

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       nonna              5        0.651260               0.166900
1       paola              5        0.607536               0.056791
2       mamma              5        0.605009               0.065876
3    natalina              5        0.590481               0.124194
4      figlia              3        0.586385               0.142823
5       terni              3        0.584809               0.066006
6      vedova              1        0.593250               0.388890
7         zia              4        0.586574               0.000000
8       balbo              1        0.588207               0.196936
9      elvira              2        0.584975               1.000000
10    sorella              1        0.577318               0.422280
11       pina              4        0.584041               0.080984
12    lisetta              2        0.580571               0.111846
13    pirrone              2        0.582383    

In [102]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("femmina", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      miseria              5        0.938001               0.334809
1     creatura              5        0.939198               0.911487
2        colla              5        0.935673               1.000000
3     apostolo              5        0.912710               0.084619
4       ventre              3        0.906555               0.242112
5      soffera              5        0.916229               0.193763
6      primaio              4        0.911573               0.000000
7      diventa              3        0.909018               0.969933
8      diavolo              1        0.902747               0.306260
9   tentazioni              5        0.908531               0.397651
10      riceve              2        0.915212               0.423196
11      imperò              4        0.907093               0.229883
12      toglie              2        0.906550               0.

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       balba              5        0.709918               1.000000
1       teseo              3        0.664732               0.652967
2     guercia              5        0.699521               0.007684
3      pecora              4        0.683921               0.799216
4     immonda              4        0.674307               0.312291
5    leonessa              4        0.666467               0.160873
6      vipera              2        0.675229               0.171134
7        luco              5        0.666998               0.106981
8    mammella              3        0.663676               0.070711
9       zoppa              3        0.666603               0.000000
10      cagna              4        0.691948               0.497949
11      serpe              3        0.674909               0.228315
12  meretrice              1        0.658137               0.323914
13    mannaia              2        0.661798    

In [103]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("sposa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            sposo              5        0.989600               0.425311
1         eternale              5        0.983481               0.212073
2          peccata              5        0.978753               0.382221
3          dannati              5        0.977662               0.264323
4           riceve              2        0.976260               0.515087
5       dolcissimo              5        0.980230               0.240548
6         disidero              1        0.975384               0.683362
7            beata              5        0.978495               0.103349
8       domeneddio              1        0.974925               0.240107
9            maria              4        0.974175               0.201751
10         visione              1        0.975560               0.158169
11         pecunia              1        0.973636               0.438659
12        

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     consorte              5        0.841748               0.253941
1   innamorata              3        0.816657               0.281374
2          zia              5        0.822624               0.364457
3        balia              1        0.816426               0.095782
4        sposo              2        0.807330               0.348388
5   archangela              2        0.806610               0.480315
6      bambino              4        0.821553               0.632517
7         rise              1        0.802887               1.000000
8   dolcissima              2        0.809686               0.174938
9      nutrice              2        0.806518               0.825693
10       casta              2        0.827468               0.013410
11  verginella              1        0.825249               0.477864
12  giovinetta              4        0.821458               0.354330
13     ancella              1     

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      figlia              5        0.973613               0.088469
1      povero              5        0.963998               0.232870
2        sire              4        0.947820               0.166631
3     giovine              5        0.945908               0.000000
4       carlo              3        0.944738               0.249853
5    ministro              1        0.947740               1.000000
6   intestino              5        0.943773               0.282342
7      evviva              3        0.945377               0.110173
8       sposo              1        0.941674               0.263948
9    ludovico              3        0.942816               0.207230
10     figlio              4        0.946600               0.181813
11        xii              3        0.945657               0.312620
12   chiamare              2        0.942245               0.182972
13    sovrano              1        0.943799    

### TOPOS: NOTTE/SERA

In [104]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("notte", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     incontro              4        0.965529               0.614961
1         mese              4        0.966566               0.094750
2         olio              3        0.964706               0.682206
3        ferro              4        0.963789               0.762786
4      vivanda              4        0.967877               0.142843
5        caldo              4        0.964120               0.833678
6       istare              2        0.957524               0.277994
7        monti              1        0.961584               0.437749
8   forestieri              1        0.961154               0.745574
9        beono              3        0.962568               0.608763
10     dugento              2        0.965486               0.835885
11      albero              1        0.961273               0.420329
12       entro              2        0.957860               0.

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      soletto              5        0.769116               0.779167
1        barca              5        0.760633               0.125791
2         lito              3        0.742153               0.461558
3      discese              4        0.743700               0.001047
4         buia              1        0.735537               0.338299
5       grotta              3        0.738911               0.000000
6        lemno              4        0.738868               0.320510
7      giaceva              1        0.730705               0.213636
8        palme              3        0.744682               0.212505
9      pioggia              2        0.741703               0.298031
10      trombe              2        0.741256               0.409724
11          lì              3        0.746443               0.363873
12  tempestosa              4        0.743517               0.286310
13  glicistoma              1     

In [105]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("sera", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      mangia              4        0.990537               0.519467
1        vena              5        0.990218               0.400355
2    volevano              2        0.990192               0.383492
3      guasta              2        0.991578               0.274578
4    talvolta              2        0.989282               0.640097
5      uscire              3        0.990435               0.360910
6      pruova              2        0.989551               1.000000
7       spese              3        0.989089               0.222065
8       panno              3        0.989022               0.598164
9       facea              4        0.989586               0.811889
10     saette              2        0.991521               0.324361
11    addosso              2        0.988919               0.538190
12   famiglia              1        0.988041               0.000000
13     

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     mattina              5        0.953515               0.197753
1      lunedì              5        0.941518               0.339346
2      avanti              5        0.929715               0.052701
3      dimani              3        0.925412               0.150734
4   mercoledì              5        0.934434               0.384952
5      sabato              5        0.926925               0.350140
6        ieri              5        0.934236               0.595616
7    tavolino              1        0.930067               0.202907
8     giovedì              3        0.928321               1.000000
9    corriere              5        0.933916               0.356817
10    martedì              4        0.929868               0.000000
11     scorsa              1        0.936643               0.215665
12        ier              1        0.935269               0.103697
13      piego              1        0.923849    

### Altre parole correlate:

In [106]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("tenebre", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
      Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    vedrai              3        0.996093               0.773781
1     pasce              1        0.996164               0.505917
2    piange              3        0.996104               0.244838
3   alberga              3        0.996606               0.291657
4   piaggia              4        0.996812               0.606113
5    frondi              1        0.995616               1.000000
6     poggi              5        0.995655               0.675402
7      sòle              3        0.995604               0.383533
8    súbito              3        0.995770               0.350440
9      volo              1        0.994643               0.306183
10   debile              2        0.997179               0.391431
11   

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   crepuscolo              5        0.691380               0.161745
1     notturna              2        0.677026               0.452046
2         bujo              3        0.658319               0.377723
3       nebbia              2        0.655817               0.224444
4     procella              4        0.664608               0.326428
5    squallore              1        0.656034               0.000000
6        ombre              4        0.669450               1.000000
7        tetra              1        0.652803               0.061169
8      silenzi              3        0.656325               0.553202
9        dense              1        0.651359               0.187019
10      baleno              3        0.668022               0.238274
11    caligine              4        0.662667               0.283133
12   tenebrosa              1        0.652018               0.165748
13     incalza              3     

In [107]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("ombra", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
       Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      prova              1        0.997458               0.852202
1      mette              1        0.996907               0.724181
2       amar              1        0.996798               0.437572
3     cagion              2        0.996226               0.251182
4    pesanza              1        0.996686               0.353217
5        puo              1        0.996529               0.295689
6      parer              1        0.996497               0.127500
7     podere              1        0.996394               0.290081
8   speranza              3        0.996015               0.544788
9      poria              4        0.994947               0.798672
10     segno              4        0.996217               0.000000
11     lieta              3        0.996559               0.2060

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      stilla              5        0.905399               0.251377
1    mormorar              5        0.911597               1.000000
2     securtà              3        0.905360               0.079675
3        arde              2        0.904596               0.560154
4       orror              1        0.900365               0.522519
5       svela              1        0.898333               0.604393
6     beltade              3        0.906237               0.607517
7     alterno              1        0.895722               0.564865
8        tace              2        0.899337               0.769851
9      tragga              1        0.894125               0.700140
10    solingo              3        0.911204               0.871870
11       alma              1        0.914164               0.627958
12   susurrar              3        0.912479               0.311373
13      tosco              3        0.910198    

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       spariva              3        0.700394               0.088237
1   risplendeva              3        0.707367               0.397522
2      biancore              4        0.694762               0.316510
3        sagoma              3        0.696272               0.000000
4    illuminava              4        0.702419               0.140781
5       scialbo              3        0.702853               0.307359
6          elce              1        0.694485               1.000000
7         landa              2        0.708777               0.683551
8      striscia              3        0.701452               0.570004
9      bagliore              4        0.697209               0.643849
10       smorta              2        0.698263               0.780468
11       torcia              1        0.688471               0.780920
12     lampione              4        0.704650               0.488072
13  abbagliante     

### TOPOS: FORTUNA

In [108]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("fortuna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        oltra              2        0.992418               0.122994
1         deve              3        0.991277               0.467044
2        penso              1        0.989118               0.402234
3      mostrar              1        0.989098               0.463823
4       vaglia              1        0.988721               0.454751
5         ochi              3        0.991118               0.264943
6     becchina              2        0.989272               0.725914
7        vento              2        0.990646               0.000000
8         fate              2        0.988389               0.322341
9      vedesse              3        0.990938               0.257194
10      poiché              4        0.992744               0.762702
11       laido              3        0.993866               0.132266
12       quant              1        0.991879               0.

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         effetti              5        0.713349               0.881375
1          scarsa              2        0.712225               0.458505
2        profonde              4        0.701221               0.628298
3        colleghi              3        0.702681               0.493376
4       accademia              1        0.709130               0.174273
5   rincrescevoli              3        0.695237               0.700066
6      svantaggio              2        0.693989               0.775077
7          comodi              1        0.705349               0.103656
8           animi              2        0.695330               0.697583
9         perizia              1        0.701204               1.000000
10       aggiunga              2        0.694090               0.416880
11         scarso              1        0.687798               0.000000
12          pegni              1        0.687647               0

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          siccome              2        0.990209               0.715979
1           alcuno              1        0.990874               0.000000
2           pagare              4        0.987682               1.000000
3      particolare              1        0.990586               0.724367
4          credere              2        0.988770               0.280362
5         rispetto              1        0.990361               0.775200
6         riuscita              4        0.986790               0.573283
7            farsi              2        0.988568               0.526566
8       importante              3        0.989029               0.499599
9          abilità              5        0.988216               0.565335
10           causa              4        0.988295               0.753595
11            data              1        0.986881               0.802847
12         vederlo              1        0.986128  

### TOPOS: NOIA

In [109]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("noia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    pietanza              3        0.994617               0.620781
1     gioiosa              5        0.994250               0.416870
2     madonna              3        0.993773               0.783344
3         fin              1        0.991596               0.795524
4         ahi              2        0.992017               0.454700
5      savere              4        0.993353               0.690228
6    tormento              1        0.990361               0.657373
7       aggio              4        0.993163               0.544056
8   dannaggio              3        0.992230               0.342072
9      valore              1        0.989673               0.483558
10     piacer              1        0.995296               0.397577
11      inver              3        0.993787               0.692688
12   coraggio              1        0.994146               0.814025
13    a

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          abuso              3        0.953851               0.557530
1       cicalare              3        0.954057               0.808560
2          scema              4        0.954621               1.000000
3   interrompere              2        0.958279               0.000000
4        mestier              1        0.952945               0.569437
5        critica              1        0.951543               0.424428
6    tollerabile              1        0.950211               0.716104
7        dispero              1        0.950104               0.562639
8       evitando              4        0.952105               0.140219
9      mostrarsi              2        0.946182               0.341434
10     apparente              2        0.958203               0.575115
11      sollievo              1        0.961298               0.456860
12    nutrimento              1        0.956780               0.175049
13    

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       abbondante              3        0.997078               0.778562
1            corno              1        0.997411               0.603377
2            morso              1        0.997363               0.266252
3          arrosto              1        0.997188               0.843373
4           docile              2        0.997148               0.531856
5             nodo              1        0.997001               0.699136
6              ohi              1        0.996907               0.974551
7      provvidenza              1        0.996858               0.471216
8       fantastico              2        0.996841               0.819889
9          scrigni              1        0.996707               0.000000
10           preme              1        0.997454               0.639059
11       aumentava              2        0.996659               0.503071
12       intestini              2        0.996413  

### TOPOS: TEMPORALE/PIOGGIA

In [110]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("temporale", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      spirituale              5        0.870049               0.000000
1     esaltazione              5        0.813868               0.101887
2        mancassi              1        0.813661               0.121116
3      esterminio              2        0.802967               0.159197
4   conservazione              2        0.796260               0.154394
5       deprimere              4        0.805943               0.195622
6        pretende              4        0.802910               0.072824
7       mantenuta              2        0.803871               0.301173
8        esaltare              2        0.794811               0.115354


              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           fiumano              5        0.766973               0.687085
1        illuminare              1        0.750365               0.546333
2     purificazione              2        0.748208               0.105103
3           funesta              4        0.750249               0.602929
4        pestilenza              3        0.749280               0.600225
5       battesimale              1        0.729399               0.778600
6       liberazione              2        0.754910               0.679915
7         memoranda              1        0.725369               0.860555
8          propizio              1        0.724774               0.549282
9         trasporta              2        0.736349               0.957083
10       dannazione              3        0.762644               0.958928
11          sorgerà              2        0.748563               0.578895
12      trionfatore              2    

In [111]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pioggia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     scender              1        0.998108               0.682835
1        fere              1        0.998072               0.500152
2       tenea              3        0.998165               0.219471
3       sante              1        0.997979               0.334724
4      fuggir              2        0.998005               0.268621
5        uscì              3        0.997715               1.000000
6      tesoro              2        0.997849               0.679100
7        fera              1        0.997756               0.467878
8      soglia              1        0.997732               0.403975
9        pesa              2        0.997748               0.746435
10       temo              2        0.998238               0.358996
11  selvaggia              1        0.998454        

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       soffio              5        0.935323               0.272357
1      rugiada              5        0.927376               0.277837
2        dorso              2        0.908706               0.413660
3       pelago              1        0.911159               0.054472
4   lentamente              2        0.911113               0.697640
5      soletto              3        0.915573               0.552025
6        tuono              5        0.921043               0.345418
7        mugge              1        0.907173               0.542954
8         fumo              3        0.916896               0.139980
9        verno              4        0.922194               0.000000
10  tempestosa              4        0.920656               0.537934
11      flutto              1        0.926811               0.476044
12       stelo              1        0.924250               0.141618
13       fosca              1     

### TOPOS: GIOVINEZZA

In [112]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("giovinezza", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          volge              5        0.994807               0.616977
1         angelo              5        0.993763               0.460584
2     conditione              5        0.993164               0.040332
3          narro              4        0.993231               0.406099
4         truova              4        0.995908               0.510098
5       gloriosa              5        0.992688               0.648390
6         quinta              2        0.993389               0.401629
7         scrive              2        0.993416               0.425179
8        gentili              1        0.992557               0.545923
9      particola              1        0.992387               0.577819
10       invidia              4        0.993365               0.522000
11       apresso

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       gioventù              5        0.702801               0.414166
1       maturità              5        0.663543               0.000000
2    adolescenza              5        0.651999               0.104606
3         purità              2        0.630031               0.229561
4      rifiorire              4        0.630849               1.000000
5        nutrita              5        0.641548               0.264880
6   fanciullezza              4        0.635991               0.417253
7         goduta              5        0.652267               0.073233
8    rifioritura              2        0.621714               0.547284
9    spensierata              3        0.629879               0.293647
10      virilità              3        0.631766               0.160831
11    celestiale              1        0.616084               0.170258
12        godevo              3        0.639054               0.064925
13    

In [113]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("giovane", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         vecchio              3        0.981326               0.114623
1          nacque              1        0.980196               0.475166
2            pari              1        0.979821               0.465353
3         ricorda              1        0.979671               0.744399
4         vincere              3        0.981060               0.084269
5         sovrano              2        0.978754               0.407981
6          andate              1        0.977552               0.373603
7          costui              1        0.977239               0.046833
8         intendi              1        0.976806               0.312206
9           muore              1        0.976653               0.139109
10          croce              2        0.980220               0.338898
11     imperadore              1        0.979862               0.042221
12       veggendo      

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      successore              5        0.855468               0.057643
1   liechtenstein              4        0.835227               0.421013
2          spagna              2        0.818190               0.423191
3     primogenito              5        0.823551               0.471173
4         kaunitz              5        0.822826               0.276535
5        sassonia              3        0.815527               0.891595
6         eugenio              1        0.818781               0.308828
7         potenza              4        0.823696               0.356297
8    irreparabile              4        0.820419               0.600853
9       cattolica              2        0.826108               0.000000
10     elettorale              3        0.826218               0.327486
11      estherazi              4        0.828651               0.373444
12           majo              2        0.821365               0

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        robusto              5        0.675409               0.469188
1     giovanotto              5        0.677471               0.417466
2       gioviale              2        0.632492               0.459874
3           giua              4        0.644292               1.000000
4   intelligente              4        0.636418               0.251561
5        anziano              3        0.642463               0.447910
6   servizievole              5        0.642083               0.000000
7         lattes              4        0.635239               0.316883
8        ragazzo              2        0.630862               0.428968
9          nancy              3        0.627190               0.297268
10     austriaco              1        0.645011               0.163336
11      ventenne              1        0.625836               0.385936
12    innamorata              1        0.617374               0.364089
13    

### TOPOS: BACIO

In [114]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("bacio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     pentito              5        0.919054               0.377243
1        nutr              1        0.926380               0.290328
2         dam              2        0.916943               0.533461
3       polin              3        0.921898               0.579116
4        nebb              2        0.925489               0.535012
5     celarsi              4        0.913869               0.555295
6   guerriera              2        0.914542               0.648084
7     uditemi              1        0.915508               0.356603
8        fulc              1        0.913260               0.584947
9        erof              1        0.91

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    abbraccio              5        0.710653               1.000000
1      carezza              5        0.585199               0.248923
2         baci              5        0.584267               0.090020
3      baciare              4        0.578295               0.155349
4    effusione              4        0.560027               0.213436
5   baciandomi              4        0.568520               0.000000
6      jackson              4        0.554969               0.135481
7       baciai              4        0.557527               0.091624
8   stringermi              1        0.559941               0.254466
9        desti              5        0.562970               0.020059
10     guancia              1        0.557356               0.122363
11       baciò              3        0.557797               0.104876
12      vagito              1        0.546159               0.089004
13      ciocca              1     

### TOPOS: LUNA

In [115]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("luna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          montare              5        0.992423               0.864166
1             alti              5        0.991257               0.403899
2            torre              4        0.988095               0.086558
3          animali              4        0.987304               0.312759
4            stati              3        0.987932               0.337084
5        scritture              4        0.987533               1.000000
6             cari              1        0.986521               0.000000
7          tornano              3        0.986788               0.295361
8   pellegrinaggio              5        0.986372               0.002221
9           tenuti              1        0.985409               0.201772
10           vende              4        0.987028               0.359563
11           cerca              1        0.986730               0.430624
12        

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         nube              5        0.835472               0.451622
1      pioggia              4        0.821289               0.482948
2         nera              5        0.818939               0.375089
3       stelle              5        0.836513               0.251942
4      azzurro              5        0.820366               0.067677
5       nebbia              5        0.822031               0.697748
6       piante              1        0.819653               0.508168
7   lentamente              3        0.827613               0.403174
8        palla              1        0.818386               0.964647
9         lago              1        0.809061               0.384383
10    ghiaccio              3        0.818112               0.427408
11     rugiada              2        0.812424               0.482974
12      raggio              3        0.821128               0.477513
13       verde              1     

### TOPOS: SILENZIO

In [116]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("silenzio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   piacevolmente              1        0.770017               0.495963
1          tofano              4        0.779988               0.831936
2      fortarrigo              4        0.773583               0.771573
3         ascoltò              1        0.753191               0.559008
4        anichino              2        0.765213               0.801054
5      stupefatto              1        0.747970               0.307230
6        vedendol              4        0.766235               0.572632
7       dipartito              2        0.754003               0.924002
8           sadoc              2        0.758069               0.429274


            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           regnò              5        0.564504               0.004114
1   raccoglimento              5        0.486770               0.017503
2             toc              4        0.468045               0.076606
3         silente              4        0.464424               0.075341
4      solitudine              5        0.475421               0.091903
5         regnava              5        0.481222               0.000000
6       incombeva              3        0.458661               0.320907
7        ripiombò              1        0.457698               0.272432
8      cicaleccio              2        0.452041               0.088177
9        rintocco              1        0.457078               1.000000
10   imbarazzante              2        0.458806               0.061514
11       successe              4        0.456759               0.066928
12         quiete              2        0.450037               0

### TOPOS: AMICIZIA

In [117]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amicizia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    benivolenza              5        0.754063               0.135565
1   congiunzione              5        0.669457               1.000000
2     inimicizia              5        0.690848               0.097482
3         unione              5        0.673004               0.000000
4    coniunzione              5        0.667570               0.160788
5         amistà              5        0.666914               0.142073
6        vincolo              5        0.660762               0.011103
7   confederarsi              4        0.653246               0.098197
8    familiarità              3        0.654366               0.372975
9      par

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      benevolenza              5        0.774920               0.326398
1         simpatia              4        0.750993               0.383584
2          merighi              4        0.756998               0.232779
3         essergli              1        0.759320               0.312556
4       confidente              4        0.751816               1.000000
5      conquistata              3        0.738469               0.170314
6        antipatia              1        0.742012               0.517362
7         tolosani              4        0.744743               0.238537
8        lusingato              5        0.742929               0.162710
9         ammirata              2        0.738194               0.224192
10           erlis              2        0.737684               0.222138
11         chiesta              3        0.735203               0.355218
12   accontentarsi              1        0.745412  

In [118]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amico", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
       Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       caro              4        0.970296               0.408932
1        ave              5        0.963122               0.399610
2       dono              2        0.963994               1.000000
3    partire              4        0.961227               0.116070
4        car              3        0.963009               0.480792
5      fusse              5        0.962279               0.204630
6     figlio              4        0.960642               0.339490
7      lauda              1        0.965485               0.047187
8        ven              2        0.961918               0.603575
9      ormai              5        0.962655               0.016947
10    stesso              4        0.965172               0.182685
11   mercede              1        0.958364               0.310619
12  villania              2        0.958159               0.370454
13     torna         

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     gemello              5        0.774194               0.175770
1   servitore              3        0.690547               0.067535
2   pensateci              5        0.706631               0.146109
3       bassà              5        0.688053               0.135971
4       abate              3        0.674505               0.287789
5     tenente              4        0.678849               0.122161
6     buzzano              1        0.668021               0.058593
7     carlani              2        0.666214               0.107216
8      antico              3        0.669691               0.107527
9   carissimo              1        0.660333               1.000000
10  veramente              3        0.663948               0.059380
11      amate              3        0.670918               0.194721
12  perdonate              2        0.675074               0.133562
13       orsù              2        0.670609    

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    assistente              5        0.689027               0.735974
1    psichiatra              4        0.677940               0.393153
2        turati              3        0.683727               0.987393
3       rasetti              5        0.679428               0.000000
4   inquisitore              5        0.689180               0.875437
5   affezionato              2        0.682808               0.532336
6    conoscente              5        0.689339               0.199246
7          bisi              1        0.669532               1.000000
8        amedeo              3        0.674328               0.678615
9        cugino              4        0.681390               0.656659
10        belom              2        0.686258               0.461495
11    letterato              3        0.679942               0.588705
12         gino              4        0.676019               0.993572
13     avvocato     

### TOPOS: SOLITUDINE

In [119]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("solitudine", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       domestica              4        0.772499               0.734082
1      maninconie              3        0.760398               0.090135
2         inerzia              1        0.755079               1.000000
3     malivolenza              2        0.752965               0.612560
4      perversità              4        0.753102               0.662484
5         desidia              5        0.751987               0.481145
6   infelicissima              3        0.747279               0.624609
7        sfrenata              2        0.743747               0.516085
8         mondana              2        0.744957               0.427300


            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          quiete              5        0.679180               0.326800
1       incombeva              5        0.632032               0.513832
2     desolazione              5        0.643309               0.309181
3      sconfinata              2        0.629779               0.111613
4        regnasse              3        0.630868               0.360990
5           oblìo              5        0.624597               0.093577
6      sconfinato              1        0.613291               0.193263
7   inaccessibile              2        0.613633               0.252953
8           oblio              3        0.618409               0.176345
9   raccoglimento              2        0.619958               0.260212
10     nostalgico              2        0.624061               1.000000
11     isolamento              2        0.612302               0.197341
12     tetraggine              1        0.605398               0

In [120]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("solitario", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
La chiave non esiste nel modello3_petrarchismo
###4_boccaccio_e_umanesimo###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       rimoto              5        0.841604               0.247196
1    salvatico              5        0.838001               0.238584
2      solingo              5        0.829501               0.259515
3      inculto              4        0.799616               0.111137
4    altissimo              3        0.799928               0.080375
5     rilevato              4        0.801287               0.272455
6        ameno              5        0.799451               0.041272
7     pertugio              1        0.797679               0.079937
8      segnato              2        0.797406               0.241560
9        tondo              1 

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   antichissimo              5        0.757139               0.425543
1        passero              5        0.745135               0.870176
2        errante              5        0.763026               0.340540
3          natio              4        0.726767               0.210614
4          vasto              3        0.730589               0.276126
5           ermo              5        0.741488               0.306256
6       edificio              3        0.731569               0.524997
7       ricovero              1        0.719232               1.000000
8          vetta              1        0.718595               0.592802
9       maestoso              2        0.725980               0.712264
10       isolato              1        0.749979               0.000000
11       etrusco              3        0.744562               0.182999
12         aereo              1        0.736978               0.094002
13    

### TOPOS: INFINITO

In [121]:
for periodo in lista_periodi_letterari:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("infinito", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###1_duecento_poetica_e_prosa Medievo###
La chiave non esiste nel modello1_duecento_poetica_e_prosa Medievo
###2_dolce_stil_novo###
La chiave non esiste nel modello2_dolce_stil_novo
###3_petrarchismo###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      disparte              1        0.994877               0.318918
1         arbor              5        0.994182               0.662904
2        pargli              3        0.994809               0.031838
3   biastemando              2        0.994830               0.372774
4      lasciata              4        0.994308               0.296133
5         bande              2        0.994925               0.623232
6         dando              1        0.993515               0.591174
7          tela              3        0.994125               0.263177
8         tenea              3        0.994230               0.351257
9     disciolta              1        0.993314               0.148200
10        abito            

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         infinita              5        0.692440               0.276861
1          immenso              5        0.678041               0.000000
2       divisibile              5        0.653088               0.034961
3       concepisce              4        0.650815               0.316348
4         immagina              1        0.645730               0.456715
5        immanente              3        0.648147               0.188519
6       indefinito              4        0.649200               0.230511
7           innato              5        0.652568               0.749843
8         infinità              3        0.640515               0.128053
9   potenzialmente              4        0.646746               0.313073
10        ingenito              2        0.657960               0.248794
11     immortalità              1        0.642250               1.000000
12          motore              2        0.643247  

## ANALISI PER AUTORE

# ANALISI CADE

## ANALISI PER PERIODI STORICO

In [9]:
os.chdir(path_storico_cade)

In [10]:
lista_periodi_cade = ['CADE_1250_1454_Tardo_Medioevo_cade', 'CADE_1454_1559_Rinascimento_cade', 'CADE_1559_1707_Seicento_cade',
                      'CADE_1707_1796_Settecento_cade', 'CADE_1796_1814_Periodo_napoleonico_cade', 
                      'CADE_1814_1860_Risorgimento_cade', 'CADE_1861_1914_L\'Italia_liberale_cade', 
                      'CADE_1915_1922_Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra_cade', 
                      'CADE_1922_1945_Ventennio_Fascista_Seconda_Guerra_Mondiale_cade', 
                      'CADE_1945_presente_La_Repubblica_Italiana_cade']

### TOPOS: PASTORE

In [11]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pastore", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      trentino              5        0.480828               1.000000
1       agapito              5        0.501311               0.209217
2       gattivo              5        0.466049               0.351550
3       commuta              5        0.449576               0.126886
4        vacava              1        0.461700               0.246865
5   vendereccia              4        0.440106               0.419166
6    arricchiva              3        0.434229               0.604117
7     reggitore              1        0.448834               0.000000
8       obbliga              2        0.435853               0.097829
9   venerabilem              2        0.434025               0.038788
10    pecorelle              4        0.437305               0.012831
11      agnello              1        0.430875               0.265102
12   melioribus              3        0.420954   

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      mandriano              5        0.515512               0.194176
1        pastori              5        0.552949               0.049423
2         gregge              5        0.529218               0.155488
3        porcaro              4        0.489052               0.321348
4        caprajo              5        0.488876               0.171522
5       pecorile              2        0.476012               0.318175
6          ovile              5        0.478402               0.748645
7          ameto              3        0.476869               0.603476
8       barcinio              5        0.473217               0.210403
9   scarpacifico              2        0.458537               0.231618
10          edra              3        0.488576               1.000000
11     summonzio              1        0.478150               0.189288
12        pastor              2        0.480404               0.185096
13    

### Altre parole correlate

In [12]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pastori", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        mercenarj              5        0.510501               0.378262
1       stupratori              5        0.466132               0.125932
2          gattivi              5        0.472977               0.456400
3       soggiaccia              3        0.469815               0.608688
4         rincalzi              5        0.503614               0.528766
5          moderni              5        0.477042               0.260754
6   amministratori              3        0.460993               0.424127
7          cassine              4        0.467215               1.000000
8          commuta              1        0.479341               0.000000
9      vendereccia              4        0.456516               0.579817
10      includendo              3        0.464645               0.035613
11      tralignano              2        0.445627               0.913504
12       s

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        pastore              5        0.552949               0.134288
1     pastorelle              5        0.513511               0.000000
2      mandriani              5        0.524297               0.263015
3          capre              1        0.496827               0.178860
4          ovile              4        0.470864               0.157595
5     pascolando              4        0.473214               0.130684
6       barbagia              1        0.477949               0.385547
7        nuoresi              1        0.474485               0.246177
8         gregge              4        0.481624               0.059495
9         caprai              1        0.473548               0.219098
10      bifolchi              3        0.476006               0.060823
11         baite              1        0.485635               0.686307
12         volàr              2        0.481084               0.140942
13    

### TOPOS: AMORE

In [13]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amore", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            amor              5        0.614733               0.000000
1      mercennaio              4        0.460568               0.071501
2         filiale              4        0.451461               0.082933
3          amanza              3        0.450561               0.007907
4         odiando              4        0.450703               0.001422
5        schiecto              1        0.442407               1.000000
6              am              3        0.448514               0.114483
7          corale              4        0.449553               0.077302
8         amatore              1        0.437130               0.101836
9         affetto              1        0.435714               0.145069
10         cresco              4        0.453945               0.087963
11         ardore              1        0.456209               0.065066
12  indissolubile      

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0              amor              5        0.643975               0.792248
1           affetto              5        0.598629               0.094397
2   svisceratissimo              5        0.450856               0.113094
3          figliale              3        0.450942               1.000000
4      sincerissimo              4        0.442366               0.131565
5          sprezzai              2        0.433450               0.059254
6            amaste              4        0.446310               0.115617
7           venerai              1        0.439374               0.000000
8       prospettare              1        0.437761               0.074669
9              odio              1        0.436474               0.070032
10          objetto              2        0.450880               0.139562
11         amistade              1        0.450391               0.115334
12     ardentissimo              2    

### Altre parole correlate

In [14]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amor", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         amore              5        0.614733               1.000000
1        corale              4        0.508602               0.063585
2       deletto              3        0.507161               0.005909
3        desïar              5        0.499645               0.035220
4       contrar              1        0.504884               0.203830
5        desiro              2        0.498447               0.109339
6      reguardo              4        0.494424               0.094837
7          vagi              1        0.496191               0.230773
8      sdegnate              4        0.487340               0.067559
9       usaggio              2        0.489661               0.099542
10          cor              3        0.491847               0.220329
11        coral              1        0.489846               0.157174
12       reposa              1        0.485589   

        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       amore              5        0.643975               0.184939
1        jesù              5        0.539281               0.070061
2       zonto              5        0.468116               1.000000
3     inspiri              5        0.473161               0.053543
4      desiro              2        0.461837               0.000771
5     serbami              3        0.458713               0.069166
6     affetto              5        0.474009               0.126567
7   incensare              1        0.452741               0.000000
8      amaste              1        0.450053               0.055526
9     serbavi              2        0.442913               0.082951
10    objetto              2        0.451237               0.180872
11     plachi              1        0.464384               0.497948
12     impera              2        0.447022               0.075737
13   sprezzai              2        0.441798    

In [15]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("innamorato", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
                Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        giovanissimo              5        0.487120               1.000000
1          giovanetta              3        0.454284               0.353612
2             disiare              2        0.460165               0.573776
3             aveduta              3        0.458443               0.770934
4          portatogli              4        0.450289               0.177016
5          avvenevole              1        0.472101               0.692830
6             cannila              1        0.466690               0.331567
7           giannotto              1        0.466072               0.598332
8           inanimito              1        0.464377               0.385594
9           divenissi              5        0.455640               0.232994
10         accompagno              1        0.487969               0.598631
11        tornatosene              3        0.4

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   perdutamente              5        0.503350               0.377558
1      innamorai              5        0.457754               0.288296
2        capirle              3        0.441512               0.094571
3        riamata              2        0.445735               0.305532
4    vagheggiava              2        0.453927               0.345144
5      affermerò              1        0.444573               0.051458
6        riamato              3        0.423866               0.070170
7     giovanotta              1        0.442620               0.138681
8   vagheggiando              1        0.439012               0.356876
9        avvertí              1        0.437771               0.517139
10   sdegnosetta              4        0.428316               0.040066
11  mellonaggine              1        0.447951               0.202291
12        nubile              2        0.431268               0.413335
13    

### TOPOS: CHIESA

In [16]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("chiesa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           santa              5        0.550585               0.199355
1       sismatici              5        0.521816               0.149919
2     scomunicato              5        0.490923               0.257398
3     confermarlo              5        0.482888               0.201200
4        ecclesia              4        0.481507               0.304691
5         barbera              4        0.472632               0.290507
6         fautore              2        0.457900               0.048039
7     persecutore              1        0.457559               0.000000
8        cappella              3        0.467819               0.473732
9   giurisdizioni              1        0.453222               0.044540
10        mistico              4        0.499346               0.152568
11       reparata              1        0.466619               0.257887
12     circonciso      

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    parrocchiale              5        0.587337               0.675454
1       anglicana              5        0.565809               0.193096
2       cattolica              5        0.559356               0.019277
3         araceli              4        0.533812               0.496522
4      cattedrale              4        0.528867               0.118552
5        cappella              5        0.553939               0.578176
6    orsanmichele              2        0.538886               0.011132
7   giuspatronato              2        0.528185               0.076371
8        basilica              5        0.522281               0.133791
9       chiesuola              1        0.526069               0.108273
10     parrocchia              4        0.546447               0.269587
11      chiesetta              4        0.521741               0.383575
12     pentecosta              2        0.521003               1

### Altre parole correlate

In [17]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("papa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    bonifazio              5        0.666368               0.000000
1    cardinali              5        0.640268               1.000000
2   innocenzio              5        0.616438               0.759638
3     pasquale              5        0.611331               0.323119
4      piuvico              3        0.584299               0.363029
5     clemento              5        0.607228               0.455360
6    innocenzo              5        0.610369               0.063960
7     clemente              5        0.597202               0.540177
8    celestino              3        0.592868               0.484286
9    scomunicò              5        0.590047               0.493152
10  concestoro              2        0.564727               0.244201
11    antipapa              1        0.557782               0.161694
12     piuvicò              1        0.575339               0.

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       pontefice              5        0.745835               0.039587
1      innocenzio              5        0.631704               0.056910
2       cardinale              5        0.632907               0.050051
3        antipapa              5        0.596536               0.042553
4         ormisda              4        0.593815               0.286659
5          papale              5        0.599524               0.046887
6       cardinali              5        0.594949               0.003703
7       scomunicò              1        0.600101               0.240442
8     vitelleschi              1        0.594568               0.279056
9       canonizzò              1        0.594263               0.039723
10           papi              4        0.599553               0.075785
11       clemente              1        0.588113               0.075968
12       vormazia              1        0.587527               0

In [18]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("prete", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     pasquino              5        0.512795               0.116068
1   frastaglia              5        0.531536               0.073360
2     pistello              5        0.499923               0.142219
3      fallera              5        0.474607               0.305874
4       juccio              5        0.483549               0.298165
5    officiava              5        0.479586               1.000000
6    giglietta              3        0.459775               0.115311
7    belcolore              2        0.459836               0.408399
8       tomasa              3        0.462252               0.359541
9    petruccio              2        0.469226               0.108164
10    orsuccio              2        0.459195               0.768959
11  bentivegna              2        0.456372               0.000000
12   codiponte              3        0.475232               0.

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     sacerdote              5        0.571589               0.541733
1      liprando              4        0.542555               0.795647
2       parroco              5        0.572863               0.000000
3    cappellano              5        0.561636               0.503269
4         frate              5        0.538122               0.378991
5       paskale              5        0.546650               0.457541
6      spretato              5        0.573169               0.532815
7   beneficiato              5        0.547307               0.994578
8        curato              4        0.545136               0.530102
9     officiava              1        0.546803               0.441446
10       pittui              4        0.541647               0.971996
11     chierico              2        0.516734               1.000000
###CADE_1915_1922_Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra_cade###
           Words

In [19]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("frate", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     ruffino              5        0.573654               0.259172
1      masseo              5        0.594907               0.215279
2      zelone              5        0.566322               0.149340
3     balasta              5        0.510972               0.425301
4   venturino              5        0.506659               0.070852
5      egidio              5        0.529794               0.026938
6     bonzeca              5        0.505999               0.119064
7    dìssegli              5        0.498528               1.000000
8   francesco              2        0.484660               0.066795
9      gomita              2        0.495098               0.000000
10     papino              3        0.489176               0.812416
11   riccieri              2        0.481608               0.221103
12       elia              1        0.489635               0.747708
###CADE

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       francescano              5        0.510542               0.512047
1       agostiniano              4        0.487856               0.153834
2        domenicano              5        0.491617               0.000000
3          ieronimo              5        0.495909               0.164566
4            fojano              4        0.469271               0.359557
5          jeronimo              5        0.470260               0.592948
6            benoni              2        0.471701               0.053437
7            foiano              4        0.466534               0.112297
8        cappuccino              5        0.480767               1.000000
9   angiolgabriello              2        0.471202               0.495193
10            frati              4        0.487072               0.685258
11       savonarola              4        0.480651               0.133573
12        beatifico              1    

In [20]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("dio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             idio              5        0.518468               0.114069
1            iddio              5        0.508126               0.161318
2         creatore              5        0.441636               0.365499
3      onnipotente              5        0.448466               0.186752
4        immediate              4        0.422330               0.123571
5          sciente              3        0.415966               0.156289
6          maiestá              1        0.420300               0.116096
7     communicasse              3        0.411190               0.914591
8           divina              2        0.416302               1.000000
9         providde              1        0.415684               0.206069
10         deitade              3        0.419991               0.109676
11      incarnarsi              3        0.410658               0.189302
12       t

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            iddio              5        0.797737               0.037003
1   misericordioso              5        0.567798               0.047892
2            allàh              5        0.546361               0.163570
3             idio              5        0.574408               0.065470
4      onnisciente              5        0.519154               0.106788
5         creatore              5        0.529296               0.000000
6        esaudisca              4        0.517062               0.033553
7         rimuneri              2        0.525214               0.086023
8         adoriamo              2        0.508323               0.066080
9          punisci              1        0.512879               0.048064
10        esaudirà              3        0.523055               1.000000
11      domeneddio              1        0.507286               0.256893
12           cielo              2        0.516878  

In [21]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("cristo", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            gesù              5        0.673622               0.133532
1            iesù              5        0.543421               0.106427
2       crocifixo              4        0.507423               0.052010
3    resurrezione              5        0.501247               0.169713
4       salvatore              3        0.508832               0.116125
5       redentore              2        0.490855               0.089412
6   resurressione              3        0.494969               1.000000
7            iesu              5        0.507929               0.185236
8       discepoli              3        0.491349               0.172243
9            iesú              3        0.500590               0.349552
10     beatissima              3        0.500180               0.135576
11      crucifixo              2        0.496696               0.000000
12           jesù      

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            gesù              5        0.685541               0.179962
1        apostoli              4        0.476876               0.000000
2         galilea              5        0.474841               0.072075
3       redentore              5        0.495661               0.061514
4      crocifisso              5        0.458125               0.106272
5         fabiola              1        0.463586               0.025687
6        coeterno              5        0.449423               0.094631
7      implorarne              2        0.442183               0.127734
8       nazzareno              5        0.459166               0.171966
9         nazaret              5        0.453912               1.000000
10      discepoli              4        0.455163               0.043620
11        betania              1        0.431083               0.089716
12  crocifissione              1        0.446736               0

### TOPOS: FELICITA'

In [22]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("felicità", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      sollevarle              3        0.524966               0.438508
1           porle              2        0.481632               0.759350
2    propagazione              3        0.498705               0.000000
3    desiderabile              4        0.467257               0.147241
4     amplitudine              1        0.501831               0.431880
5      conseguita              5        0.470348               0.569972
6        prospera              2        0.476966               0.717248
7    costituivano              1        0.497783               0.399729
8    constituendo              3        0.483624               0.094707
9      pacificata              4        0.474523               0.362955
10   procureranno              4        0.481392               0.344614
11    repressione              2        0.471195               0.415226
12      affluente      

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          duratura              5        0.496413               0.000000
1        prosperità              5        0.500882               0.156429
2        infelicità              5        0.506483               0.564499
3        diuturnità              5        0.462331               0.273604
4          gustasse              2        0.469167               0.223322
5   contraccambiata              4        0.458415               0.219141
6          sperarla              5        0.485173               0.282758
7         benessere              3        0.479674               0.836420
8            nòstra              1        0.476304               0.419825
9       fabbricarmi              4        0.470769               0.393805
10       conducenti              1        0.461768               0.106315
11        perfezion              1        0.452784               1.000000
12      contentezza              3    

### Altre parole correlate

In [23]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("gioia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       pesanza              5        0.513524               0.885563
1          gioi              5        0.530047               0.385629
2    allegranza              5        0.521949               0.387493
3           pen              3        0.490405               0.259078
4       gioiosa              5        0.492212               0.558680
5       dolzore              5        0.498819               1.000000
6         penar              3        0.487202               0.465308
7        gioiva              3        0.488768               0.416641
8     beninanza              3        0.486705               0.141184
9          noia              5        0.500267               0.579139
10  dimostranza              1        0.477619               0.439772
11       avenir              1        0.486574               0.394505
12      ingemmi              2        0.469276   

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           gioja              5        0.701060               0.468001
1     contentezza              5        0.668147               0.241678
2      allegrezza              5        0.605286               0.817174
3         giubilo              5        0.583425               0.000000
4    consolazione              5        0.564541               0.057042
5         letizia              5        0.573658               0.081318
6   soddisfazione              4        0.530149               0.095649
7        felicità              5        0.538252               1.000000
8       esultanza              5        0.535672               0.346570
9        ebbrezza              5        0.531302               0.292317
10         dolore              1        0.521033               0.050640
###CADE_1915_1922_Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra_cade###
            Words  Abs.frequency  Med.similarity  Med.similari

### TOPOS: PATRIA/ITALIA

In [24]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("patria", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       mantuani              5        0.480431               0.006159
1     tradiscono              4        0.444384               0.305556
2          regol              4        0.423117               0.450017
3       antenora              4        0.423837               0.335650
4          natio              5        0.422913               0.193158
5        exilium              4        0.423208               0.328261
6        ambidui              3        0.433580               0.270873
7       servirne              1        0.421675               0.174559
8        plebeio              1        0.421514               1.000000
9      eversione              3        0.411699               0.324799
10         natìo              4        0.438139               0.503535
11    otterranno              1        0.406028               0.356529
12     stipendii              3     

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         isolando              3        0.441966               0.249431
1         adottiva              2        0.442260               0.934690
2    permettendoci              4        0.427323               0.815418
3    desidereranno              1        0.444300               0.329965
4     zelantissimo              1        0.439822               0.768449
5       affrancare              1        0.438534               0.504783
6     rivendicarsi              1        0.438501               0.759623
7         giuriamo              2        0.427526               1.000000
8      fidatissima              2        0.426662               0.587742
9        coriolano              1        0.428830               0.677569
10         teutono              4        0.437423               0.284963
11  disavventurata              4        0.434227               0.435395
12      repubblica              2        0.428963  

In [25]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("italia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          toscana              5        0.470192               0.178976
1           gallia              5        0.495731               0.472881
2          vandali              5        0.511485               0.286333
3            gotti              5        0.482287               0.126114
4         alamagna              4        0.475751               0.191690
5           totila              2        0.462681               0.767611
6         pannonia              4        0.461572               0.000000
7          narsete              1        0.470958               0.058287
8          numidia              2        0.456989               0.206365
9          affrica              2        0.455826               0.569303
10         arnolfo              1        0.463104               0.117750
11         ungaria              1        0.462795               0.130875
12        

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       francia              5        0.664511               0.000000
1      germania              5        0.645524               0.852605
2        europa              5        0.641696               1.000000
3     lombardia              5        0.575929               0.877335
4       sicilia              5        0.545695               0.031471
5   inghilterra              5        0.536479               0.172853
6          roma              5        0.543149               0.086537
7       toscana              5        0.533213               0.427720
8   napoleonidi              2        0.516392               0.043081
9        grecia              1        0.514595               0.000018
10     penisola              1        0.515134               0.034953
11     italiani              3        0.523119               0.215944
12  nazionalitá              1        0.509727               0.232389
13  civilissima     

### Altre parole correlate

In [26]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("stato", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            istato              5        0.433389               0.734574
1         reggitore              2        0.413400               0.148115
2   excellentissimo              2        0.408109               0.469475
3         reformato              5        0.411385               0.276054
4          abassato              1        0.420647               0.216836
5           ribelle              1        0.417759               0.473409
6          pacifico              1        0.415876               0.350042
7            fotino              3        0.402187               0.468667
8        conseguito              3        0.407920               0.418829
9           istento              3        0.406955               1.000000
10           vivuto              4        0.415603               0.449373
11          gattivo              2        0.407882               0.6351

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        trasmesso              5        0.423655               0.368684
1       consultore              4        0.407753               0.411660
2      stipendiato              5        0.404355               0.619445
3        approvato              3        0.406766               0.074911
4     cospirazioni              1        0.402104               0.239647
5         inserito              3        0.407916               0.345048
6         maritato              3        0.408643               0.362106
7   contrassegnato              1        0.394949               0.446042
8        preseduto              5        0.406322               0.121594
9       comandiamo              3        0.405344               0.000000
10      approviamo              2        0.412731               0.249241
11        senatore              1        0.407488               0.639864
12      ammettersi              1        0.405533  

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         avviliva              1        0.404426               0.426622
1      compositore              3        0.387314               0.025519
2       pontificio              3        0.385417               0.723752
3          fungere              2        0.383971               1.000000
4         dissidio              3        0.377675               0.674324
5      assicurando              1        0.391602               0.373323
6       trasferito              4        0.375293               0.000000
7         borodino              1        0.389307               0.216215
8           tipico              1        0.386715               0.631380
9         pareggio              2        0.379406               0.008355
10    storicamente              1        0.377219               0.638233
11       accoppato              2        0.377808               0.232215
12        comparso              1        0.368040  

In [27]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("impero", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      monarca              5        0.540496               0.441375
1       pagina              4        0.542777               0.615940
2       amator              4        0.537826               0.673944
3      primero              2        0.543205               0.000000
4     assumpto              4        0.542674               0.208701
5        raina              1        0.569922               0.389678
6      virente              4        0.536995               0.031868
7       glesia              2        0.538187               0.386241
8        gener              2        0.537456               0.404779
9     mantegna              2        0.521224               0.514136
10       summo              1        0.542570               0.097242
11  defendendo              1        0.529319               0.611508
12   pontifice              1        0.524723               0.

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          imperio              5        0.548264               0.155638
1            incas              5        0.475870               0.064539
2         bizanzio              5        0.448845               0.232097
3       ristorator              3        0.440163               0.137036
4           scerif              2        0.456644               0.160676
5             doll              4        0.436657               0.156102
6          decader              2        0.439006               0.137962
7        chiamanlo              3        0.436537               0.089326
8   ambiziosissima              2        0.438946               0.169184
9             daci              1        0.439598               0.000000
10        lecapeno              3        0.434508               0.090142
11         rivisse              3        0.436595               1.000000
12         dominio              2        0.430360  

In [28]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("nazione", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          nazion              5        0.545657               0.643779
1        progenia              4        0.464995               0.126878
2      bassissima              1        0.499890               0.135246
3        prosapia              4        0.447110               0.074492
4      laomedonte              1        0.484428               0.241412
5       rinnegato              3        0.456316               0.136740
6          smirne              1        0.479646               0.467925
7     popolaresca              2        0.462307               0.142449
8    pessimamente              2        0.459048               0.233729
9   originalmente              1        0.473632               0.523426
10      distratti              1        0.477297               0.064246
11         maculò              1        0.454337               0.081103
12      vilissima      

               Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     conquistatrice              5        0.500373               0.168436
1             nazion              5        0.532319               0.362713
2      individualità              1        0.498766               0.136045
3           autonoma              5        0.480173               0.132917
4          riseggono              2        0.473929               0.075310
5       progenitrice              2        0.480393               0.609583
6              slava              3        0.473605               0.097189
7         rigenerata              1        0.476042               0.259841
8        guarentisce              3        0.459570               0.000000
9       innanzitutto              2        0.464859               0.110927
10           assorba              2        0.464276               0.589946
11        diseredata              3        0.458446               0.175557
12         avvalersi     

In [29]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("bandiera", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         pennone              5        0.617036               0.635557
1     travagliati              5        0.564571               0.067075
2       stendardo              3        0.554283               0.424336
3        arcadori              1        0.580508               0.012528
4       rastrello              5        0.554137               0.089540
5     pennoncello              1        0.571569               0.502201
6        ischiera              3        0.546645               0.282966
7       bruschino              2        0.559338               0.252198
8     conastabole              1        0.567142               0.034720
9     conastaboli              3        0.545169               0.000000
10    contradiare              2        0.542085               0.151613
11         pavese              2        0.534553               0.076787
12       feditori      

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    tricolore              5        0.570830               0.164750
1    stendardo              5        0.557588               0.198186
2        côrsa              5        0.536645               0.618952
3   inalberata              5        0.543045               0.028555
4     sventoli              5        0.549478               0.690295
5     vessillo              5        0.559429               0.229470
6     inalberò              5        0.501247               0.224388
7   inalberato              3        0.501703               0.852397
8     sventolò              4        0.497626               0.761999
9     coccarda              2        0.498896               0.184027
10    svestite              1        0.481116               0.000000
11   impalmava              3        0.483678               1.000000
12    bandiere              2        0.495896               0.872922
###CADE_1861_1914_L'Italia_liberal

### TOPOS: GUERRA

In [30]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("guerra", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         triegua              5        0.458666               0.255821
1     dissensione              4        0.419543               0.719720
2     conquistata              2        0.416130               0.168908
3       ribellare              2        0.407540               0.414592
4       battaglia              4        0.437812               0.057635
5          riotta              1        0.416875               0.428302
6        sperante              1        0.416808               0.366359
7          tregua              2        0.407386               0.261967
8        canavese              1        0.415110               0.245077
9       guerriata              5        0.449943               0.474175
10      battaglie              1        0.396830               0.232430
11    rinfrescata              2        0.403778               0.233353
12  assicurandosi      

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    guerreggiata              5        0.584645               0.127217
1          guerre              5        0.546971               0.461785
2      secessione              5        0.557689               0.073288
3           irani              5        0.493753               0.690643
4       battaglia              5        0.519100               0.126924
5     mitridatica              5        0.486474               1.000000
6        reclutar              3        0.474755               0.125291
7          ircano              1        0.475468               0.253657
8           lotta              4        0.463103               0.103515
9          punica              1        0.464811               0.001979
10       mossagli              3        0.478428               0.193884
11  orribilissima              1        0.466708               0.127817
12   insurrezione              2        0.478120               0

### TOPOS: FIGURA FEMMINILE

In [31]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("donna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      madonna              5        0.512177               0.048509
1      lorenza              5        0.492011               0.040443
2   margharita              5        0.513814               0.545508
3      filippa              5        0.484937               0.230531
4        monna              5        0.478241               0.101935
5         tora              2        0.460877               0.534860
6   giovanetta              1        0.475250               0.000000
7      nonnina              4        0.463528               0.378810
8     ambrogia              5        0.485598               0.183825
9     catalina              2        0.466044               0.648806
10     giuliva              1        0.463145               0.973640
11     abbiala              1        0.454744               0.024381
12   ermellina              2        0.458061               1.

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       fanciulla              5        0.600104               0.143632
1         signora              5        0.595530               0.000000
2         ragazza              5        0.598614               0.875077
3        creatura              5        0.589535               0.194353
4        guantaja              2        0.555312               0.370068
5     fanciullona              3        0.529390               1.000000
6    educatissima              4        0.546809               0.164180
7      superbiosa              2        0.532450               0.470576
8       stupidina              1        0.545518               0.301910
9     innocentina              1        0.541043               0.393190
10         moglie              4        0.564524               0.453521
11         marito              4        0.553869               0.579455
12          madre              4        0.548765               0

In [32]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("madre", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           padre              5        0.495556               0.187721
1      condussela              1        0.443102               0.075453
2         tutrice              5        0.441787               0.161450
3          schiro              3        0.435588               0.359717
4           iosef              4        0.433908               0.245746
5         ottavia              1        0.434110               0.401384
6           telda              1        0.432841               0.363429
7           dauno              1        0.429966               0.251468
8       genitrice              4        0.432988               0.158869
9         astiage              1        0.424380               0.237790
10     franciesca              1        0.419533               0.132410
11          brida              1        0.414542               0.285961
12       ortolana      

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   dilettissima              5        0.549618               0.614473
1        sorella              5        0.557244               0.185382
2        angiola              4        0.540398               0.000000
3           seth              5        0.535185               0.582789
4      genitrice              4        0.532799               0.627826
5   adoratissima              5        0.541677               0.593353
6      avvisarle              1        0.542216               0.035750
7     bamboletta              1        0.540824               0.442779
8     creaturina              4        0.534744               0.811150
9    imperadrice              1        0.536092               0.340902
10         padre              4        0.555817               0.210326
11       eponina              3        0.530631               0.246459
12       ersilia              1        0.525358               0.559155
13    

In [33]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("femmina", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     cangiandosi              5        0.501106               0.667759
1         maschio              5        0.500318               0.329772
2     sbardellata              1        0.472516               1.000000
3       avolterio              1        0.470474               0.552115
4          glauca              5        0.447026               0.086974
5     ermafrodito              3        0.456371               0.076629
6       concubito              2        0.459812               0.514801
7              iu              4        0.435624               0.664556
8        ingenera              3        0.430442               0.413443
9           morie              2        0.435596               0.267843
10      incitando              3        0.433602               0.102587
11  corrottissima              2        0.415972               0.233574
12     servigiale      

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       maschio              5        0.657750               0.065776
1        femina              5        0.522737               0.283347
2         balba              5        0.492085               0.080707
3       femmine              5        0.485439               0.191421
4        maschi              5        0.480491               0.046592
5         donna              5        0.470835               0.415945
6       blandir              5        0.445242               0.254595
7       gheppio              1        0.443113               0.071497
8        sperma              1        0.441825               0.233192
9     androgino              1        0.441791               1.000000
10  ermafrodito              3        0.437356               0.015611
11        sesso              3        0.434746               0.059269
12  soprapparto              1        0.426586               0.049987
13     procrear     

In [34]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("sposa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           sposo              5        0.538342               0.239467
1     avventurata              5        0.434361               0.238891
2           sposò              5        0.448326               0.162375
3         sorella              4        0.427097               0.207898
4      gostantina              3        0.422447               0.277759
5           cugin              1        0.435807               0.155295
6          isposa              1        0.434403               0.287797
7       prendermi              1        0.427844               0.181203
8        lebbrosa              3        0.415028               0.195000
9          immota              2        0.424349               0.208216
10        toltole              4        0.439966               0.278048
11   pietosissimo              1        0.427008               0.209278
12  abbracciarono      

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         sposo              5        0.781991               0.021313
1        moglie              5        0.581314               0.054227
2        figlia              5        0.568762               0.011234
3         nozze              5        0.564494               0.088570
4     fidanzata              5        0.550511               0.160431
5         sûryâ              5        0.536356               0.003198
6      bazvalan              4        0.533429               0.209855
7   destinatole              5        0.541789               0.143628
8         sposi              5        0.528016               0.025814
9    divorziata              2        0.530869               0.021787
10      nuziale              1        0.534572               0.000000
11       peroli              1        0.533549               1.000000
12       marito              2        0.528828               0.032837
###CADE_1915_1922_It

### TOPOS: NOTTE/SERA

In [35]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("notte", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     mercoledì              5        0.448532               0.093234
1          sera              5        0.485577               0.091775
2         nocte              5        0.446617               0.146946
3       traente              1        0.434484               0.171602
4        siando              2        0.433100               0.094211
5    mezzanotte              5        0.439034               0.286244
6   discontenti              1        0.425735               0.025709
7    riposarono              5        0.431925               0.471921
8        giorno              5        0.435507               0.336529
9       venente              1        0.422293               0.444620
10        scurò              2        0.421429               0.235178
11     vegnente              2        0.438523               0.132913
12      mattina              1        0.417230   

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          sera              5        0.670231               0.118715
1       nottata              5        0.594719               0.309285
2      giornata              5        0.539599               0.246458
3       insonne              5        0.527717               0.240001
4        dormii              5        0.503558               0.089221
5     vegliando              3        0.491908               0.090811
6       mattina              5        0.526496               0.600731
7     mattinata              4        0.497980               1.000000
8        serata              2        0.490754               0.051017
9    mezzanotte              5        0.498280               0.000000
10     stanotte              4        0.481105               0.065605
11      vegliai              1        0.495460               0.075897
12  svegliatomi              1        0.493513               0.057118
###CADE_1915_1922_It

In [36]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("sera", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      mattina              5        0.567228               0.285439
1   mezzanotte              5        0.505374               0.056078
2       cenato              5        0.473879               0.160801
3      sabbato              2        0.476809               0.065120
4     maltempo              3        0.481993               0.032698
5   arrivorono              3        0.479636               0.085306
6         cenò              3        0.474179               0.238476
7       vespro              3        0.473110               0.240925
8     avviammo              1        0.464357               0.056970
9        notte              5        0.485577               0.193129
10     caminar              1        0.452629               0.000000
11      ascoli              1        0.451520               0.304666
12     cenaron              4        0.455103               0.

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      mattina              5        0.750268               0.234880
1        notte              5        0.670231               0.080161
2     domenica              5        0.575736               0.174878
3       serata              5        0.565740               0.021779
4      mattino              5        0.585846               1.000000
5       giorno              5        0.562049               0.000000
6       sabato              2        0.529127               0.279200
7         sere              3        0.533078               0.649132
8   tardissimo              3        0.532227               0.074717
9    imbrunire              3        0.527125               0.030554
10  pomeriggio              4        0.539201               0.191059
11    giornata              4        0.540443               0.047873
12   mattinata              1        0.522157               0.044142
###CADE_1915_1922_Italia_nella_Pri

### Altre parole correlate:

In [37]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("tenebre", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      tenebrosa              5        0.494489               0.696311
1       ciechitá              5        0.497242               0.774870
2         vincia              5        0.514967               0.197832
3         nuvila              5        0.497369               1.000000
4    dissolvesse              5        0.515409               0.830323
5     ministrano              4        0.497631               0.336742
6     ignoranzia              1        0.505259               0.702807
7      offuscati              1        0.504674               0.491668
8       notturne              4        0.485000               0.698440
9     cognobbero              2        0.501941               0.820547
10       lordava              4        0.475732               0.693116
11  cognoscevano              4        0.467422               0.650292
12    pervertono              3     

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        tenebra              5        0.631466               0.353632
1       oscurità              5        0.589479               0.444582
2      tenebrore              5        0.526795               0.052875
3          ombre              5        0.537256               0.186939
4      addensate              5        0.535067               0.047737
5           buio              5        0.548631               0.368310
6         vincia              3        0.509608               0.718472
7       caligini              4        0.510207               0.047045
8      tenebroso              1        0.506125               0.103081
9   rischiarasse              1        0.497700               0.000000
10      penombra              2        0.489343               1.000000
11      caligine              1        0.490212               0.298166
12      nebbione              3        0.489446               0.159571
13   s

In [38]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("ombra", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        tenèbra              3        0.426983               0.365582
1      tenebrata              3        0.413474               0.000000
2          ficta              4        0.404789               0.292167
3          opaco              5        0.422251               0.169342
4      disgombra              4        0.401159               0.472804
5       oscurata              1        0.422872               0.463686
6           mòre              3        0.405620               0.484772
7     ricuperava              1        0.412181               0.539071
8           dime              2        0.401598               0.199229
9       rifratto              4        0.404901               0.118355
10      facienti              3        0.402175               0.556329
11      violetta              1        0.402562               0.020307
12         ammon              1     

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        ettorre              5        0.435491               0.308094
1        nascoso              4        0.373440               0.115035
2        evocata              4        0.377722               0.281112
3       sognassi              4        0.378657               0.277771
4       celatevi              4        0.373387               0.425529
5        affabil              4        0.375994               0.212246
6      apparirvi              3        0.373674               0.061569
7     collicelli              2        0.367752               1.000000
8        melensa              1        0.370367               0.328189
9          fauna              1        0.366529               0.198022
10         ombre              4        0.398809               0.145477
11     ingombrar              1        0.371334               0.000000
12    scioperate              1        0.362435               0.253900
13    

### TOPOS: FORTUNA

In [39]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("fortuna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           fati              2        0.446530               0.439018
1       prospera              5        0.457164               0.171905
2        avversa              4        0.425198               0.000000
3    casualmente              5        0.422332               0.388719
4     pacificata              4        0.427597               0.635857
5     volubilità              5        0.444250               0.813393
6        ventura              4        0.418473               0.481118
7      avversità              2        0.409978               1.000000
8   sostenitrice              1        0.400592               0.305604
9        destino              5        0.436414               0.213885
10      rabbuffa              4        0.417596               0.168528
11      volubile              2        0.422438               0.007188
12    rimettiamo              1     

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         sorte              5        0.552419               0.097209
1       ventura              5        0.523697               0.367242
2         juvat              5        0.474835               0.033036
3       destino              5        0.449805               0.138320
4     arridesse              5        0.471908               0.134500
5        arriso              3        0.442780               0.193530
6        arrida              4        0.437549               0.175098
7       fortune              5        0.451237               0.204784
8   malaventura              2        0.437139               0.237503
9        arrise              1        0.429384               0.390250
10      furtuna              4        0.455400               0.093981
11    disgrazia              2        0.447460               0.000000
12      audaces              3        0.442294               1.000000
13     prospera     

### TOPOS: NOIA

In [40]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("noia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            gioia              5        0.500267               0.197013
1          martìre              5        0.465485               0.387292
2         oblianza              3        0.446652               0.147858
3          discara              1        0.470108               0.590098
4              pen              4        0.453997               0.089715
5         tardanza              1        0.462423               0.028443
6          gravosa              3        0.442181               1.000000
7          pesanza              5        0.436961               0.248332
8            solei              1        0.455817               0.224491
9       amarissima              1        0.455186               0.210530
10      lagrimetta              3        0.453325               0.361994
11         mendica              1        0.455299               0.053167
12     dis

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          fastidio              5        0.488565               0.510074
1             tedio              5        0.505172               0.162085
2      scontentezza              5        0.488918               0.400516
3       accrescermi              5        0.465661               0.000000
4              noja              5        0.480823               0.794074
5        malinconia              4        0.458636               0.387928
6      svogliatezza              4        0.448753               0.103229
7       inquietezza              1        0.461188               0.231139
8         febbretta              1        0.455810               0.454615
9       sospettassi              1        0.451476               0.422971
10         disgusto              4        0.456825               0.128717
11   indicibilmente              2        0.443165               0.698085
12       cascaggine              1    

### TOPOS: TEMPORALE/PIOGGIA

In [41]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("temporale", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       spirituale              5        0.582713               0.222751
1        meschiano              5        0.577143               0.000000
2       substanzia              5        0.530045               0.194709
3    distribuivano              4        0.546329               0.642883
4         monarchi              5        0.531197               0.361289
5      giudiziaria              2        0.516534               0.350448
6       privandolo              5        0.519931               0.404630
7      umiliandosi              1        0.537712               0.945324
8    remunerazione              1        0.537450               0.494366
9       prelazione              4        0.520717               0.391510
10      successiva              1        0.519288               0.099280
11  ecclesiastiche              1        0.517638               0.383793
12       s

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      spirituale              5        0.545478               0.037367
1       usurparsi              5        0.456498               0.099322
2      episcopale              1        0.467168               0.394251
3       temporali              5        0.473444               0.361301
4     raccorciava              4        0.439376               0.054932
5      immutevole              5        0.462908               0.136200
6         simonia              1        0.442350               0.145005
7         staglio              2        0.442781               0.261867
8        mutevole              1        0.434946               0.109954
9          ostava              1        0.430414               0.098023
10       violarle              1        0.436320               0.158178
11        opporlo              3        0.450053               0.000000
12        potestà              2        0.436649               0

In [42]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pioggia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         adona              5        0.558740               1.000000
1         urlar              5        0.543819               0.157224
2        grando              5        0.539047               0.330684
3      grandine              5        0.513706               0.212392
4        quattr              3        0.503514               0.440715
5     atterrate              5        0.504847               0.755435
6          neva              5        0.489984               0.146109
7      annaffia              5        0.490880               0.720116
8         piova              5        0.536204               0.314130
9        fiacco              4        0.495322               0.000000
10  trapassammo              3        0.478220               0.814698
###CADE_1454_1559_Rinascimento_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     grand

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        dirotta              5        0.613311               0.176364
1   temporalesca              5        0.528563               0.202954
2    torrenziale              5        0.523291               0.353815
3     decresceva              5        0.512431               0.124889
4     scrosciava              5        0.513873               1.000000
5       nevicare              4        0.489756               0.000000
6   acquerugiola              5        0.484728               0.208567
7       sferzava              2        0.493761               0.260990
8    pioggerella              5        0.497940               0.186898
9      ammollare              4        0.488879               0.007895
10       piovere              1        0.486373               0.363050
11      nebbione              1        0.457195               0.234226
12     diluviale              2        0.476692               0.457145
13    

### TOPOS: GIOVINEZZA

In [43]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("giovinezza", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         florida              5        0.525039               0.361578
1         belezza              5        0.506186               0.278560
2        snervare              1        0.544330               0.412276
3        speciosa              3        0.510024               0.243296
4             etá              3        0.503120               0.779399
5      gioventute              5        0.494353               0.591678
6   bamboleggiare              1        0.523722               0.085587
7        provetta              1        0.522677               0.338746
8       rinovella              1        0.520766               0.000000
9         venustà              4        0.482537               0.558859
10         onestá              4        0.489798               0.793496
11    aggrandisce              1        0.489427               0.301272
12    adolescenza      

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       sfiorata              5        0.557758               0.432552
1       gioventù              5        0.565229               0.224585
2     giovanezza              5        0.505447               0.131154
3   stanchissima              5        0.472908               0.291841
4       virilità              5        0.472732               0.149581
5   fanciullezza              4        0.462316               0.247321
6      giovanile              2        0.478410               0.331947
7   freschissimo              3        0.468145               0.588628
8     appalesava              1        0.482765               0.402586
9     inebbriata              1        0.481734               0.351320
10    freschezza              4        0.455847               1.000000
11       appassì              1        0.452543               0.425460
12        salivi              1        0.447772               0.258762
13    

In [44]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("giovane", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
                Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             clonico              5        0.480011               0.212740
1            riteneva              1        0.480174               0.385934
2           giannetta              2        0.453078               0.326720
3   ardentissimamente              4        0.444919               0.215693
4           giuffredi              1        0.469072               0.997167
5         figlioletta              4        0.452327               0.277321
6          gentilesca              4        0.448229               0.332911
7        giovanissimo              1        0.467395               0.102805
8         intimissimo              1        0.464416               0.340765
9            minutolo              2        0.452865               0.514965
10          martuccio              4        0.450881               0.443580
11         partigiano              2        0.4

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        giovine              5        0.658780               0.168430
1      fanciulla              5        0.489373               0.160948
2    giovincello              5        0.471733               0.065344
3        aitante              5        0.467977               0.079779
4      avvenente              3        0.471960               0.159456
5        meldola              1        0.475216               0.178464
6   perdutamente              3        0.468810               0.070466
7     giovinetto              5        0.467348               0.255033
8    accertatosi              1        0.469679               0.129982
9     giovinetta              1        0.466923               0.177463
10     attempato              2        0.467638               1.000000
11      narrasse              3        0.462223               0.156311
12        nubile              4        0.452820               0.158179
13    

### TOPOS: BACIO

In [45]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("bacio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        bascio              5        0.577209               0.422301
1       voltasi              5        0.569917               0.310768
2   tovagliuolo              1        0.601986               0.778140
3           lel              1        0.600838               0.094436
4         glene              3        0.541815               0.698820
5   comandatemi              2        0.568351               0.902524
6       compero              1        0.591100               0.805902
7     narratoli              4        0.552776               0.601075
8      dichiate              1        0.580309               0.582929
9            pa              1        0.576926               0.169888
10     partiste              4        0.537016               0.461216
11       diteli              1        0.557435               0.989723
12      diviato              1        0.555408   

               Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             saluto              5        0.543323               0.173670
1         baciandole              5        0.537969               0.467735
2   tenerissimamente              5        0.524784               0.461778
3        chiedendole              5        0.518679               0.381968
4          scoccarle              5        0.511628               0.927536
5       abbracciollo              3        0.503557               0.143735
6          riverisco              1        0.507360               0.208012
7            baciate              1        0.506879               0.506736
8           amplesso              4        0.497661               0.000000
9         battaglini              3        0.485862               0.222274
10         benedirmi              3        0.488022               0.251505
11         abbraccio              2        0.503997               0.666452
12           baciare     

### TOPOS: LUNA

In [46]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("luna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     quintadecima              5        0.547879               0.332241
1          eclissi              5        0.538008               0.146830
2   interposizione              5        0.520177               0.741610
3            scurò              5        0.513941               0.493395
4         tramonta              5        0.519495               0.176607
5           oscurò              3        0.501845               0.147097
6           solare              2        0.505464               0.238193
7       sagittario              5        0.505711               0.879021
8           ariete              3        0.508601               0.292008
9         numerano              2        0.495301               0.406574
10            sole              4        0.556569               0.000000
11          stelle              1        0.488427               0.288572
12        

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         lunare              5        0.622528               0.031190
1     plenilunio              5        0.571492               0.090286
2        falcata              5        0.571993               0.051602
3         stelle              5        0.528283               0.227613
4   quintadecima              5        0.530998               0.631942
5           râkâ              5        0.530775               0.000000
6       chiarore              5        0.516478               1.000000
7           sole              5        0.494512               0.039092
8        chiaror              2        0.492312               0.628291
9        anumati              3        0.489509               0.333361
10       calante              1        0.497623               0.331743
11     novilunio              2        0.490594               0.067628
12        inalba              2        0.485227               0.313591
###CAD

### TOPOS: SILENZIO

In [47]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("silenzio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         squilli              5        0.469946               0.649435
1      lacrimabil              5        0.451140               0.000000
2       trasmutar              5        0.448784               0.413777
3      congiurava              3        0.415564               0.349942
4      cantassero              2        0.420904               0.268985
5           fioco              2        0.440949               0.666348
6       dicesseno              1        0.442034               0.318352
7       bisticcio              1        0.440619               0.653386
8         rondini              1        0.440303               0.254996
9       sonassero              1        0.437097               0.013625
10         fiocco              4        0.412921               0.088034
11          poser              3        0.400657               0.775203
12          cucco      

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   raccoglimento              5        0.526820               0.097405
1      silenziosa              5        0.485264               0.113903
2      solitudine              5        0.504964               0.000000
3      silenzioso              5        0.472203               0.091985
4           regnò              5        0.474842               0.047000
5         regnava              5        0.482505               0.033610
6    ineccitabile              3        0.456375               1.000000
7      sentendoci              2        0.464980               0.428791
8        susseguì              5        0.479579               0.330018
9           calma              3        0.456796               0.202902
10          pausa              1        0.451846               0.464352
11     susseguito              2        0.454491               0.296001
12     camminammo              2        0.458076               0

### TOPOS: AMICIZIA

In [48]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amicizia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     benivolenza              5        0.587327               0.224723
1     familiarità              5        0.562716               0.295295
2    constituisce              5        0.535674               0.174132
3      inimicizia              5        0.516807               1.000000
4         vinculo              4        0.521849               0.367321
5         esserti              1        0.516266               0.592012
6         vincoli              2        0.498548               0.094027
7   conciliatrice              5        0.507227               0.438717
8     acquistarti              3        0.489173               0.805604
9         assiduo              1        0.498108               0.442986
10    principiata              4        0.501030               0.000000
11        società              3        0.491261               0.262190
12      attissimi      

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         simpatia              5        0.556999               0.177364
1            stima              5        0.534134               0.597213
2        affezione              5        0.529166               0.069727
3       cordialità              5        0.512864               0.015161
4        parentela              5        0.516077               0.083797
5     riconoscenza              3        0.503815               0.000000
6       confidenza              4        0.493689               0.361292
7          affetto              5        0.500154               0.247588
8         amistate              1        0.492716               0.020931
9         cordiale              1        0.487898               0.360362
10        intimità              3        0.512799               0.454779
11  intrinsichezza              1        0.519324               0.407789
12         sincera              3        0.500872  

In [49]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amico", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           intimo              3        0.434918               0.000000
1          parente              5        0.458307               0.237515
2            vetar              2        0.429633               0.593648
3         indovino              1        0.443433               0.127497
4          discaro              1        0.443234               0.423217
5   singularissimo              3        0.437814               0.688153
6         servirne              1        0.436931               0.421611
7       confidente              4        0.430429               0.296489
8          dottare              3        0.430351               0.478887
9      intimissimo              2        0.431666               0.452651
10        portaste              2        0.448530               0.692259
11       carissimo              4        0.438667               0.448809
12       r

               Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       continuatemi              5        0.604043               1.000000
1        divotissimo              4        0.574775               0.188100
2   affezionatissimo              5        0.611974               0.873515
3      preclarissimo              4        0.567208               0.270317
4    obbedientissimo              5        0.579287               0.138426
5        sottoscrivo              5        0.562656               0.406765
6          carissimo              3        0.561063               0.478721
7         crediatemi              4        0.557948               0.286097
8      veneratissimo              2        0.549711               0.305080
9         riveritemi              1        0.549268               0.173765
10      rinfrescarle              2        0.563750               0.230954
11          tenetemi              3        0.569701               0.000000
12     pregiatissimo     

### TOPOS: SOLITUDINE

In [50]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("solitudine", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         coniunta              5        0.577682               0.000000
1        rimozione              5        0.556124               0.360285
2     tranquillità              1        0.592306               0.406437
3     propagazione              4        0.561147               0.428689
4       provocarne              2        0.555449               0.851781
5        instituto              3        0.541825               0.401020
6       voluttuosa              4        0.540319               0.285720
7     molestissima              3        0.549382               0.091340
8            porle              1        0.575270               0.018642
9          commoda              3        0.535936               0.379909
10   infelicissima              4        0.531122               0.488380
11         assidua              1        0.533740               0.564970
12        

                Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          isolamento              5        0.578748               0.277932
1              quiete              5        0.551889               0.146682
2           solitaria              5        0.521412               0.594314
3          consumarti              5        0.532697               0.258910
4        risvegliarvi              2        0.507347               0.245687
5     tranquillizzava              1        0.511539               0.319328
6       immalinconiva              4        0.493123               0.159336
7   inconsolabilmente              1        0.501482               1.000000
8           appartava              1        0.496783               0.763388
9            grigiore              1        0.494624               0.000000
10          tristezza              4        0.490953               0.370988
11            turberò              3        0.483697               0.027322
12          

In [51]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("solitario", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      estratto              5        0.523628               0.982829
1        estasi              5        0.538953               0.510894
2       horrore              5        0.534471               1.000000
3      romitoro              5        0.525433               0.825759
4    assalitolo              2        0.537443               0.573168
5      solitari              2        0.537426               0.949037
6        rimoto              4        0.487815               0.000000
7     solitaria              5        0.518275               0.499576
8   giovandogli              4        0.501411               0.418585
9       stavasi              4        0.497768               0.846281
10      eremita              4        0.495934               0.982639
11  porziuncola              4        0.494938               0.764904
12    porticale              1        0.450998   

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            ermo              5        0.571936               0.529420
1       solitaria              5        0.538758               0.173547
2        cenobita              1        0.521339               1.000000
3          sverni              5        0.499483               0.082435
4   passeggiatore              5        0.502150               0.076255
5         solingo              5        0.519236               0.282429
6           èremo              5        0.490775               0.171932
7     malinconico              1        0.494782               0.242958
8      montericco              1        0.490538               0.350954
9      frondeggia              4        0.473279               0.271523
10           vïal              4        0.484408               0.000000
11    melanconico              3        0.482902               0.049865
12       praticel              1        0.464416               0

### TOPOS: INFINITO

In [52]:
for periodo in lista_periodi_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("infinito", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_1250_1454_Tardo_Medioevo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   satisfaczione              5        0.481154               0.342011
1   infinitamente              5        0.479942               0.511388
2         recansi              1        0.497383               0.781837
3          capire              3        0.481883               0.743342
4      divisibile              5        0.494597               0.489630
5    inextimabile              4        0.465964               0.960786
6       ineffabil              4        0.476361               0.421827
7    consumazione              1        0.479149               0.601339
8         volerci              1        0.474815               0.704105
9         temiamo              5        0.452530               0.557756
10            dái              1        0.469060               0.264018
11      spregiata              3        0.444884               0.754506
12      allevando      

               Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            immenso              5        0.581400               0.299364
1         sterminato              5        0.482161               0.147769
2             eterno              5        0.483675               0.243227
3           increato              4        0.454258               1.000000
4        grandissimo              5        0.462530               0.144000
5          immensità              3        0.475845               0.088461
6       infinitesimo              4        0.455652               0.118953
7           infinita              5        0.478981               0.265709
8            volgasi              5        0.480536               0.098456
9   innumerabilmente              3        0.448223               0.168153
10           innondi              1        0.465373               0.045197
11        indefinito              3        0.458511               0.000000
12  incommensurabile     

## ANALISI PER FASE LETTERARIA

In [53]:
os.chdir(path_letterario_cade)

In [54]:
lista_periodi_letterari_cade = ['CADE_01_duecento_poetica_e_prosa Medievo_cade', 'CADE_02_dolce_stil_novo_cade', 
                                'CADE_03_petrarchismo_cade', 'CADE_04_boccaccio_e_umanesimo_cade', 
                                'CADE_05_manierismo_e_barocco_cade', 'CADE_06_classicismo_arcadia_cade', 
                                'CADE_07_illuminismo_neoclassicismo_cade', 'CADE_08_romanticismo_cade', 'CADE_09_verismo_cade',
                                'CADE_10_decadentismo_cade', 'CADE_11_avanguardie_primo_novecento_cade', 
                                'CADE_12_neorealismo_cade']

### TOPOS: PASTORE

In [55]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pastore", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            aide              1        0.849538               0.779468
1           prude              1        0.849132               1.000000
2      ritrovando              1        0.845062               0.490951
3        sparisce              3        0.763542               0.957788
4         andassi              1        0.842219               0.913258
5        spartito              2        0.758592               0.236750
6       approvare              1        0.834499               0.229042
7        mentendo              1        0.833294               0.023483
8         mettiti              1        0.832459               0.278800
9       saettando              1        0.831789               0.467861
10       acquistò              1        0.811303               0.015118
11           suto              2        0.734256               0.455050
12          

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         agnel              5        0.571405               0.437546
1        titiro              4        0.548666               0.835932
2   sacerdotale              5        0.561249               0.527697
3       agnello              4        0.532067               0.328850
4   semplicetto              1        0.584133               0.210783
5       armento              3        0.540691               0.440010
6       bifolco              4        0.532075               0.230141
7       agnella              2        0.547493               0.328955
8       ademaro              5        0.540499               0.004422
9       bifolci              3        0.524410               0.344296
10       travia              1        0.537359               0.000000
11    garzoncel              1        0.533266               0.074013
12    pescatore              4        0.512599               0.045251
13   anfitrione     

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       disonorante              2        0.614677               0.322638
1     chiacchierata              3        0.585523               0.348513
2         occorrerà              3        0.581600               0.958954
3             balìa              2        0.598273               0.649104
4              mara              1        0.654165               0.062477
5            indaga              1        0.653042               0.166577
6          scordati              3        0.577671               0.251533
7          scarpone              1        0.650470               0.333619
8          soggioga              1        0.648922               0.295867
9         adempiuto              1        0.647627               0.569064
10          accette              2        0.546271               0.105986
11         cencioso              2        0.542007               0.284541
12         permetti              2    

### Altre parole correlate

In [56]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pastori", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       mercenarj              5        0.790856               0.464217
1         fazioni              1        0.789112               0.209741
2         novelli              5        0.699080               0.190178
3        naschino              3        0.702363               0.527184
4        carretta              1        0.757316               0.058624
5       officiali              2        0.738226               0.132005
6          terran              1        0.750796               0.227584
7      somigliano              1        0.750445               0.777389
8      concedette              1        0.748838               0.287229
9        lasciata              1        0.746963               0.721120
10        codardi              2        0.683464               0.135844
11    avventurosi              1        0.719618               0.988421
12        an

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       tuguri              5        0.664011               0.082587
1     vedransi              5        0.636190               1.000000
2      situati              2        0.647276               0.177937
3     irrigato              5        0.643675               0.432521
4        alati              1        0.695500               0.367695
5    pastorali              3        0.643874               0.428165
6      nereidi              1        0.687068               0.000000
7      seguìti              5        0.628954               0.908860
8        feaci              1        0.685909               0.005078
9    boschetto              4        0.635780               0.188232
10     cantiam              1        0.635322               0.507542
11     ombrosi              2        0.610269               0.919877
12  cacciatori              1        0.628958               0.190104
13       ulivi              4     

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      pastorelle              5        0.711116               0.414037
1         puntano              3        0.650362               0.087648
2        giuocano              5        0.637531               0.317851
3          sciami              2        0.674342               0.306427
4      saltellano              1        0.700019               0.417255
5        vagavano              2        0.638763               0.007227
6        trapezio              2        0.649283               0.169488
7           genti              4        0.619460               0.178932
8          orfane              1        0.691047               0.000000
9          cicale              4        0.604348               0.058467
10         anitre              2        0.586136               0.499735
11       zampogne              1        0.612688               0.598368
12      affondate              1        0.603758               0

### TOPOS: AMORE

In [57]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amore", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         autore              4        0.407642               0.589535
1            ama              5        0.381431               0.849048
2          disia              5        0.402619               0.442630
3       adattare              5        0.391513               0.051922
4         oppone              4        0.381824               0.579463
5            ami              4        0.390822               0.915420
6         propri              2        0.388764               0.067127
7       ubidisce              1        0.419684               0.824544
8        amoroso              1        0.413879               0.138191
9        aggiate              1        0.409324               0.316389
10      prossimo              1        0.374360               0.164483
11         brama              2        0.360537               0.751348
12       concede         

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         amor              5        0.436094               0.553498
1       ardore              4        0.391046               0.691886
2       dividi              4        0.389144               1.000000
3          ven              5        0.382230               0.403742
4      inganno              2        0.395092               0.313103
5   affaticati              1        0.412815               0.116011
6      accendi              1        0.402360               0.378573
7    endimione              4        0.386548               0.344206
8    pescatore              2        0.388284               0.137265
9      involar              2        0.385221               0.033876
10      osando              4        0.392888               0.457303
11     franger              3        0.370913               0.326641
12      siegua              1        0.373189               0.101110
13        core              3     

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            amor              4        0.383608               0.102515
1       dibattuta              5        0.393058               0.769623
2        disamare              5        0.367439               0.644975
3       impulsivo              1        0.391453               0.366444
4    incantamento              1        0.389229               0.160129
5   riconquistare              4        0.366081               0.373981
6       verifichi              3        0.367209               0.011317
7   concupiscenza              3        0.367069               1.000000
8        piacerle              5        0.361605               0.261860
9        armonico              1        0.378495               0.375451
10     facciamolo              1        0.365936               0.000000
11     sviscerato              1        0.365455               0.143797
12      flussioni              1        0.364750               0

### Altre parole correlate

In [58]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amor", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      amoroso              1        0.571468               0.446228
1      deletto              4        0.517665               0.879575
2     disianza              5        0.516261               0.756660
3       altera              1        0.559853               1.000000
4        desio              1        0.558788               0.204869
5       ontoso              1        0.554469               0.414456
6        porea              4        0.520450               0.178548
7      pesanza              2        0.518193               0.052899
8       sembri              4        0.502430               0.250624
9      amorosa              1        0.547998               0.442159
10   spiacente              1        0.518508               0.868536
11    pietanza              1        0.517860               0.427600
12      mister              1        0.516921      

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          ardor              4        0.407076               0.000000
1          amore              5        0.436094               0.691828
2          togli              1        0.437025               0.430435
3        accendi              2        0.416359               0.986116
4   sconsigliato              2        0.416398               0.553402
5          cieco              2        0.423754               0.802105
6         serbar              1        0.435173               0.963424
7       confondi              1        0.434572               0.176974
8          error              3        0.407480               0.383910
9        inganno              3        0.396050               0.203260
10        laccio              4        0.396359               0.559427
11     disciogli              3        0.395709               0.375361
12           ven              3        0.387993               0.388982
13    

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            piagò              5        0.540883               0.569385
1        blandisce              1        0.598832               0.857603
2           irsuto              1        0.589239               0.444169
3           vorace              3        0.515621               0.390444
4         benedico              5        0.505383               1.000000
5         apprestò              1        0.577002               0.154379
6      schifiltosa              2        0.523147               0.477166
7       pacificato              1        0.569813               0.004442
8         chiomata              2        0.518678               0.423877
9          abusato              2        0.532816               0.347809
10          patrio              4        0.506326               0.199476
11           tange              4        0.509700               0.937425
12      uccidevano              2        0.482246  

In [59]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("innamorato", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    altamente              5        0.681886               0.270515
1        rusca              5        0.687534               0.705174
2    ringrazia              4        0.651857               0.435750
3      mettete              1        0.725861               0.000000
4     piacesse              1        0.721584               0.370570
5         vago              2        0.676613               0.341916
6    impiccato              1        0.718853               0.953016
7    ritornato              2        0.655327               0.179388
8     spietata              4        0.635166               0.941315
9      istrano              1        0.717108               1.000000
10      pensai              3        0.628085               0.237185
11     venduta              3        0.620980               0.344463
12        cert              1        0.641701      

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       ravvivasse              5        0.528374               0.769672
1            amerò              4        0.532677               0.685453
2         adottivo              2        0.540907               1.000000
3         trascurò              5        0.534286               0.532869
4          ravvivi              4        0.519497               0.001542
5         gradisce              5        0.519106               0.500000
6       ammogliato              5        0.517766               0.000000
7       dispietato              2        0.537228               0.987833
8       ammiratore              1        0.563416               0.732024
9        dispiacer              1        0.562227               0.305242
10       conoscerò              4        0.514554               0.331173
11          rivera              1        0.511772               0.546588
12      mascherato              3        0.497184  

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     significhi              5        0.552728               0.732895
1        temessi              5        0.543469               0.180516
2      gentaccia              3        0.555030               0.395058
3         amerei              2        0.570917               0.122327
4    corrisposto              4        0.544366               0.954159
5       fanatico              1        0.602285               0.331396
6      errabondo              1        0.600661               0.956915
7        capirle              1        0.597225               0.115538
8         avezza              1        0.596901               0.506802
9         spetro              1        0.596784               0.201765
10     sembrando              2        0.516388               0.451021
11         vacua              2        0.520773               0.091764
12        scrivi              3        0.507784               0.391647
13    

### TOPOS: CHIESA

In [60]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("chiesa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      sosteneva              5        0.574108               0.788359
1        galizia              2        0.526190               0.004984
2       convertí              2        0.537289               0.128554
3          lieti              5        0.506402               0.382586
4       proposti              1        0.554679               0.233883
5        batista              2        0.506266               0.347660
6          abati              1        0.550853               0.664917
7         odiava              2        0.536935               0.362908
8      presenzia              1        0.548459               0.360272
9        udirono              3        0.502955               0.584407
10    guardavano              4        0.511144               0.563904
11        egitto              1        0.528442               0.462186
12     cristiani         

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       cappuccino              4        0.551750               0.000000
1            feudo              4        0.550687               0.969262
2          martedì              4        0.540106               0.011607
3         ripeterà              3        0.543546               0.494597
4          roberto              2        0.564545               0.734149
5       stabilirsi              1        0.580151               0.117534
6            ricci              2        0.559238               0.281952
7     soprascritto              3        0.518801               0.089405
8        ripartirà              1        0.577802               0.374545
9         aiutante              1        0.576867               0.724198
10         viviani              4        0.520872               0.344253
11          vacare              1        0.536327               0.072327
12              mm              1        0.533315  

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      scorgemmo              5        0.386779               1.000000
1       cappella              5        0.391989               0.279876
2      chiesetta              5        0.388888               0.543757
3      abbaziale              5        0.412743               0.000000
4      sagrestia              1        0.395489               0.475571
5       romanica              5        0.390655               0.329112
6         navata              2        0.384844               0.092115
7        portale              3        0.386335               0.386670
8    benedizione              1        0.379986               0.094518
9      quadretto              1        0.379523               0.419461
10        guglia              2        0.371445               0.269711
11      crollata              4        0.382225               0.403011
12   sconsacrata              2        0.372388               0.514429
13  sa

### Altre parole correlate

In [61]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("papa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          acri              5        0.682020               0.685920
1   arcivescovi              5        0.645553               1.000000
2       vescovi              4        0.621411               0.606663
3       disseli              5        0.650499               0.403585
4    sottoposto              4        0.627507               0.660013
5      frategli              5        0.645544               0.606576
6         abati              1        0.652901               0.239338
7       parlati              4        0.615032               0.093249
8        legato              4        0.611497               0.028960
9      clemente              1        0.626725               0.255511
10   pregandoli              4        0.621347               0.277472
11     mandasse              2        0.597953               0.345577
12    niccolaio              1        

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       pontefice              5        0.585901               0.000000
1          urbano              5        0.514801               0.144283
2       bonifacio              1        0.497737               0.043506
3        anacleto              4        0.466177               0.136610
4     scomunicava              1        0.493631               0.369851
5        clemente              5        0.523520               0.039347
6         formoso              5        0.471742               0.023926
7          rivoca              1        0.467834               0.053226
8         eugenio              3        0.454363               0.554132
9    persuadevano              2        0.456993               0.015950
10      iscusarsi              4        0.461007               0.059837
11      cardinali              3        0.453438               0.184055
12     assunzione              2        0.452078               0

In [62]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("prete", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            beda              5        0.791700               0.417864
1   confermazione              5        0.736184               1.000000
2     confessione              2        0.756867               0.380586
3      bartolomeo              4        0.720110               0.471650
4          rinato              2        0.745449               0.548570
5           dassi              4        0.715364               0.716756
6       verginità              5        0.738371               0.205801
7     compunzione              5        0.726725               0.379272
8          strema              1        0.762926               0.493700
9          varria              1        0.761446               0.322337
10        dottore              4        0.708362               0.173315
11       catolica              2        0.715526               0.623910
12         s

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             cunto              1        0.847862               0.191921
1      sopracoperta              4        0.796174               0.031723
2         scrivesse              5        0.779338               0.229475
3          pregarmi              1        0.836058               0.041088
4       scetticismo              2        0.812326               0.003966
5         ripasserà              3        0.790719               0.335556
6       ricercarono              1        0.832331               1.000000
7   inesorabilmente              2        0.809885               0.198847
8        intenderci              3        0.790227               0.580248
9           fioriva              1        0.828208               0.850165
10      soggiaciuto              3        0.764791               0.294224
11          eroismi              1        0.792802               0.710477
12       mandandola              4    

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         ansava              5        0.462686               0.996151
1       allevata              5        0.447704               0.541850
2          goffo              5        0.447324               0.666573
3          tossì              4        0.441253               0.727996
4      pidocchio              5        0.460968               0.694286
5           lupo              1        0.484578               0.255390
6     alchimista              1        0.484269               0.108479
7       drusilla              1        0.481843               0.182226
8   approfittato              2        0.459973               0.356041
9     angustiata              2        0.457743               0.966011
10    fermiamoci              4        0.435442               0.389498
11   manrovescio              2        0.442781               0.000000
12       curguss              3        0.432750               0.431538
13    

In [63]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("frate", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    esaltazione              5        0.606231               0.302937
1          nocte              5        0.599650               0.441719
2         morali              5        0.606855               0.462441
3          guido              3        0.570697               0.416919
4         boccio              4        0.556105               0.393626
5      niccolaio              5        0.561527               0.832999
6          petro              1        0.609762               0.964626
7        cavando              2        0.551561               0.608633
8      dilettoso              1        0.593092               0.038950
9      volontate              1        0.590658               0.377574
10      guittone              3        0.547633               0.970882
11         berto              1        0.554036               0.516766
12         tucte         

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     francescano              5        0.584293               0.659636
1         aicardo              5        0.614816               1.000000
2     agostiniano              5        0.567372               0.133756
3           porro              5        0.532187               0.436670
4         zoccoli              5        0.560001               0.090164
5      domenicano              5        0.538692               0.453885
6     carmelitano              5        0.536293               0.781025
7           rocco              1        0.524478               0.091488
8    plenipotente              1        0.521990               0.046806
9          vedano              1        0.518853               0.000000
10  pronunziarono              4        0.510922               0.060627
11     cappuccino              2        0.494069               0.334062
12        dissono              1        0.487993               0

In [64]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("dio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           idio              4        0.398674               0.839185
1          iddio              5        0.396725               0.105300
2         grazie              5        0.419661               0.156695
3    onnipotente              5        0.421909               0.899390
4      osservare              5        0.397898               0.359159
5      legittimo              1        0.421560               0.964581
6      partefici              2        0.392936               0.603090
7   eternalmente              1        0.415525               0.633813
8    annunziasse              1        0.411705               0.994503
9   gastigamenti              5        0.381248               0.220898
10       procede              1        0.369767               1.000000
11      creatore              1        0.368401               0.621581
12       mandata         

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0             oh              5        0.525302               0.171939
1       illumini              5        0.444696               0.000000
2          gliel              5        0.388804               0.050434
3        perdoni              5        0.395969               0.253188
4          iddio              2        0.369137               0.145590
5       benedica              5        0.373953               0.558987
6      intendimi              5        0.368280               0.024836
7     rimetterle              1        0.380592               0.100122
8           raby              1        0.379208               0.140902
9       piangerà              1        0.375475               0.158457
10    improvvida              3        0.360827               1.000000
11       odiarlo              4        0.371489               0.292390
12       sorgete              2        0.354227               0.180263
13    

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          iddio              5        0.469887               0.000000
1           gesú              4        0.400661               0.080926
2        perdoni              3        0.400539               0.204695
3          saulo              2        0.391982               0.110585
4    glorificato              5        0.406230               0.114380
5         messia              5        0.390715               0.007793
6   soccorretemi              5        0.387194               0.280732
7      redentore              1        0.385642               0.338596
8   misericordia              3        0.381294               1.000000
9    onnipotente              1        0.380017               0.177392
10      benedica              4        0.384651               0.079500
11      deperire              1        0.374392               0.066237
12     aiutatemi              3        0.381431               0.279463
13    

In [65]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("cristo", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0            jesù              5        0.641665               0.199115
1            gesù              5        0.553252               0.179181
2    pietosissimo              5        0.515311               0.146458
3         unzione              4        0.507625               0.507180
4       salvatore              1        0.533893               0.194338
5    dilettissimo              5        0.494408               0.172239
6            gesú              5        0.497127               1.000000
7    dilettissima              5        0.497984               0.225572
8   sapientissimo              4        0.492482               0.295696
9        dilicate              3        0.488537               0.399276
10    nobilissimo              2        0.502730               0.168445
11     sofferendo              4        0.485533               0.129714
12       inf

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           gesù              5        0.534482               0.000000
1          moisè              2        0.482802               0.281639
2      redentore              2        0.472768               0.025197
3   predicazione              2        0.475868               0.342886
4        vangelo              3        0.466372               0.309254
5      discepoli              4        0.453145               0.295884
6       apostoli              5        0.451500               0.127886
7     spedalieri              1        0.473329               1.000000
8     rendendolo              1        0.471666               0.142245
9   giustificati              5        0.452901               0.293767
10      concilia              4        0.458575               0.552617
11   concludesse              1        0.454690               0.066004
12      evangeli              3        0.433739               0.100522
13    

### TOPOS: FELICITA'

In [66]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("felicità", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      procurano              4        0.839391               0.425054
1       deludere              4        0.825497               0.426770
2        disfido              1        0.858061               0.871787
3     faccendomi              1        0.856877               0.000000
4        accolga              3        0.822420               0.236796
5       limosine              1        0.854421               0.282722
6          ponta              4        0.841404               0.476249
7       fallisse              4        0.828066               1.000000
8      chiedendo              1        0.850089               0.103090
9          unica              4        0.828858               0.980961
10     congionto              1        0.810609               0.517571
11      dissenso              2        0.802296               0.801208
12         lxxiv         

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   sacrificata              5        0.494858               0.079265
1    prosperità              5        0.462351               0.448615
2       terrene              4        0.461136               0.000000
3    infelicità              3        0.460782               0.457911
4     succedono              4        0.463513               0.692846
5       garante              4        0.453267               0.030260
6    conseguita              5        0.449652               0.492710
7     depravato              4        0.450414               0.034077
8       mondane              4        0.449488               0.584373
9    dimentichi              1        0.469675               0.446613
10  riordinando              3        0.449346               0.404917
11    perseveri              2        0.446069               0.603506
12   sgomentati              1        0.452984               0.444885
13     calcolar     

### Altre parole correlate

In [67]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("gioia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       gioiosa              5        0.578877               0.000000
1           nòi              5        0.598306               0.464515
2         beltà              5        0.552054               0.429303
3       donando              5        0.549086               0.595939
4         tenìa              4        0.550866               0.480944
5        prendo              1        0.594462               0.570006
6        gioiva              4        0.524036               0.410805
7     dubitanza              3        0.530862               0.830268
8       leggera              1        0.579884               0.351176
9     sembrando              1        0.577012               0.165753
10         noia              4        0.528161               0.839059
11        desio              1        0.538891               0.387204
12       amarla              1        

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        provò              2        0.506565               0.085426
1    scintillò              5        0.476311               1.000000
2      terrore              3        0.480748               0.891755
3      avvampo              1        0.527638               0.014127
4    rivedersi              5        0.471215               0.000000
5    piangesti              5        0.492895               0.443675
6      lagnare              4        0.460667               0.014526
7    intenerir              2        0.476734               0.304875
8        balza              1        0.515586               0.542813
9     svegliar              2        0.466504               0.109309
10     letizia              4        0.444380               0.398509
11      insane              4        0.450242               0.252224
12      esulta              1        0.465696               0.434783
13     martiro              1     

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   riconciliava              4        0.386149               0.354949
1    contentezza              3        0.368710               0.856205
2      insperata              4        0.367023               0.675295
3   sconvolgente              1        0.384982               0.667657
4    liberatrice              1        0.379959               0.634730
5         arebbe              5        0.379284               0.276161
6    traboccante              1        0.378735               0.182740
7     refrigerio              5        0.371939               0.245907
8       commossa              1        0.370282               0.382429
9    incredulità              1        0.366716               0.506248
10    schiattava              4        0.356875               0.592949
11      ghignava              1        0.362576               0.599391
12        fruire              4        0.356665               0.870350
13    

### TOPOS: PATRIA/ITALIA

In [68]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("patria", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   quintodecimo              5        0.709665               0.301578
1           arra              4        0.714359               0.255980
2       traggano              3        0.717645               0.066819
3        valichi              1        0.792233               0.083085
4       morranno              1        0.791005               0.491659
5         elesse              2        0.721628               0.276599
6      conchiudo              1        0.788388               0.242809
7        rifarne              1        0.786266               0.974903
8       limosine              1        0.785893               0.357311
9   dimenticanza              1        0.785580               0.226153
10         spose              1        0.724157               0.956574
11       tremerà              1        0.714179               0.445354
12     accidiosi         

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       grugniti              5        0.393375               0.486223
1        innesto              4        0.390067               0.103570
2      partenope              2        0.377029               0.428988
3     abbandonai              4        0.370553               0.400903
4     emancipata              3        0.364533               0.368654
5       disporlo              5        0.381821               0.739892
6   concittadino              5        0.365330               0.157211
7       liberate              3        0.360996               0.350712
8     fuggirsene              3        0.361411               0.280427
9      sacrificò              4        0.365017               1.000000
10    combattuta              1        0.368010               0.925538
11     comunione              4        0.366763               0.271079
12      rivolger              2        0.348955               0.097051
13    

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   prolungamento              5        0.466948               0.497600
1       riassunto              2        0.482083               0.708114
2        successa              5        0.470689               0.987335
3            cuna              1        0.498969               1.000000
4         novello              1        0.498568               0.292527
5        igienica              1        0.498375               0.992934
6    associazione              1        0.497905               0.195273
7   vergognandomi              3        0.462548               0.482540
8     nascondermi              4        0.466669               0.623742
9    appartenente              2        0.474438               0.319050
10       lucifero              1        0.467347               0.615679
11      separiamo              4        0.458434               0.000000
12      nutriente              2        0.439369               0

In [69]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("italia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           lamagna              3        0.717176               0.625032
1          barberia              3        0.716037               0.405201
2          sergente              4        0.702070               0.161391
3         selvaggia              1        0.764048               0.000000
4           lontana              3        0.688396               0.304087
5               fuo              2        0.740815               0.629174
6            marina              1        0.760327               0.156735
7           volemmo              1        0.759055               0.199985
8            fidato              1        0.758311               0.303713
9      raccomandata              1        0.756081               0.955690
10             suto              4        0.680654               0.820551
11      racquistata              1        0.725410          

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          germania              4        0.390022               1.000000
1          lussuria              2        0.393559               0.483650
2      richiedevano              5        0.379557               0.134183
3             araba              1        0.406747               0.494048
4           regnava              5        0.389008               0.466077
5           provava              5        0.379144               0.133130
6          classico              3        0.374323               0.733657
7           sposina              3        0.379399               0.000000
8         rientrata              1        0.394865               0.198845
9            curare              3        0.374976               0.240519
10        imbeversi              3        0.356439               0.310269
11      letteratura              1        0.363436               0.789986
12         dominava              1    

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       affermiamo              5        0.401695               0.519946
1        diffusero              3        0.391797               0.472860
2        tollerati              2        0.404870               0.000000
3         saporita              4        0.404376               1.000000
4      conclusioni              1        0.430553               0.503778
5       scaramucce              3        0.401743               0.355294
6         vedevamo              5        0.396796               0.045318
7      minutissimi              5        0.394400               0.197918
8      piedigrotta              2        0.390388               0.347534
9            mafia              1        0.413785               0.417161
10          rinata              2        0.386639               0.721063
11            bevo              1        0.398864               0.398604
12  glorificazione              1        0.395516  

### Altre parole correlate

In [70]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("stato", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       affogato              5        0.417342               0.891669
1        tornato              3        0.437533               0.538976
2        volersi              1        0.456903               0.842851
3        ribelle              1        0.455463               0.051307
4       avanzare              2        0.427063               0.652260
5       servando              2        0.431208               0.327973
6          afare              1        0.447607               0.041151
7        allegro              1        0.443418               0.809035
8           port              2        0.408072               0.333905
9   considerando              1        0.440516               0.403617
10      glorïoso              2        0.402026               0.231966
11      burbanza              2        0.399235               0.360488
12    esaminando         

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     giustificato              5        0.376857               0.462788
1           istato              4        0.368322               0.363055
2     presupponeva              1        0.368712               0.215383
3    impraticabile              3        0.349556               0.557694
4       indisposto              1        0.364714               0.607562
5       deviazione              1        0.364666               0.737911
6        gastigati              1        0.362724               0.400399
7      desiderarla              1        0.358603               0.696155
8       defraudato              1        0.356284               1.000000
9      travaglioso              5        0.361522               0.413063
10          nocivo              1        0.354547               0.738928
11        rendermi              2        0.341115               0.243846
12     antecessore              1        0.351869  

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    secondarla              1        0.407282               0.304096
1     trucidato              4        0.375847               0.148534
2    disdegnato              1        0.390906               0.161195
3   legittimato              2        0.379671               0.616502
4   riuscirebbe              1        0.387865               0.025754
5     sembrarmi              1        0.385179               0.556722
6    ricoverato              5        0.392407               0.000000
7    restaurato              1        0.382917               0.163795
8     tribolata              1        0.381718               0.252591
9       amadore              1        0.381573               0.262483
10      paventi              4        0.391476               0.572156
11    iracondia              4        0.376390               0.026669
12   degradante              4        0.393115               0.340464
13  maltrattato     

In [71]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("impero", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       iniquitade              5        0.748034               0.371982
1       conservato              1        0.774890               0.422316
2          gustare              2        0.737038               0.372203
3         promessa              1        0.768803               0.887606
4         procurar              3        0.723339               0.112452
5          disceso              1        0.763410               0.223967
6         accresca              3        0.688825               1.000000
7            ornai              1        0.762101               0.157581
8            ardor              1        0.758145               0.131350
9          delizie              3        0.701133               0.077963
10         superna              3        0.683387               0.504927
11       rifiutata              1        0.717904               0.056963

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     credesti              1        0.514662               0.384097
1   ribellante              4        0.468259               1.000000
2       esalta              3        0.467294               0.420210
3        finor              1        0.506829               0.172838
4   vendicator              3        0.478554               0.379943
5     monarchi              2        0.482101               0.904123
6    usurpator              1        0.500057               0.000000
7        macon              1        0.493975               0.128017
8       nazion              1        0.492905               0.245373
9    oppressor              1        0.492025               0.746789
10     presuma              2        0.464478               0.142312
11   obedienti              4        0.462470               0.601444
12    femmineo              1        0.464005               0.733686
13     trionfò              1     

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         arroga              5        0.491298               0.432849
1   hohenzollern              5        0.495290               1.000000
2      liberator              5        0.493259               0.921679
3   appartenersi              5        0.497572               0.325849
4      fondatore              3        0.459536               0.000000
5    stabilmente              1        0.489259               0.423276
6       crociato              2        0.451374               0.525162
7      inoculato              3        0.448807               0.391012
8        primato              1        0.475061               0.971510
9    aggiudicare              5        0.452141               0.391223
10     germanici              2        0.448790               0.278180
11      ministra              1        0.466663               0.946158
12      vassallo              3        0.438189               0.614138
13    

In [72]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("nazione", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          duca              3        0.747584               0.383007
1     combatter              3        0.761258               0.973238
2        groppo              3        0.740783               0.508813
3       paragon              1        0.806840               0.358536
4        domane              1        0.805499               0.000000
5       sentita              2        0.752393               0.222272
6        ascosa              1        0.803560               0.628121
7      opinione              1        0.803438               0.364864
8        tenire              2        0.744000               0.851812
9       vegnano              1        0.801964               0.242527
10      masnada              1        0.802997               0.814530
11      istrano              1        0.790118               0.799442
12      creasti              1        

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         dotato              2        0.508784               0.380383
1       italiana              2        0.501442               0.124892
2         lirica              2        0.487881               0.558629
3    letteratura              2        0.498620               0.481496
4      anteposta              5        0.482667               0.292292
5     evangelica              1        0.528921               0.077326
6      arditezza              3        0.474583               0.932233
7       dominava              5        0.478836               0.086817
8   cittadinanza              1        0.521819               0.522075
9    pronunciata              1        0.520845               0.807846
10    introdusse              4        0.457995               0.271707
11       affolla              1        0.463987               1.000000
12       imitata              1        0.462627               0.138951
13    

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         economica              5        0.651759               0.033305
1          beneficî              5        0.644276               0.517746
2     rappresentati              5        0.671970               0.875011
3      collettività              2        0.692107               0.353910
4    rappresentanza              1        0.708841               0.523860
5      proletariato              1        0.707977               0.423392
6         sindacato              2        0.675898               0.403935
7       individuale              1        0.700058               0.573638
8       coscrizione              3        0.645095               0.125802
9             quota              2        0.663693               0.563037
10       costituita              2        0.631644               0.934400
11      concorrenza              2        0.622342               0.148627
12     indipendenza              1    

In [73]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("bandiera", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
La chiave non esiste nel modelloCADE_01_duecento_poetica_e_prosa Medievo_cade
###CADE_02_dolce_stil_novo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         grieve              3        0.846657               0.591040
1        volessi              1        0.864837               0.929978
2          muoio              1        0.863541               0.881221
3    incontrasse              2        0.847407               0.438885
4      spiraglio              1        0.861745               0.012396
5           lxvi              1        0.861371               0.481888
6         amarvi              1        0.860942               0.673197
7       travolse              2        0.850780               0.686682
8        calendi              1        0.858230               0.937688
9          pezza              1        0.857290               0.056524
10    accorgesse              4        0.842086       

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     stendardo              5        0.518171               0.038818
1     sventolar              5        0.475636               0.004966
2      vessillo              5        0.461186               0.755732
3     tricolore              5        0.482526               0.098497
4     borbonica              5        0.497078               0.230626
5    sventolare              5        0.461417               0.096720
6      coccarda              2        0.451735               0.000000
7    sventolava              3        0.455825               0.153919
8   sventolando              5        0.461334               1.000000
9       iridata              2        0.445111               0.186627
10      aguglia              1        0.432249               0.341788
11  dissolvente              2        0.440172               0.507423
12   aggressiva              2        0.435172               0.342748
13     sventola     

### TOPOS: GUERRA

In [74]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("guerra", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          romani              3        0.472149               0.582015
1            movi              5        0.452746               0.752433
2       consolato              5        0.450219               0.670028
3       terminata              5        0.443293               0.505562
4        paladini              4        0.451945               0.359600
5           vinta              2        0.448061               0.104479
6         perlato              1        0.464291               0.000000
7           scesa              1        0.460053               0.708044
8   cominciassero              4        0.444370               0.387132
9      confondere              1        0.455696               0.696002
10       dovesser              2        0.439289               0.881768
11     richiesero              2        0.429435               0.255372
12     torne

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           turco              5        0.371069               0.143290
1           fiera              4        0.398954               0.465999
2          guerre              4        0.369768               0.104463
3        ostinata              5        0.371902               0.355416
4       intestina              3        0.372139               0.249392
5        solimano              1        0.355189               0.456967
6       temendosi              2        0.349845               1.000000
7          domati              2        0.367376               0.000000
8         ardendo              1        0.346877               0.498116
9     sollecitata              1        0.345918               0.315261
10          pugna              3        0.364091               0.265881
11     sovrastava              2        0.352015               0.538873
12   guerreggiata              3        0.359135               0

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      mondiale              5        0.496559               0.199553
1         corea              4        0.404244               0.176244
2       etiopia              5        0.442006               0.061874
3       algeria              3        0.386875               0.121700
4     internato              1        0.391852               0.000000
5     rivederle              5        0.383692               0.040483
6     prigionia              3        0.390876               0.260779
7     abissinia              5        0.396598               0.026333
8     arruolato              2        0.371677               0.091416
9     auschwitz              2        0.376733               0.563166
10   impoverita              2        0.377860               0.089983
11       eccidi              1        0.374966               0.049508
12    accoppare              4        0.393687               0.161546
13  importabile     

### TOPOS: FIGURA FEMMINILE

In [75]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("donna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        chiede              2        0.458885               0.578666
1        saggia              4        0.437875               0.517727
2       amadore              3        0.445474               0.689423
3        amanza              4        0.442295               0.444577
4      criatura              4        0.449373               0.835699
5      openione              2        0.445536               0.653382
6      dichiara              1        0.467458               1.000000
7       amorosa              4        0.451253               0.019731
8      protesta              5        0.457062               0.461110
9       meglior              1        0.460628               0.869102
10        provo              3        0.444076               0.579855
11     pietanza              3        0.440212               0.477818
12      villana              4        

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        adorata              5        0.419462               0.011531
1          pelia              4        0.407192               0.554164
2       impudica              4        0.432380               0.413066
3        zenobia              1        0.423840               0.526702
4      andronica              1        0.418866               0.540800
5   incantatrice              4        0.419346               0.392646
6        camilla              1        0.413794               0.367459
7          amata              1        0.413172               0.427020
8        egregia              3        0.411036               0.395223
9          celan              1        0.412377               0.179389
10        costei              1        0.439336               1.000000
11       signora              1        0.432922               0.365144
12    amatissima              1        0.418238               0.388324
13   c

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        ragazza              5        0.451504               0.087053
1       donnetta              5        0.383111               0.116913
2          clara              5        0.389585               0.089449
3       caterina              5        0.384578               0.320341
4           oddo              5        0.401629               0.371192
5   scarmigliata              5        0.382354               0.139419
6       bruttina              2        0.361842               0.305441
7     grassoccia              4        0.368198               0.059474
8         etiope              2        0.358890               0.153997
9       teresita              1        0.353034               0.125546
10      seminuda              1        0.364187               0.493506
11      seminudo              1        0.361609               0.214200
12        quietò              2        0.374126               1.000000
13    

In [76]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("madre", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        ventre              5        0.580946               0.184337
1         fuggì              5        0.568797               0.270174
2      matrigna              5        0.545201               0.099770
3     uccidesti              4        0.501593               0.209355
4     rinchiusa              5        0.494126               0.528746
5        nacque              1        0.545752               0.113131
6            eh              1        0.518532               0.908269
7   avventurato              3        0.472918               0.730107
8      onorarli              1        0.513829               0.734268
9      creatura              2        0.484287               0.051880
10     fuggirei              3        0.463488               0.493232
11      ubidito              2        0.488236               0.109513
12    riverisce              2        

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         esclamò              4        0.419679               0.351419
1      accostarmi              5        0.430697               1.000000
2          vedova              5        0.421609               0.807799
3   riguardandola              5        0.415820               0.563792
4         accorra              1        0.441682               0.976624
5        assalita              1        0.440346               0.969822
6     avventurosa              1        0.439958               0.000000
7        contesse              5        0.412973               0.205646
8    festeggiando              3        0.415342               0.111257
9       rinfaccia              2        0.433570               0.497688
10        nudrice              4        0.403166               0.081702
11         argene              1        0.416751               0.615022
12       accomoda              2        0.404613               0

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    consanguinei              5        0.421164               0.005890
1        bonciani              5        0.405211               0.481653
2   rimproverando              3        0.389828               0.206467
3        tradisse              1        0.412414               0.664823
4         materno              1        0.406150               0.130712
5      impietrita              3        0.392549               0.519294
6       snaturata              3        0.402273               1.000000
7         cognata              1        0.398285               0.042765
8       prenderla              1        0.398044               0.403743
9      giovinetta              1        0.397075               0.056426
10        infermo              2        0.387542               0.083881
11      cacciarla              2        0.379791               0.493862
12     imprecando              2        0.376787               0

In [77]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("femmina", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      creatura              5        0.597254               0.140889
1       maschio              5        0.576977               0.000000
2        creata              5        0.521198               0.582087
3     congiugne              1        0.536812               0.474128
4            eh              5        0.498733               1.000000
5            iu              5        0.516324               0.318803
6      incendio              2        0.522088               0.254548
7       miseria              5        0.504192               0.480281
8   riconoscere              2        0.518514               0.440432
9    partorisse              3        0.488408               0.337962
10     giaciuto              2        0.489440               0.859802
11       legame              1        0.501504               0.356542
12     angoscie              3        

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        treman              5        0.777628               0.487100
1        priamo              1        0.816812               0.397491
2      vilipesa              5        0.747243               0.243288
3        onusto              1        0.813311               0.953324
4   raggiungerò              5        0.754061               0.933340
5         cirra              1        0.810441               0.255515
6       prevede              2        0.783329               0.908606
7     faretrato              1        0.807714               0.378353
8         notar              5        0.731970               0.000000
9      infocata              1        0.804338               0.974347
10       zefiro              3        0.725449               0.754666
11     rabbioso              2        0.723042               1.000000
12       appare              1        0.746812               0.312935
13        sorda     

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        maschio              5        0.534691               0.000000
1     maschietti              5        0.473458               0.267445
2    femminuccia              5        0.466241               0.301265
3        domarlo              5        0.457489               0.241281
4     vincenzino              5        0.460797               0.380474
5     lussuriosa              5        0.448401               0.478468
6   vergognavano              2        0.464175               0.130695
7      augurarsi              3        0.453752               0.219017
8     fidanzarsi              1        0.477068               0.445800
9         pelosa              1        0.472181               1.000000
10     quietarlo              1        0.460606               0.434242
11      lupanari              4        0.438643               0.329479
12      riordinò              1        0.444742               0.185899
13    

In [78]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("sposa", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          sposo              5        0.734712               0.343521
1    avventurata              5        0.737024               1.000000
2    nobilissimo              5        0.715290               0.000000
3   dilettissima              5        0.726769               0.058373
4   dilettissimo              5        0.719722               0.288559
5           jesù              5        0.684226               0.855545
6   contemplante              4        0.686515               0.756295
7     soavissimo              2        0.696873               0.256316
8    abbracciata              3        0.690417               0.693861
9   pietosissimo              5        0.706364               0.568139
10   ricchissimo              1        0.688465               0.316025
11      cognosci              2        0.670172               0.434463
12     benedetta         

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        pudica              5        0.492462               0.476452
1     cittadina              4        0.489386               0.600365
2       esprima              2        0.503885               0.586746
3        coppia              3        0.496472               0.000000
4          fida              4        0.498626               0.683825
5      amplesso              3        0.482721               0.760733
6         pelia              2        0.496822               0.050673
7    raccomando              1        0.511093               0.352783
8    amatissima              4        0.483656               0.787041
9       medesma              1        0.506810               0.895015
10     condurne              1        0.494666               0.455818
11     trovasti              2        0.478921               0.818406
12      germana              1        0.489454               0.240311
13      nomarti     

### TOPOS: NOTTE/SERA

In [79]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("notte", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      vegghi              5        0.433101               0.094329
1   vegghiare              5        0.431587               0.637354
2     levammo              3        0.430796               0.936949
3       verrà              5        0.426226               0.534906
4       pesca              5        0.444394               0.000000
5        alba              5        0.429398               0.024248
6        sera              1        0.451664               0.660128
7      piango              4        0.426469               0.657092
8   dormivano              5        0.423108               0.333464
9       coppo              1        0.447793               0.881593
10  soggiorno              1        0.432544               0.589657
11     nacqui              1        0.430610               1.000000
12     parria              3        0.410739               0.688

         Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        varcò              5        0.447321               0.168989
1    tenebrosa              5        0.435835               0.304823
2        fosca              1        0.426490               0.332096
3         sera              1        0.423551               0.248110
4       cingea              4        0.412799               0.580517
5       spuntò              1        0.414319               0.609626
6      acquosa              2        0.406053               0.418729
7       dormii              4        0.414153               0.225908
8        erina              1        0.409983               0.673798
9     nebulosa              1        0.409557               0.248900
10     sedenti              1        0.431776               0.239548
11    tenebria              3        0.414052               0.079864
12     involge              4        0.427935               0.223502
13     musiche              1     

In [80]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("sera", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         suona              5        0.642905               0.732089
1       rifanno              5        0.617129               0.328882
2        tegoli              4        0.567923               0.318501
3   celatamente              1        0.648317               0.639702
4    intagliato              4        0.577191               0.025796
5      giunsono              1        0.646994               0.707676
6      confortò              4        0.571733               1.000000
7          sott              2        0.626976               0.516025
8    beveraggio              2        0.591304               0.660352
9         coppo              3        0.594572               0.000000
10    appiccato              1        0.605923               0.074563
11       ispada              1        0.602229               0.124999
12       cavare              1        

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        mattina              5        0.585134               0.055840
1          notte              5        0.503985               0.191463
2           ieri              5        0.436913               0.204128
3        sabbato              1        0.427844               0.000000
4            jer              5        0.439235               1.000000
5     tempissimo              4        0.410606               0.029614
6       svegliai              2        0.412037               0.045621
7     mezzanotte              3        0.410177               0.557339
8   stanchissimo              5        0.417766               0.056684
9   piovigginava              2        0.411075               0.050516
10    passeggiai              2        0.412205               0.141994
11       mattino              2        0.407243               0.077533
12      domenica              3        0.412051               0.087018
13    

### Altre parole correlate:

In [81]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("tenebre", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     mettetelo              5        0.769109               0.231093
1     battiture              5        0.672814               0.420863
2      stridori              5        0.684038               0.497481
3       mandalo              4        0.634461               0.532242
4       atterrò              5        0.658133               0.000000
5       manderà              5        0.636777               0.087274
6       caddono              1        0.679476               0.503247
7   oscurissimo              2        0.627681               0.206885
8         formò              1        0.673919               0.216638
9      scandali              2        0.643231               0.297331
10      vangelo              1        0.633841               0.309418
11  abbandonerà              1        0.621768               1.000000
12  rivelazione              1        

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   enigmatiche              5        0.585644               0.482278
1      erravano              5        0.552490               0.000000
2     autentici              5        0.549102               0.304760
3   inviluppati              2        0.561834               0.174814
4       gustano              4        0.557220               0.288716
5   inviluppato              1        0.618734               0.969003
6    rammentata              2        0.581908               0.753969
7      fiaccole              5        0.542158               0.507464
8        deviar              1        0.614565               0.430647
9        idioti              1        0.611113               0.542638
10      comedie              3        0.525227               0.943335
11   permanenti              4        0.527626               0.020535
12    descriver              4        0.523069               0.097405
13      latrati     

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        fugate              5        0.479662               0.208681
1      stellato              4        0.478120               0.462092
2    immutabile              1        0.532103               0.052827
3   illuminando              1        0.523598               0.158772
4        orride              4        0.470854               0.075815
5     schiariva              3        0.470395               0.741061
6      frangono              2        0.494781               0.135671
7    fluttuando              2        0.471339               0.454134
8      velavano              2        0.486169               0.664720
9    diffondono              1        0.503279               0.528905
10        romba              4        0.456175               0.923596
11  illuminarla              1        0.479175               0.524529
12    barbanera              4        0.472225               1.000000
13      potenze     

In [82]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("ombra", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    tempestosa              4        0.630137               0.417517
1          ista              5        0.602317               0.432098
2        amarle              1        0.671254               0.142116
3         talpa              3        0.608129               1.000000
4      disfarsi              4        0.593252               0.053980
5     spendendo              2        0.612085               0.182659
6          pino              3        0.607141               0.111047
7      perversa              1        0.663283               0.080475
8   rammarichio              2        0.618308               0.284977
9    acquistate              2        0.607710               0.331877
10      folgore              4        0.598368               0.486509
11     imbolato              1        0.615237               0.457511
12     infinchè              1        

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     agghiaccio              5        0.542869               0.643311
1           aura              5        0.546051               0.680740
2         baleno              2        0.560954               0.856059
3       susurrar              4        0.533674               0.475616
4         spenga              2        0.562229               0.184837
5        orribil              5        0.535327               0.443310
6        tremulo              5        0.526132               0.226812
7         sibilo              1        0.573190               0.305012
8          fibra              3        0.528614               0.554589
9        inchina              4        0.525868               0.019714
10          nube              2        0.514112               0.126802
11      infausta              2        0.507464               0.160320
12        tralci              3        0.498402               1.000000
13    

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        cubica              5        0.486891               0.344781
1   sottomarino              1        0.518035               0.468943
2        stuoia              3        0.468476               0.603180
3       disegna              3        0.469132               0.263456
4        calava              2        0.461760               0.813815
5      chiarore              2        0.475985               0.706104
6      ramaglia              1        0.499187               0.538510
7       cortina              1        0.498305               0.351852
8         tetra              1        0.497201               0.284828
9        scorge              2        0.455281               0.191342
10     quadrato              4        0.464625               0.225128
11  trasparisse              4        0.454748               0.346443
12      rientro              1        0.457657               0.495981
13    profilava     

### TOPOS: FORTUNA

In [83]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("fortuna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       nochier              5        0.671794               0.513810
1   infortunato              5        0.651400               0.246152
2     sorprende              4        0.635792               0.597355
3       beltate              4        0.631794               0.622976
4        fallir              1        0.685825               1.000000
5      disperde              4        0.641078               0.544668
6     doglienza              5        0.643926               0.690232
7     procaccia              2        0.655189               0.082342
8    provvedere              1        0.676730               0.873742
9     operazion              2        0.645886               0.497046
10      pesanti              4        0.623420               0.640613
11   tempestoso              1        0.658623               0.438460
12         foll              1        

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           sorte              5        0.404061               0.011471
1       sottragga              5        0.376822               0.492288
2         avversa              4        0.376569               0.122216
3       mendicare              4        0.371995               0.026175
4   pretendessero              3        0.362494               0.284720
5          favori              5        0.392988               0.795502
6      bastassero              1        0.356531               0.090366
7             vuò              1        0.356501               0.407798
8    persecutrice              5        0.368562               0.150060
9      collocarmi              1        0.351331               0.344409
10      congiurar              4        0.365330               0.232346
11    stravaganza              1        0.352315               0.161881
12      trascinar              3        0.349913               1

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         amando              5        0.433073               0.484655
1          narra              3        0.431793               0.752790
2     ritrovarla              2        0.426330               0.142868
3     tratteremo              3        0.407726               0.158803
4      candidato              4        0.386886               0.323607
5     trattenuta              1        0.459677               0.000000
6     guadagnava              1        0.455242               0.236008
7        mancati              4        0.400561               0.081601
8        farvela              2        0.403754               0.340336
9       sboccati              1        0.447785               0.391635
10      abitando              2        0.404217               0.480350
11      esponete              1        0.394207               0.958794
12       stuarda              1        0.390779               0.290143
13   s

### TOPOS: NOIA

In [84]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("noia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
        Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       croia              5        0.613687               1.000000
1     donando              5        0.601871               0.660098
2         nòi              5        0.591030               0.585163
3   dubitanza              5        0.579533               0.948652
4      calere              2        0.582703               0.838606
5     affanno              1        0.619201               0.630575
6     gioiosa              2        0.584597               0.533477
7       fella              3        0.569107               0.135752
8       laida              1        0.612955               0.677732
9       tarda              2        0.581977               0.037403
10  spiacente              2        0.565410               0.941828
11   partenza              1        0.576472               0.369381
12   contezza              1        0.574657               0.383

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    ricompensato              5        0.514827               0.297830
1        insipida              5        0.525123               0.144462
2      prolungare              4        0.505550               0.003345
3      prevenisse              5        0.512092               0.524109
4         scusare              2        0.521698               0.490450
5     inefficacia              2        0.521666               0.423705
6       soffrirli              1        0.548023               0.754196
7      largamente              2        0.516871               0.159757
8         terzana              1        0.541413               0.786432
9           evito              3        0.496995               0.385126
10       pochetto              2        0.478458               0.709475
11     aspettasse              4        0.472732               0.454239
12      incallita              1        0.493386               0

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           usando              5        0.548076               0.275592
1       attingendo              5        0.518851               0.682865
2         attutito              2        0.532406               0.788198
3         fastidio              5        0.493311               0.330044
4           timidi              2        0.524822               0.343895
5          rendevo              2        0.534650               0.216164
6      moderazione              3        0.512165               0.158779
7     distruggendo              1        0.559664               0.024477
8        soverchie              3        0.501529               1.000000
9        indegnità              1        0.555111               0.484415
10       raramente              1        0.523127               0.165822
11     sopportarla              2        0.482982               0.964624
12          sudare              2        0.463610  

### TOPOS: TEMPORALE/PIOGGIA

In [85]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("temporale", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       liberare              5        0.671120               0.452324
1        disceso              4        0.647726               0.298265
2         finita              5        0.667395               0.916915
3          botta              5        0.688566               0.273555
4   furtivamente              3        0.654713               0.470736
5        ritegna              1        0.697140               0.696366
6      intenzion              1        0.696876               0.521147
7         ottima              2        0.645850               0.711754
8          mesta              1        0.696423               0.588854
9        delizie              3        0.665807               0.417813
10    guidardone              3        0.637063               0.997444
11       nemiche              3        0.621525               0.747850
12         dator         

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       spirituale              5        0.709564               0.285632
1       accessoria              5        0.581115               0.275858
2   intraprendesse              5        0.613386               0.000000
3       dependente              5        0.583525               0.031406
4        spiritual              5        0.589958               0.125890
5       amministra              2        0.579357               0.299037
6      emanciparsi              4        0.544779               0.083552
7     deformazione              3        0.552392               0.026189
8        spettasse              1        0.570580               0.465108
9      impacciarsi              3        0.534465               0.094617
10    riconoscerle              4        0.556947               0.151227
11         rendasi              3        0.546769               0.227045
12       conocchia              3        0.529756  

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           tuoni              5        0.451287               0.876520
1      declinando              4        0.407633               0.446246
2         bassura              5        0.402688               0.584496
3         fulmini              5        0.398818               0.295137
4          destai              1        0.441921               0.500644
5      acquazzoni              1        0.440450               0.096750
6           boati              2        0.414821               0.502593
7        nevicare              1        0.433031               0.893282
8   castelvecchio              5        0.407462               0.226459
9         stridìo              1        0.426493               0.091656
10          tuono              4        0.397727               0.402143
11   lampeggiante              3        0.383762               0.230599
12        diluvio              2        0.383399               0

In [86]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("pioggia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      distretta              1        0.850439               0.312194
1           cadi              2        0.828849               0.069170
2          poner              1        0.847620               0.890596
3         obedir              2        0.809364               0.643037
4         partia              3        0.789630               0.817605
5        negasse              3        0.787212               0.076473
6       seguiamo              3        0.783767               0.255776
7        gradito              2        0.788128               0.892546
8      disperati              1        0.837720               1.000000
9    affaticarsi              1        0.837489               0.158639
10        valuto              1        0.824821               0.090634
11      peccando              2        0.778955               0.660608
12    rispondesi         

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      stillando              5        0.622752               0.000000
1       grondano              5        0.616058               0.253931
2           neve              5        0.586435               0.413637
3   serpeggiando              5        0.617855               0.784556
4         gelata              3        0.580443               0.383526
5        bagnati              5        0.565109               0.059217
6    infiammando              5        0.577552               0.176808
7          mugge              4        0.557704               0.476034
8        spruzza              1        0.636744               0.533114
9          spume              1        0.630157               0.040099
10       scaglie              4        0.591915               0.803278
11      scendono              3        0.544578               1.000000
12   lucidissime              3        0.546413               0.931395
13    

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      scrosciava              5        0.485625               0.696875
1       nevischio              5        0.430406               0.422350
2         dirotta              5        0.460603               0.418622
3         durando              4        0.437634               0.394076
4         scrosci              4        0.432831               0.000000
5     sgocciolare              3        0.430080               0.331057
6     scrosciando              1        0.441856               0.007094
7      schiantava              5        0.440771               0.376097
8      scrosciare              3        0.429121               0.344466
9            plac              5        0.434124               0.456626
10      fischiata              4        0.435608               0.360433
11  rimpiccioliva              1        0.431513               0.469766
12     disseccato              1        0.437942               0

### TOPOS: GIOVINEZZA

In [87]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("giovinezza", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
La chiave non esiste nel modelloCADE_01_duecento_poetica_e_prosa Medievo_cade
###CADE_02_dolce_stil_novo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        temperata              5        0.734060               0.004173
1           virile              5        0.617439               0.437996
2        autentica              5        0.650605               0.497683
3      adolescenza              5        0.640688               0.273906
4          fervida              5        0.617007               0.000000
5        snellezza              4        0.602256               0.057157
6          tredici              1        0.656841               0.295457
7       procedeano              2        0.634057               0.324582
8      comunemente              3        0.613426               0.280225
9         seguenti              1        0.649732               0.398083
10        radicale              

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          etade              3        0.603625               0.128871
1         senili              5        0.576802               0.640403
2     disdegnosa              3        0.607818               0.333893
3     verginella              1        0.628088               0.260306
4      giovenili              4        0.568509               0.646593
5         toscar              1        0.627149               0.000000
6          cesse              2        0.594337               0.942071
7         diemmi              1        0.622556               0.787748
8       giovenil              1        0.621584               0.743841
9          moina              1        0.620565               0.132951
10        anciso              1        0.593746               0.165684
11    rinnovella              1        0.592799               0.612289
12    risplendea              1        0.591274               0.545816
13    

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    adolescenza              5        0.466109               1.000000
1       gioventù              5        0.440002               0.499316
2    sacrificate              2        0.441657               0.315756
3           punì              5        0.431243               0.187120
4       maturità              2        0.428494               0.346372
5       infanzia              2        0.425228               0.582683
6         goditi              5        0.452444               0.683117
7     avvampante              5        0.424859               0.834130
8       fatidica              4        0.431728               0.503789
9    caninamente              4        0.409737               0.000000
10  fanciullezza              3        0.412259               0.543427
11      sfiorire              1        0.422960               0.576802
12      rivelata              1        0.411845               0.341350
13    

In [88]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("giovane", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   svergognato              5        0.625668               0.523973
1       superbo              3        0.525341               0.644889
2         etade              4        0.562306               0.058923
3        puniti              4        0.536396               0.303016
4       vecchio              4        0.525177               0.000000
5      superbio              5        0.553412               0.405003
6       impazza              5        0.532896               0.117480
7       gastiga              1        0.578401               0.019703
8     richiesto              1        0.575904               0.329316
9       rimasto              1        0.571178               0.108610
10        pazzo              4        0.546471               0.626136
11      salvato              4        0.501489               0.590630
12     guastano              3        

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      vigilanzia              5        0.463984               0.000000
1           irene              5        0.446456               0.350151
2         cesario              2        0.451306               0.545017
3        marciano              1        0.484363               0.136733
4     inettissimo              5        0.449579               0.624696
5        eudossia              2        0.456715               0.001182
6     focosamente              3        0.440659               0.342537
7         arcadio              2        0.444287               0.686109
8        glicerio              1        0.463191               0.503825
9     torrismondo              4        0.426147               0.135588
10       teodosio              3        0.410815               0.759634
11       placidia              2        0.417256               0.605090
12         annone              1        0.427123               0

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        robusto              5        0.395221               0.529054
1       vecchino              1        0.403757               0.601827
2     occhialuto              2        0.378343               0.440834
3    adolescente              2        0.377675               0.105411
4       innamorò              4        0.394998               0.433351
5     abbronzato              5        0.381790               0.052337
6   giovanissimo              2        0.370391               0.182434
7   millantatore              5        0.372779               0.284688
8      tenentino              3        0.366819               0.000000
9   vergognatevi              1        0.377189               0.013909
10       paffuto              4        0.391543               0.812219
11     abruzzese              4        0.385184               0.335882
12      fervente              3        0.365622               0.455160
13    

### TOPOS: BACIO

In [89]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("bacio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
La chiave non esiste nel modelloCADE_01_duecento_poetica_e_prosa Medievo_cade
###CADE_02_dolce_stil_novo_cade###
La chiave non esiste nel modelloCADE_02_dolce_stil_novo_cade
###CADE_03_petrarchismo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   procacciare              3        0.780575               0.118206
1        ornare              1        0.829095               0.902730
2   trapassando              2        0.780642               0.060363
3       chiesia              1        0.821730               0.928256
4        teodor              1        0.820813               0.000000
5       incolpo              2        0.780118               0.270364
6       assolvi              1        0.819109               0.632052
7    immaginata              1        0.818443               0.861405
8   ringraziate              1        0.815165               0.178336
9        creare              1        0.81

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      cassettone              1        0.677256               0.010698
1          caschi              4        0.605448               0.437236
2        puntando              2        0.633351               0.105948
3        fasciato              4        0.593186               0.790693
4      procedessi              5        0.641406               0.007301
5           gelso              2        0.627659               0.089682
6         piegato              1        0.654084               0.813827
7          bracci              1        0.652905               0.451001
8        annodata              1        0.649879               1.000000
9         riposar              3        0.607564               0.386767
10        tiepida              1        0.632223               0.000000
11         sanato              1        0.619040               0.302809
12       indugiai              1        0.606882               0

### TOPOS: LUNA

In [90]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("luna", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   maravigliano              5        0.706392               0.001397
1      avvengono              5        0.682752               0.129382
2       cadranno              5        0.670054               0.846852
3         spessi              5        0.695706               0.764586
4           sole              5        0.664557               0.290255
5         stelle              5        0.626488               1.000000
6        formate              4        0.596837               0.361844
7           rare              5        0.607816               0.186906
8       oscurata              2        0.580884               0.000000
9   maravigliosi              2        0.607887               0.361379
10      apparire              2        0.572005               0.728977
11       pianeti              2        0.552034               0.930771
12          aere         

          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    specchiava              5        0.485544               0.930496
1         opaca              4        0.466351               0.254275
2        giamai              5        0.476055               0.267533
3   inargentata              5        0.458247               0.342124
4     azzurrina              1        0.481605               0.232961
5      tenebria              1        0.480842               0.466400
6     scoloriva              5        0.453509               0.832559
7     risplenda              2        0.452147               0.129190
8    illuminava              1        0.470939               0.742294
9          nubi              1        0.470009               0.518329
10   tramontare              2        0.478085               0.725879
11  tramontando              3        0.460445               0.661326
12    cavernosa              3        0.448261               0.313070
13      cantavi     

### TOPOS: SILENZIO

In [91]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("silenzio", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0     vincitrice              4        0.771784               0.431400
1   temperamento              5        0.791632               0.106062
2         lucano              5        0.767819               0.069609
3           cxxx              5        0.763084               0.075045
4     accrescere              3        0.760909               0.296508
5         cuopre              1        0.813450               1.000000
6      invecchia              2        0.793436               0.454935
7         mangio              4        0.760984               0.007357
8        liberai              2        0.789587               0.913866
9          finga              1        0.805491               0.783406
10        udirlo              1        0.775037               0.741759
11    mettendomi              2        0.758380               0.506178
12     ritornerà         

              Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       rispettando              5        0.453229               0.000000
1           scusare              5        0.439128               0.357758
2      interrompere              5        0.416435               0.309256
3             vinco              5        0.438158               0.269747
4        mescolasse              5        0.431113               0.310688
5        pitagorico              5        0.449174               0.421190
6         ufficioso              1        0.438059               0.097876
7        sospettare              1        0.433205               0.652101
8           diffido              4        0.412402               0.432489
9           scusate              1        0.428449               0.936215
10  dolcissimamente              3        0.409319               0.198435
11        mostrassi              1        0.417153               1.000000
12          fondava              2    

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        echeggiò              5        0.355163               0.464172
1        storditi              2        0.337247               0.507346
2        tacquero              1        0.344334               0.705174
3       arrendeva              1        0.338371               0.722313
4          stremo              1        0.337391               0.169404
5          sonoro              1        0.336579               0.170272
6         regnava              1        0.336236               0.187899
7          ampliò              3        0.336899               1.000000
8   raccoglimento              1        0.332419               0.056013
9      vestizione              1        0.330880               0.006776
10          pausa              4        0.348176               0.258876
11  sommessamente              3        0.340431               0.368062
12           colò              4        0.345698               0

### TOPOS: AMICIZIA

In [92]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amicizia", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0           quieta              4        0.713901               0.123042
1             fèro              5        0.696632               0.902946
2           impone              1        0.761665               0.226273
3          pregati              1        0.758728               0.615487
4         sostenne              4        0.689399               0.459786
5       credessero              3        0.710455               0.939409
6      contrastare              1        0.755005               0.534103
7           fraude              2        0.704382               0.062923
8           accusa              1        0.750997               0.479193
9          importa              5        0.707874               0.316412
10      salvazione              1        0.727663               0.478848
11      confessare              1        0.719401               0.408340

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         mantegna              3        0.489532               0.997404
1        contrassi              5        0.498707               0.000000
2       ingelosita              3        0.489312               0.003039
3        contrarsi              5        0.508270               0.287990
4       incostanza              2        0.494424               0.985766
5         razional              1        0.509379               0.863002
6        ambiscono              1        0.509177               0.970751
7       sottomessa              1        0.509072               0.202042
8          credula              1        0.507838               0.980217
9      correlativo              1        0.507100               0.020256
10     fidelissimo              3        0.476156               0.462868
11        maritale              3        0.475028               0.573493
12         estimai              2        0.467539  

            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0      sospettato              3        0.532269               0.378137
1      spiacevole              1        0.588714               0.326018
2       desistere              1        0.588607               0.975559
3       associati              1        0.585379               0.927456
4      differenze              1        0.583499               0.232561
5       edificate              3        0.519818               0.149486
6         venissi              4        0.522351               0.293498
7      certissimo              1        0.578815               0.999211
8   giustificarmi              1        0.577879               0.208392
9        fermezza              1        0.577485               0.961786
10      ardimento              2        0.496742               0.107836
11        rinnego              3        0.486279               0.495119
12         alfred              4        0.503854               0

In [93]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("amico", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0          caro              5        0.491693               0.344261
1       sguarda              1        0.509237               0.411600
2      grazioso              1        0.508981               0.368825
3        disvia              2        0.468096               0.776529
4        intero              5        0.456995               0.783489
5        mostro              1        0.495291               0.076710
6      rendesti              3        0.458543               0.308559
7           car              1        0.491129               0.492275
8       provedi              1        0.490796               0.374714
9          leal              1        0.488751               0.496109
10        danai              3        0.453210               0.996849
11         serv              4        0.452691               0.355275
12      giuochi              1        

                Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         affezionato              5        0.409690               0.252196
1        dilettissimo              5        0.417973               0.855210
2           ceremonie              5        0.388936               0.621212
3             collega              1        0.385029               0.361741
4        intercessore              1        0.379394               0.220654
5        sperimentato              2        0.375923               0.755802
6   intelligentissimo              3        0.379847               0.492415
7           carissimo              2        0.373389               0.424120
8           sapendolo              3        0.366387               0.000000
9             sentivo              1        0.367959               0.213939
10            buzzano              4        0.381460               0.206615
11          servitore              1        0.373178               1.000000
12          

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0       assistente              4        0.422793               0.326220
1       conoscente              5        0.443994               0.245826
2         modenese              4        0.399187               0.677106
3            occam              3        0.395173               0.095744
4         biografo              1        0.397931               0.298447
5   informatissimo              1        0.397924               0.000000
6         tanghero              1        0.397543               1.000000
7       avarissimo              3        0.384622               0.271753
8        livornese              2        0.401103               0.298337
9           turati              1        0.394329               0.090607
10          walter              4        0.401557               0.167411
11         salvami              4        0.399158               0.229700
12     ergastolano              1        0.396178  

### TOPOS: SOLITUDINE

In [94]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("solitudine", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
La chiave non esiste nel modelloCADE_01_duecento_poetica_e_prosa Medievo_cade
###CADE_02_dolce_stil_novo_cade###
La chiave non esiste nel modelloCADE_02_dolce_stil_novo_cade
###CADE_03_petrarchismo_cade###
La chiave non esiste nel modelloCADE_03_petrarchismo_cade
###CADE_04_boccaccio_e_umanesimo_cade###
            Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        fuggiamo              1        0.531490               0.321805
1       rimozione              5        0.491698               0.020885
2       convivono              4        0.480111               0.383873
3      frequentia              5        0.475840               0.552980
4         inerzia              1        0.521425               0.387451
5      dispiacemi              3        0.485000               0.276944
6          inezia              1        0.518521               0.595518
7         rognoso              1        0.514228               0.27

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        sterilità              5        0.431749               0.230655
1             dini              2        0.425484               0.479353
2   disoccupazione              2        0.423161               0.670081
3      inavvertito              5        0.414558               0.596004
4        pigliarne              5        0.421454               0.212064
5        immensità              1        0.428844               0.387785
6     disturbavano              4        0.416632               0.777215
7         vegliano              1        0.425821               0.321654
8       solitudini              1        0.419585               0.422990
9          dissipò              3        0.411121               0.000000
10        silenzio              4        0.435830               0.541561
11     desolazione              4        0.419503               0.004617
12       incombeva              1        0.417736  

In [95]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("solitario", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
La chiave non esiste nel modelloCADE_01_duecento_poetica_e_prosa Medievo_cade
###CADE_02_dolce_stil_novo_cade###
La chiave non esiste nel modelloCADE_02_dolce_stil_novo_cade
###CADE_03_petrarchismo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0         scola              4        0.664146               0.203903
1        seggia              2        0.705326               0.132962
2       nudrito              5        0.649783               0.303403
3     terebinto              1        0.728001               0.384355
4      famelici              1        0.727029               0.287301
5     verdeggia              4        0.645815               0.193870
6           ima              1        0.722019               0.203636
7        sorgea              2        0.655472               0.991972
8       pastori              2        0.688592               0.523797
9      sfogando              4        0.65

           Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0        passero              5        0.485640               1.000000
1        vagante              5        0.454698               0.463386
2   ristoratrice              2        0.447066               0.953555
3          opaco              3        0.447504               0.312114
4     addormenta              1        0.490589               0.166805
5     assiderato              1        0.486332               0.141127
6        arbusto              4        0.441397               0.388073
7    rintracciar              1        0.483379               0.050997
8         vedovo              1        0.480597               0.038398
9         pascea              5        0.436767               0.000000
10      frondoso              4        0.434493               0.439832
11     spasimava              1        0.447777               0.381536
12    seppelliva              1        0.444680               0.080597
13   m

### TOPOS: INFINITO

In [96]:
for periodo in lista_periodi_letterari_cade:
    try:
        print("###" + periodo + "###")
        df_periodo = similar_words_per_topos("infinito", 5, 10, periodo)
        print(df_periodo[['Words', 'Abs.frequency', 'Med.similarity', 'Med.similarity(norm.)']])
    except KeyError:
        print("La chiave non esiste nel modello" + periodo)

###CADE_01_duecento_poetica_e_prosa Medievo_cade###
          Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0    accostarsi              5        0.864635               1.000000
1       dolersi              2        0.817499               0.185799
2      dubitava              4        0.786499               0.039974
3        menami              1        0.822065               0.571590
4       volemmo              1        0.820895               0.041242
5        strema              5        0.808013               0.480759
6    accrescere              4        0.790724               0.139265
7   compunzione              5        0.825745               0.112366
8     opportuno              3        0.789139               0.373261
9    veggendolo              1        0.814030               0.332979
10      infinta              4        0.807741               0.149209
11   innocenzio              1        0.804084               0.270297
12       mangio              1        

             Words  Abs.frequency  Med.similarity  Med.similarity(norm.)
0   strabocchevole              5        0.468009               0.994656
1       sterminato              2        0.482533               1.000000
2           ascesi              4        0.450845               0.104989
3    multiplicando              5        0.468942               0.601810
4    picciolissimo              5        0.458285               0.581073
5          giubilo              4        0.447723               0.892807
6        colmarono              2        0.475668               0.891101
7       allegrezza              1        0.484812               0.330291
8       abjurarono              1        0.484356               0.149609
9        reclutare              1        0.484261               0.325323
10        concorse              1        0.475522               0.565212
11         immenso              2        0.434680               0.412311
12     cancellando              1        0.449076  